In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import serial.tools.list_ports
import re
import shutil
import numpy as np
import datetime
from scipy.optimize import curve_fit
import os
os.getcwd()

'C:\\Users\\serge\\Google_Drive\\Python_Coding\\Library'

In [3]:
while not os.getcwd().endswith('Python_Coding'):
    # move one up until the directory folder is 'Python_Coding'
    os.chdir('..')
    if os.getcwd().endswith(':\\'):
        break

if os.getcwd().endswith('Python_Coding'):
    Directory_address = os.getcwd() +'\\'
    Library_address = Directory_address + 'Library\\'
    print('Root Directory: \'%s\' \n  Library path: \'%s\' ' %(Directory_address, Library_address) )
else : 
    raise ValueError("Could not find the path of 'Python_Coding' folder ")

folder_address = Directory_address + "Inputs\\"
output_folder_address = Directory_address + "Outputs\\"

def_file_name= "data"
file_labels = ""
data_type= ".txt"

%run $Library_address\data_io.ipynb

Root Directory: 'C:\Users\serge\Google_Drive\Python_Coding\' 
  Library path: 'C:\Users\serge\Google_Drive\Python_Coding\Library\' 
Root Directory: 'C:\Users\serge\Google_Drive\Python_Coding\' 
  Library path: 'C:\Users\serge\Google_Drive\Python_Coding\Library\' 


<>:50: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-3-7fe4b96f6c08>:50: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif header_rows is 1:
<>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-3-416e5a7fc9f8>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif header_rows is 1:


In [4]:
def thermal_decay(t, t_decay, temp_initial, temp_final):
    return (temp_initial-temp_final) * np.exp(-t/t_decay) + temp_final

# Global Variables

In [5]:
output_subfolder_address = '\Outputs\\'
output_folder_address = Directory_address + output_subfolder_address

try: 
    SN_selected_output_folder_address = output_folder_address + SN_selected +'\\'
except:
    SN_selected_output_folder_address = output_folder_address 
    print('No SN is selected yet')

calibration_subfolder_address = '\Inputs\Calibration\\'
calibration_folder_address = Directory_address + calibration_subfolder_address

initialization_subfolder_address = '\Inputs\Initialization\\'
initialization_folder_address = Directory_address + initialization_subfolder_address

max_gc_current = 1100 # mA
max_frequency = 198 # THz
min_frequency = 190 # THz

min_comb_temperature = 16 # °C
max_comb_temperature = 100 # °C

No SN is selected yet


In [6]:
initialization_folder_address

'C:\\Users\\serge\\Google_Drive\\Python_Coding\\\\Inputs\\Initialization\\'

In [7]:
thermal_decay(14, 1 , 50, 25)

25.000020788217977

In [5]:
# TODO : USE THE 0.08 wait time with other cables
wait_time = 0.08 # seconds
wait_time = 0.10 # Andy Turk's cable needs more wait time 


In [6]:
def board_io(  command, 
             output = 0, # no. of characters to read
             wait_time = wait_time # waiting time after writing a command before-->
             #reading the output to make sure that the board has enough time to respond
            ):
    
    bytes_to_write = bytes( command + '\r', encoding = 'utf-8')
    n_bytes_written = smartboard.write( bytes_to_write )
    
    time.sleep(wait_time)
    
    if output == 0:
        output = smartboard.in_waiting
        
    bytes_read = smartboard.read(output)
        
    return bytes_read.decode().split('\r\n')[:-1]

# Calibrate Comb RTD

In [10]:
# Turns off the laser (gc=0) scans the bench2 temperature and measures comb and fscs resistances
# bench2 is disabled after the scan
def scan_bench_for_comb_and_fscs ( 
                                temp_min = 25, temp_max = 35, temp_step = 0.1,
                                bench_wait_time = 1, # wait time after setting the bench2 target temperature et each scan
                               ):
    
    k_bench = get_pid(heater ='bench2')
    print('PID for bench: ' + str(k_bench))
    
    # Turn off the GC
    gc_read = gain_current(0)
    print('Gain Current is %.3f mA' %gc_read)
    
    # Set Target Temperature and PID of bench
    target_c = temp_min #Celcius 
    
    # switch the bench to the temperature mode
    set_bench_mode('temp')
    heater_temp('bench2', target_c)  # Set Target T of bench
    board_io( 'bench2 pid %s %s %s' %( tuple(k_bench) )) # Set PID of bench
    board_io( 'bench2 enable on')# Enable bench
    
    # Read the Temperatue of bench2 to make sure it is at the target temperature
    temp_actual = heater_temp('bench2')
    
    # Disable the comb and fscs heater closed loops as they are not in use, but being measured
    board_io( 'fscs enable off'); heater_heat('fscs', 0)
    board_io( 'comb enable off'); heater_heat('comb', 0)
    
    temp_range = range(temp_min, temp_max + temp_step, temp_step)
    temp = []; temp_diff = []; rtd_r = []; fscs = []; temp_comb =[] ; temp_fscs = []
   

    
    user_input = 0; i=0;

    start_time = time.time()
    
    while( user_input != '1' ):
        print( '%.3f °C (target T: %.3f °C) ΔT=%.3f C' %(temp_actual, target_c, abs(target_c-temp_actual)))
        # Get confirmation from the user that the actual temperature is good enough to contiune
        user_input = input("Okay to continue ?: (0 or 1) ")
        i+=1 # allows only 10 steps in this while loop
        if i > 10:
            break
        
        temp_actual = heater_temp('bench2')
    
    print('Starting measurement')
    # scan over the bench2 target temperatures and perform measurements
    for temp_write in temp_range:

        heater_temp('bench2', temp_write) 
        time.sleep(bench_wait_time) # so that temperature stabilizes 

        # Measure bench2 Temperature
        temp_actual = heater_temp('bench2')
        temp.append(temp_actual)
        temp_diff.append( abs(temp_write-temp_actual) ) # difference between the input and the read temp value
        
        # Measure Comb (RTD) Temperature
        temp_comb_read = heater_temp('comb')
        temp_comb.append(temp_comb_read)
        
        # Measure RTD Resistance
        rtd_r_read = board_io( 'comb rtd_r')[-1]
        rtd_r_read = float ( re.findall('[0-9-.]+' , rtd_r_read)[0]) # extract the resistance from the returned string
        rtd_r.append(rtd_r_read)
        
        # Measure FSCS Temperature
        temp_fscs_read = heater_temp('fscs')
        temp_fscs.append(temp_fscs_read)
        
        # Measure FSCS Thermistor Resistance
        fscs_read = board_io( 'fscs thermistor_r')[-1]
        fscs_read = float ( re.findall('[0-9-.]+' , fscs_read)[0]) # extract the resistance from the returned string
        fscs.append(fscs_read)
        

        print( '%.3f °C (target T: %.3f °C) ΔT=%.3f C' %(temp_actual, temp_write, abs(temp_write-temp_actual)))

    print('Max difference between target & actual T = %.3f °C' %max(temp_diff))

    # Turn off the bench
    board_io( 'bench2 enable off')
    
    elapsed_time = time.time() - start_time
    print('Calibration Scan took %.1f seconds (= %.2f hours)' %(elapsed_time, elapsed_time/3600) )
                           
    # Plot the RTD - Temperature curve
    theme = theme_pyplot()
    data_title = 'bench2 at %.1f °C, PID: %.2f, %.2f, %.2f, wait time: %d s' %(target_c, k_bench[0], k_bench[1], k_bench[2], bench_wait_time)
    temp_axis_label = 'Temperature (°C)'; 
    rtd_r_axis_label = 'RTD (ohm)'; fscs_r_axis_label = 'FSCS Thermistor (ohm)'
    comb_T_axis_label = 'Comb Temperature (°C)'; fscs_T_axis_label = 'FSCS Temperature (°C)'

  
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize = (theme.figsize[0], theme.figsize[1]*2 ))
    plt.rcParams['axes.linewidth'] = 3 #set the value globally
    
    p = np.polyfit(temp, rtd_r,  1) # linear fit of RTD resistance
    R_RT = p[1] + p[0]*25 # RTD resistance at 25 °C
    TCR = p[0]/R_RT # RTD resistance temperature coefficient
    
    
    theme.line_kwargs['label'] = 'Measurement'
    ax1.plot(temp, rtd_r, **theme.line_kwargs)
    theme.line_kwargs['ls'] = '--'; theme.line_kwargs['marker'] = ''; theme.line_kwargs['label'] = 'Linear Fit'
    theme.line_kwargs['color'] = 'red'
    ax1.plot(temp, p[1] + np.asarray(temp)*p[0], **theme.line_kwargs) # linear fit of RTD resistance
    
    ax1.set_xlabel(temp_axis_label, fontsize = theme.fontsize); ax1.set_ylabel(rtd_r_axis_label, fontsize = theme.fontsize)
    ax1.grid(**theme.grid_kwargs); ax1.tick_params(**theme.tick_kwargs); 
    ax1.legend(**theme.legend_kwargs)
    
    theme.line_kwargs['label'] = 'FSCS Thermistor Resistance (ohms)'; theme.line_kwargs['ls'] = '-';
    ax2.plot(temp, fscs, **theme.line_kwargs)
    ax2.update_from(ax1)
    ax2.set_xlabel(temp_axis_label, fontsize = theme.fontsize); ax2.set_ylabel(fscs_r_axis_label, fontsize = theme.fontsize)
    ax2.grid(**theme.grid_kwargs); ax2.tick_params(**theme.tick_kwargs)
    
    theme.line_kwargs['label'] = ''; 
    ax3.plot(temp, temp_comb, **theme.line_kwargs)
    theme.line_kwargs['color'] = 'blue'; theme.line_kwargs['ls'] = '--';
    ax3.plot(temp, temp, **theme.line_kwargs)
    ax3.set_xlabel(temp_axis_label, fontsize = theme.fontsize); ax3.set_ylabel(comb_T_axis_label, fontsize = theme.fontsize)
    ax3.grid(**theme.grid_kwargs); ax3.tick_params(**theme.tick_kwargs)
    
    theme.line_kwargs['label'] = ''; theme.line_kwargs['ls'] = '-';
    ax4.plot(temp, temp_fscs, **theme.line_kwargs)
    theme.line_kwargs['color'] = 'blue'; theme.line_kwargs['ls'] = '--';
    ax4.plot(temp, temp, **theme.line_kwargs)
    ax4.set_xlabel(temp_axis_label, fontsize = theme.fontsize); ax4.set_ylabel(fscs_T_axis_label, fontsize = theme.fontsize)
    ax4.grid(**theme.grid_kwargs); ax4.tick_params(**theme.tick_kwargs)
    
    plt.tight_layout()
    plt.show()
    
    print('RTD(T) = %.6f(1 + %.6f(T - 25 °C) ohms ) [Fit Result]' %(R_RT, TCR) )

    TCR_firmware = float( re.findall('[0-9-.]+' , board_io( 'comb tcr')[-1])[0] ); R_RT_firmware = float( re.findall('[0-9-.]+' , board_io( 'comb r_rt')[-1])[0] )
    print('RTD(T) = %.6f(1 + %.6f(T - 25 °C) ohms ) [Firmware]' %(R_RT_firmware, TCR_firmware) )

    data_title = 'Calibrating Comb of %s' %SN_selected
    simple_email(email_subject='Experiment %s is done' %data_title, email_body='%s' %data_title)
    
    # Set the Comb RTD Resistor Parameters to the firmware
    board_io( 'comb r_rt %f' %R_RT); board_io( 'comb tcr %f' %TCR)
    print('Fit values have been set in the firmware')
    
    return temp, rtd_r, fscs, R_RT, TCR, R_RT_firmware, TCR_firmware

In [11]:
def scan_comb_fscs_open_loop( 
                            comb_heat_min = 0, comb_heat_max = 15, comb_heat_step = 1,
                            fscs_heat_min = 0, fscs_heat_max = 15, fscs_heat_step = 1,
                             target_c = 25,
                        bench_wait_time = 5 # wait time after setting the bench2 target temperature et each scan
                           ):

    # Turn off the GC
    gc_read = gain_current(0)
    print('Gain Current is %.3f mA' %gc_read)
    
    # Set Target Temperature and PID of bench
    
    k_bench = get_pid(heater ='bench2')
    
    heater_temp('bench2', target_c)  # Set Target T of bench
    board_io( 'bench2 pid %s %s %s' %( tuple(k_bench) )) # Set PID of bench
    board_io( 'bench2 enable on')# Enable bench
                       
    # Disable the comb and fscs closed loop heaters as they are not in use, open loop is in use
    board_io( 'fscs enable off'); heater_heat(fscs, 0)
    board_io( 'comb enable off'); heater_heat(comb, 0)
    
    # Read the Temperatue of bench2 to make sure it is at the target temperature
    temp_actual = heater_temp('bench2')
   
    comb_heat_range = range(comb_heat_min, comb_heat_max + comb_heat_step, comb_heat_step)
    fscs_heat_range = range(fscs_heat_min, fscs_heat_max + fscs_heat_step, fscs_heat_step)

    temp_comb = []; rtd_r = []; temp_fscs = []; i_tecb = []; power_tecb = []

    user_input = 0; i=0;
                       
    while( user_input != '1'):
        print( '%.3f °C (target T: %.3f °C) ΔT=%.3f C' %(temp_actual, target_c, abs(target_c-temp_actual)))
        # Get confirmation from the user that the actual temperature is good enough to contiune
        user_input = input("Okay to continue ?: (0 or 1) ")
        i+=1 # allows only 10 steps in this while loop
        if i > 10:
            break
        
        temp_actual = heater_temp('bench2')
                           
    start_time = time.time()
    print('Starting measurement')
    for comb_heat_write in comb_heat_range:

        heater_heat(comb, comb_heat_write)  # give heat to the comb
        time.sleep(bench_wait_time) # so that comb temperature stabilizes 

        # Measure Comb (RTD) Temperature
        temp_comb_read = heater_temp('comb')
        temp_comb.append(temp_comb_read)
        
        
    for fscs_heat_write in fscs_heat_range:
    
        heater_heat('fscs', fscs_heat_write) # give heat to fscs

        time.sleep(bench_wait_time) # so that fscs temperature stabilizes 

        # Measure FSCS Temperature
        temp_fscs_read = heater_temp('fscs')
        temp_fscs.append(temp_fscs_read)
        
        # Measure FSCS Current (If the T-E-C-B+, T-E-C-B- pins are used)
        i_tecb_read = board_io( 'adc itecb')[-1]
        i_tecb_read = float ( re.findall('[0-9-.]+' , i_tecb_read)[0])
        i_tecb.append(i_tecb_read)
    
    # FSCS heater resistance
    fscs_heater_r = board_io( 'fscs coarse_r')[-1]
    fscs_heater_r = float ( re.findall('[0-9-.]+' , fscs_heater_r)[0])
    
    power_tecb = [1000*i*i*fscs_heater_r for i in i_tecb] #mW, actual electrical power used on the fscs heater resistor
                            
    # Turn off the bench
    board_io( 'bench2 enable off')[-1]; print('bench2 is off now') 

    # Turn off the comb and fscs open loop heaters
    heater_heat('comb', 0); print('Comb heater is off now')
    heater_heat('fscs', 0); print('FSCS heater is off now')                   

    elapsed_time = time.time() - start_time
    print('Scanning took %.1f seconds (= %.2f hours)' %(elapsed_time, elapsed_time/3600) )
                            
    # Plot the RTD - Comb power curve
    # Plot the RTD - Temperature curve
    theme = theme_pyplot()
    data_title = 'T = %.1f °C, PID: %.2f, %.2f, %.2f, wait time: %d s' %(target_c, k_bench[0], k_bench[1], k_bench[2], bench_wait_time)
    comb_heat_axis_label = 'Comb Heating Power (mW)'; fscs_heat_axis_label = 'FSCS Heating Power (mW)'
    temp_comb_axis_label = 'Comb Temperature (°C)'; temp_fscs_axis_label = 'FSCS Temperature (°C)'
    power_tecb_axis_label = 'Applied Power (mW)'; 


    fig, ((ax1,ax2), (ax3,ax4) ) = plt.subplots(2, 2, figsize = theme.figsize)
    plt.rcParams['axes.linewidth'] = 3 #set the value globally
    
    ax1.set_xlabel(comb_heat_axis_label, fontsize = theme.fontsize); ax1.set_ylabel(temp_comb_axis_label, fontsize = theme.fontsize)
    ax1.grid(**theme.grid_kwargs); ax1.tick_params(**theme.tick_kwargs); ax1.legend(**theme.legend_kwargs)
    
    ax2.update_from(ax1)
    ax2.set_xlabel(fscs_heat_axis_label, fontsize = theme.fontsize); ax2.set_ylabel(temp_fscs_axis_label, fontsize = theme.fontsize)
    
    ax4.set_xlabel(fscs_heat_axis_label, fontsize = theme.fontsize); ax4.set_ylabel(power_tecb_axis_label, fontsize = theme.fontsize)

    comb_open_loop_fit = np.polyfit(comb_heat_range, temp_comb, 1) # linear fit of comb open loop temperature-heat scan
    print('Comb(T) = %.3f °C + %.3f * Power/mW )' %(comb_open_loop_fit[1], comb_open_loop_fit[0]) )
    
    fscs_open_loop_fit = np.polyfit(fscs_heat_range, temp_fscs, 1) # linear fit of fscs open loop temperature-heat scan
    print('FSCS(T) = %.3f °C + %.3f * Power/mW )' %(fscs_open_loop_fit[1], fscs_open_loop_fit[0]) )

    theme.line_kwargs['label'] = 'Comb Open Loop'
    ax1.plot(comb_heat_range, temp_comb, **theme.line_kwargs)
    ax1.plot(comb_heat_range, comb_open_loop_fit[1] + comb_open_loop_fit[0]*np.asarray(comb_heat_range), **theme.line_kwargs)

    theme.line_kwargs['label'] = 'FSCS Open Loop'
    ax2.plot(fscs_heat_range, temp_fscs, **theme.line_kwargs)
    ax2.plot(fscs_heat_range, fscs_open_loop_fit[1] + fscs_open_loop_fit[0]*np.asarray(fscs_heat_range), **theme.line_kwargs)
    
    ax4.plot(fscs_heat_range, power_tecb, **theme.line_kwargs)

    
    plt.tight_layout()
    plt.show()
    
    return comb_heat_range, temp_comb, fscs_heat_range, temp_fscs

In [12]:
# This function performs a step reponse of the bench2 from an inital temperature to a final temperature
def bench_step_response(temp_initial = 25,
                        temp_final = 27,
                        n_steps = 100, # number of steps to monitor the bench2 temperature
                        k_bench=[]
                       
                       ):

    if len(k_bench) == 0: # if user has not defined it
        k_bench = get_pid(heater ='bench2')
        print('PID for bench: ' + str(k_bench))
        
    # Set Target Temperature and PID of bench
    heater_temp('bench2', temp_initial)  # Set Target T of bench
    board_io( 'bench2 pid %s %s %s' %( tuple(k_bench) ))

    # Enable bench
    board_io( 'bench2 enable on' )
    
    n_range = range(0, n_steps + 1, 1)

    temp_time = []; time_measured = []
    
    temp_actual = heater_temp('bench2')
    
    user_input = 0; i=0;
                       
    while( user_input != '1'):
        print( '%.3f °C (target T: %.3f °C) ΔT=%.3f C' %(temp_actual, temp_initial, abs(temp_initial-temp_actual)))
        # Get confirmation from the user that the actual temperature is good enough to contiune
        user_input = input("Okay to continue ?: (0 or 1) ")
        i+=1 # allows only 10 steps in this while loop
        if i > 10:
            break
        
        temp_actual = heater_temp('bench2')

    # Set to the end temperature
    heater_temp('bench2', temp_final)  # Set Target T of bench
    start_time = time.time()
    print('Starting measurement')
    for n in n_range:

        time_measured.append( time.time() - start_time )

        temp_actual = heater_temp('bench2')
        temp_time.append(temp_actual)

    elapsed_time = time.time() - start_time

    # reset the bench
    heater_temp('bench2', temp_initial)

    # Plot the Temperature
    data_title = 'Initial = %.1f °C, Final = %.1f °C. PID: %.2f, %.2f, %.2f' %(temp_initial, temp_final, k_bench[0], k_bench[1], k_bench[2])
    x_axis_label = 'Time (s)'; y_axis_label = 'Temperature (°C)'
    
    theme = theme_pyplot()
    fig, ax = plt.subplots(figsize=theme.figsize)
    plt.rcParams['axes.linewidth'] = 3 #set the value globally

    theme.line_kwargs['label'] = data_title
    ax.plot(time_measured, temp_time, **theme.line_kwargs)
    ax.grid(**theme.grid_kwargs); ax.tick_params(**theme.tick_kwargs); ax.legend(**theme.legend_kwargs)

    ax.set_xlabel(x_axis_label, fontsize = theme.fontsize); ax.set_ylabel(y_axis_label, fontsize = theme.fontsize)
    plt.show()
    
    return time_measured, temp_time

In [13]:
10 % 4

2

In [3]:
# This function monitors stedy-state or free-cooling reponse of any heaters at a set temperature
def heater_steady_response(heater = 'comb',
                      is_heater_on = 1,
                      t_initial = 25,
                           t_final = 30,
                        n_steps = 100, # number of steps to monitor the temperature
                        k_heater=[], # default pid parameters
                           step_wait_time = 0, # wait time between each step
                       user_input = '1'
                       ):
    
    # Set Target Temperature and PID of heater
    if len(k_heater) == 0: # if user has not defined it
        k_heater = get_pid(heater = heater)
        print('PID for ' + heater + ' : ' + str(k_heater))
    if t_final == 0: # if user does not want to change the temperature; steady-state response
        t_final = t_initial
        
    heater_temp(heater, t_initial)
    board_io( heater + ' pid %s %s %s' %( tuple(k_heater) ))
    if is_heater_on:
        # Enable bench
        board_io( heater + ' enable on')
    else :
        # Enable bench
        board_io( heater + ' enable off')

    n_range = range(0, n_steps + 1, 1)

    temp_time = []; temp_time_average = []; time_measured = []; temp_sled = []; heat_time=[]
    temp_ext1 = [] ; temp_ext2 = []
        
    temp_actual = heater_temp(heater)
    
    i=0;
                       
    while( user_input != '1'):
        print( '%.3f °C (target T: %.3f °C) ΔT=%.3f C' %(temp_actual, t_initial, abs(t_initial-temp_actual)))
        # Get confirmation from the user that the actual temperature is good enough to contiune
        user_input = input("Okay to continue ?: (0 or 1) ")
        i+=1 # allows only 10 steps in this while loop
        if i > 10:
            break
        
        temp_actual = heater_temp(heater)
    
    heater_temp(heater, t_final)
    #if not is_heater_on:
        # Disable bench2 if user wants to monitor cooling
        #board_io( heater + ' enable off')
    
    start_time = time.time()
    print('Starting measurement')
    
    for n in n_range:

        
#         if n == 150:
#             gain_current(490)
#         if n == 300:
#             gain_current(500)
            
        time_measured.append( time.time() - start_time )

        temp_actual = heater_temp(heater)
#        temp_average_actual = heater_temp(heater, target = -500)
        temp_average_actual = heater_temp(heater)
        temp_time.append(temp_actual)
        temp_time_average.append(temp_average_actual)
        
        heat_time.append( heater_heat(heater) )
        
        # TPS01 Thorlabs Temperature Sensor
        try:
            temp_ext1_read = tsp01.measure_temperature('th0')
            temp_ext2_read = tsp01.measure_temperature('th1')

        except:
            temp_ext1_read = 0
            temp_ext2_read = 0
        
        temp_ext1.append(temp_ext1_read)
        temp_ext2.append(temp_ext2_read)
        
        time.sleep(step_wait_time)
        
#         if n%10 == 0:
#             gain_current(750-2*n/10)
    
#     gain_current(750)
    
    elapsed_time = time.time() - start_time
    print('Measurement took %.1f seconds' %elapsed_time )
    heater_temp(heater, t_initial)
    

    squared_error = [(t_final - i)**2 for i in temp_time]
    variance =  sum(squared_error)/len(temp_time)
    std_deviation = variance**0.5
    # Plot the Temperature
    data_title = 'Std Deviation = %.4f °C,Target = %.1f °C' %(std_deviation, t_final)
    x_axis_label = 'Time (s)'; temp_axis_label = 'Temperature (°C)'; 
    
    if heater == 'bench2':
        heat_axis_label = 'Current (A)'
    else :
        heat_axis_label = 'Heat (mW)'
    
    theme = theme_pyplot()
    theme.figsize = (10,8)
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=theme.figsize)
    plt.rcParams['axes.linewidth'] = 3 #set the value globally

    theme.line_kwargs['label'] = data_title #, theme.line_kwargs['ms'] = 6
    ax1.plot(time_measured, temp_time, **theme.line_kwargs)
    
    theme.line_kwargs['label'] = 'Average' ; theme.line_kwargs['color'] = 'g'
    ax1.plot(time_measured, temp_time_average, **theme.line_kwargs)
    
    if temp_ext1_read != 0:

        theme.line_kwargs['label'] = 'External Temp 1 (°C)' ; #theme.line_kwargs['color'] = 'g'
        ax1.plot(time_measured, temp_ext1, **theme.line_kwargs)

    if temp_ext2_read != 0:
    
        theme.line_kwargs['label'] = 'External Temp 2 (°C)' ; #theme.line_kwargs['color'] = 'g'
        ax1.plot(time_measured, temp_ext2, **theme.line_kwargs)

    
    if not is_heater_on: # if a free cooling is desired, we can fit it
        # exponential fit to the thermal decay
        popt, pcov = curve_fit(thermal_decay, time_measured, temp_time)
        t_decay, temp_initial, temp_final = popt
        ax1.plot(np.asarray(time_measured), thermal_decay(np.asarray(time_measured), *popt), 'r--', label="%.3f*exp(-t/%.3f) + %.3f" %(temp_initial-temp_final, t_decay, temp_final) ) 

    theme.line_kwargs['color'] = 'r'
    ax2.plot(time_measured, heat_time, **theme.line_kwargs)
    
    theme.legend_kwargs['fontsize'] = theme.fontsize/2
    theme.legend_kwargs['title'] = '%s, PID: %.2f, %.2f, %.2f' %(heater, k_heater[0], k_heater[1], k_heater[2])
    ax1.grid(**theme.grid_kwargs); ax1.tick_params(**theme.tick_kwargs); ax1.legend(**theme.legend_kwargs)
    ax2.grid(**theme.grid_kwargs); ax2.tick_params(**theme.tick_kwargs); ax2.legend(**theme.legend_kwargs)
    

    ax1.set_xlabel(x_axis_label, fontsize = theme.fontsize); ax1.set_ylabel(temp_axis_label, fontsize = theme.fontsize)
    ax2.set_xlabel(x_axis_label, fontsize = theme.fontsize); ax2.set_ylabel(heat_axis_label, fontsize = theme.fontsize)
    plt.tight_layout()
    plt.show()
    
    output = time_measured, temp_time, heat_time, temp_ext1, temp_ext2
    output_labels = ['Time (s)', 'Temperature (°C)' , heat_axis_label, 'External Temp 1 (°C)', 'External Temp 2 (°C)']
    
    data_pd = pd.DataFrame([output], columns=output_labels)
    df_dict = dict(zip( output_labels, output))
    data_pd = pd.DataFrame.from_dict( df_dict, orient='index').transpose()
    measurement_types = 'Bench Calibration Tests'
    
    data_title = 'Bench Calibration Test_%s' %(SN_selected); date_label = time.strftime("_%Y-%m-%d_%H-%M-%S")
    
    output_address = export_pd(data_pd, folder_address = SN_selected_output_folder_address + measurement_types, file_name = data_title, header=True, date_label = date_label)
    email_results(email_subject = data_title, attach_file_address = output_address)
    
    return output

# Bias Temperature, Scan Gain Chip Current

In [15]:
def bias_T_scan_gc(data_title='',
                   gc_min = 49 , gc_max = 50, gc_step = -1, # Set min, max gain current, the step size in mA
                   temp_min = 22, temp_max = 30, temp_step = -1, # Set min, max temperature, the step size in Celcius
                   heater = 'bench2', # the heater to scan T
                   bench_enabled = 'on', # enable bench
                   temp_fscs = 25, fscs_enabled = 'on', # fscs temperature 
                   temp_comb = 25, comb_enabled = 'on', # comb temperature
                   bench_wait_time = 3, # wait time between each step while scanning the temperature so that the loop stabilizes
                   gc_wait_time = 0.5, # wait time at each gc step during the scan
                   mode_lock = 1 , 
                   is_maximize_mzis = 0, # to maximize the optical power with MZIs 
                   min_bench_temp = 15, max_bench_temp = 60, #for  safety check during mode_lock
                   is_print_scan = 0, # whether to print the results at each step during the scan
                   measurement_types = '', is_email_results = 1
):
    
    if data_title == '':
        raise ValueError('Always provide data_title')
    print('Data title: %s' %data_title)
    
    print_thermal_optical_state()
        
    start_time = time.time()

    # Prepare the Temperature and Gain Current Ranges
    temp_range = np.arange(temp_max, temp_min + temp_step, temp_step) if temp_step < 0 else np.arange(temp_min, temp_max, temp_step)    
    gc_range = np.arange(gc_max, gc_min + gc_step, gc_step) if gc_step < 0 else np.arange(gc_min, gc_max + gc_step, gc_step) 

    temp_bench2 = temp_range[0] # target is the first temperature value
    
    heater_temp('bench2', temp_bench2); board_io( 'bench2 enable '+ bench_enabled)     # Enable bench
    heater_temp('fscs', temp_fscs); board_io( 'fscs enable ' + fscs_enabled) # Enable the heater
    heater_temp('comb', temp_comb); board_io( 'comb enable ' + comb_enabled) # Enable the heater

    
    time_measured = [] ; vgain_temp = []; vl_temp = []
    power_temp = []; temp_diff = []; frequency_temp = []; gc = []; mod_i_oop_temp = []; mod_q_oop_temp= [];mod_i_apd_temp = []; mod_q_apd_temp= []; mod_i_vgain_temp = []; mod_q_vgain_temp = [];
    frequency_read = 0 ; temp = []; 
    fscs_heat_temp = []; comb_heat_temp = []
    T_fscs_actual_temp = []; T_bench_actual_temp = []; T_comb_actual_temp = [] 
    mpd1_temp = []; mpd2_temp = []; mpd3_temp = []; mpd4_temp = []; mpd_oop_temp = []; mpd_orp_temp = []; mpd_ap_temp = []; mpd_bp_temp = [];
    mzi1_max_temp = [] ; mzi2_max_temp = [] ; mzi3_max_temp = []; mzi4_max_temp = []
    
    temp_ext1_temp = []; temp_ext2_temp =[] # external temperatures measured by Thorlabs TPS01
    
    try :
        power_meter.sense.power.dc.range.upper = 60/1000 # 60 mW, set to maximum range of optical power by PM100D
    except:
        print('Power meter range cannot be set')
    
    print('Temperature Range: %s' %temp_range)

    for temp_write in temp_range:

        print_thermal_optical_state()
        gc_read = gain_current(gc_range[0])
        
        heater_temp('bench2', temp_write) 
        time.sleep(bench_wait_time) # so that temperature stabilizes 

        temp_actual = heater_temp('bench2')

        temp.append(temp_actual)
        temp_diff.append( abs(temp_write - temp_actual) ) # difference between the input and the read temp value

        print('T = %.3f °C (Target: %.3f °C)' %(temp_actual, temp_write) )

        gc = []; vgain = []; vl = []
        power = []; mpd = []; frequency = []; mod_i_oop = []; mod_q_oop = []; mod_i_apd = []; mod_q_apd = []; mod_i_vgain = []; mod_q_vgain = [];
        fscs_heat=[]; comb_heat=[]
        T_fscs_actual = []; T_bench_actual = []; T_comb_actual = [] 
        mpd1 = []; mpd2 = []; mpd3 = []; mpd4 = []; mpd_oop = []; mpd_orp = []; mpd_ap = []; mpd_bp = [];
        mzi1_max = [] ; mzi2_max = [] ; mzi3_max = []; mzi4_max = []
        
        temp_ext1 = [] ; temp_ext2 = []

        expected_measurement_duration = (2.4+gc_wait_time)*(gc_min-gc_max)/gc_step
        print('Starting Locked PI %s, ~%s s (%s)' %(time.strftime("on %Y_%m_%d at %H:%M"), expected_measurement_duration, str(datetime.timedelta(seconds=expected_measurement_duration)) ) ) 

        for gc_write in gc_range:


            gc_read = gain_current(gc_write); gc.append(gc_read); 
            if is_print_scan:
                print('Gain Current is %.3f mA' %gc_read)
            
            time.sleep(gc_wait_time) # so that the spectrum stabilizes 
            time_measured.append( time.strftime("%Y-%m-%d_%H:%M:%S") )
            
            # MZI heat values
            T_mzi1_actual_read = heater_temp('mzi 1'); T_mzi2_actual_read = heater_temp('mzi 2'); T_mzi3_actual_read = heater_temp('mzi 3'); T_mzi4_actual_read = heater_temp('mzi 4');
            
            if is_maximize_mzis:
                # optimize MZIs
                mzi_maximize_wait_time = 1
                mzi1_heat_read = maximize_power_via_mzi(1, 0.5, mzi_maximize_wait_time); mzi2_heat_read = maximize_power_via_mzi(2, 0.5, mzi_maximize_wait_time)
                mzi3_heat_read = maximize_power_via_mzi(3, 0.5, mzi_maximize_wait_time); mzi4_heat_read = maximize_power_via_mzi(4, 0.5, mzi_maximize_wait_time)
            else :
                # no optimization, append current values
                mzi1_heat_read = T_mzi1_actual_read; mzi2_heat_read = T_mzi2_actual_read; mzi3_heat_read = T_mzi3_actual_read; mzi4_heat_read = T_mzi4_actual_read;
                
            mzi1_max.append(mzi1_heat_read); mzi2_max.append(mzi2_heat_read); mzi3_max.append(mzi3_heat_read); mzi4_max.append(mzi4_heat_read)

            # FSCS Actual Temperature
            T_fscs_actual_read = heater_temp('fscs')
            T_fscs_actual.append(T_fscs_actual_read)
            
            # TPS01 Thorlabs Temperature Sensor
            try:
                temp_ext1_read = tsp01.measure_temperature('th0')
                temp_ext2_read = tsp01.measure_temperature('th1')
                
            except:
                temp_ext1_read = 0
                temp_ext2_read = 0
            
            temp_ext1.append(temp_ext1_read)
            temp_ext2.append(temp_ext2_read)

            # bench2 Actual Temperature
            T_bench_actual_read = heater_temp('bench2')
            T_bench_actual.append(T_bench_actual_read)
            
            temp_diff.append( abs(temp_write - T_bench_actual_read) ) # difference between the input and the read temp value
            

            # Comb Actual Temperature
            T_comb_actual_read = heater_temp('comb')
            T_comb_actual.append(T_comb_actual_read)

            temp_diff.append( abs(temp_write - T_bench_actual_read) ) # difference between the input and the read temp value

            ### Measurements ###
            
            vgain.append(float( re.findall('[0-9-.]+', board_io('adc vgain')[-1])[0] ))
            vl.append( float( re.findall('[0-9-.]+', board_io('gc vl')[-1])[0] ) )
            
            
            power_read = read_power()

            power.append(power_read)

            # Optical Power and Peak frequency from Bristol 438A Wavelength Meter
            #power_bristol.append(float(bristol.getSimpleMsg(b':READ:SCAL:ALL?').split(', ')[3]))
            frequency_read = read_frequency()
            frequency.append(frequency_read)

            # Read Photodiodes
            mpd_oop_read, mpd_orp_read, mpd_ap_read, mpd_bp_read, mpd1_read, mpd2_read, mpd3_read, mpd4_read = read_mpds()
            mpd_oop.append(mpd_oop_read); mpd_orp.append(mpd_orp_read); mpd_ap.append(mpd_ap_read); mpd_bp.append(mpd_bp_read) 
            mpd1.append(mpd1_read); mpd2.append(mpd2_read); mpd3.append(mpd3_read); mpd4.append(mpd4_read) 
            
            # Read modulation values
            mod_i_oop_read, mod_q_oop_read, mod_i_apd_read, mod_q_apd_read, mod_i_vgain_read, mod_q_vgain_read = read_modulation()
            mod_i_oop.append(mod_i_oop_read); mod_q_oop.append(mod_q_oop_read); mod_i_apd.append(mod_i_apd_read); mod_q_apd.append(mod_q_apd_read); mod_i_vgain.append(mod_i_vgain_read); mod_q_vgain.append(mod_q_vgain_read);
            
            fscs_heat_read = heater_heat('fscs'); fscs_heat.append(fscs_heat_read)
            comb_heat_read = heater_heat('comb'); comb_heat.append(comb_heat_read)
            
            if is_print_scan:
                print( '%.2f °C (actual %.2f °C), Comb:%.2f °C, FSCS:%.2f °C, %.1f mA P.current, %.1f mW Opt.Power, %.5f THz, %.1f mA GC' %(temp_write,T_bench_actual_read, T_comb_actual_read, T_fscs_actual_read, mpd_oop_read, power_read, frequency_read, gc_read))
            
            if T_bench_actual_read < min_bench_temp or T_bench_actual_read > max_bench_temp:
                print('Mode-locking did not work')
                board_io('bench2 enable off') # turn off the bench for safety
                print_thermal_optical_state()
                simple_email(email_subject='Experiment Interrupted', email_body='Mode locking did not work')
                break
            
            
        mzi1_max_temp.append(mzi1_max); mzi2_max_temp.append(mzi2_max); mzi3_max_temp.append(mzi3_max); mzi4_max_temp.append(mzi4_max)
            
        vgain_temp.append(vgain); vl_temp.append(vl)
        mpd_oop_temp.append(mpd_oop); mpd_orp_temp.append(mpd_orp); mpd_ap_temp.append(mpd_ap); mpd_bp_temp.append(mpd_bp);
        mpd1_temp.append(mpd1); mpd2_temp.append(mpd2); mpd3_temp.append(mpd3); mpd4_temp.append(mpd4);
        mod_i_oop_temp.append(mod_i_oop); mod_q_oop_temp.append(mod_q_oop);
        mod_i_apd_temp.append(mod_i_apd); mod_q_apd_temp.append(mod_q_apd); 
        mod_i_vgain_temp.append(mod_i_vgain); mod_q_vgain_temp.append(mod_q_vgain); 
        power_temp.append(power); frequency_temp.append(frequency)
        fscs_heat_temp.append(fscs_heat); comb_heat_temp.append(comb_heat)
        T_fscs_actual_temp.append(T_fscs_actual); T_bench_actual_temp.append(T_bench_actual); T_comb_actual_temp.append(T_comb_actual)
        
        temp_ext1_temp.append(temp_ext1); temp_ext2_temp.append(temp_ext2)


        elapsed_time = time.time() - start_time
        print('Elapsed time %.1f seconds (%s)' %(elapsed_time, str(datetime.timedelta(seconds=elapsed_time)) ) )

        
    print('Max difference between target & actual T = %.3f °C' %max(temp_diff))
    
    if not mode_lock:
        # Switch back to thermal control on the bench
        set_bench_mode('temp')

        # Set the bench2 back to temp_min
        heater_temp('bench2', min(temp_range) )
        print('bench2 is set to %s' %min(temp_range))

    # Turn off the GC
    gc_read = gain_current(0)
    
    print('Gain Current is %.3f mA now' %gc_read)
    elapsed_time = time.time() - start_time
    print('All took %.1f seconds (%s)' %(elapsed_time,  str(datetime.timedelta(seconds=elapsed_time)) ) )


    # No need to Transpose the measured arrays because the scan direction is Bias T, Scan GC
    
    output = [time_measured, temp_range, gc_range, vgain_temp, vl_temp, mpd_oop_temp, mpd_orp_temp, mpd_ap_temp, mpd_bp_temp, mpd1_temp, mpd2_temp, mpd3_temp, mpd4_temp, power_temp, frequency_temp, mod_i_oop_temp, mod_q_oop_temp, mod_i_apd_temp, mod_q_apd_temp, mod_i_vgain_temp, mod_q_vgain_temp, T_fscs_actual_temp, T_bench_actual_temp, T_comb_actual_temp, fscs_heat_temp, comb_heat_temp, mzi1_max_temp, mzi2_max_temp, mzi3_max_temp, mzi4_max_temp, temp_ext1_temp, temp_ext2_temp]  
    
    try :
        output_address, output_date_stamp = export_pit_data(output, data_title = data_title, folder_address = SN_selected_output_folder_address + measurement_types)
    except:
        print('Could not export the data, returning it here to save it.')
        simple_email(email_subject='Experiment Interrupted', email_body='Could not export the data, data is returned to save it')
        return output
        
    plot_pit_sweeps(temp_range, gc_range, mpd_oop_temp, power_temp, frequency_temp, 
                    mod_i_oop_temp, mod_q_oop_temp, mod_i_apd_temp, mod_q_apd_temp,
                    T_bench_actual_temp=T_bench_actual_temp, T_comb_actual_temp = T_comb_actual_temp,
                    mpd1_temp = mpd1_temp, mpd2_temp = mpd2_temp, mpd3_temp = mpd3_temp, mpd4_temp = mpd4_temp,
                temp_step_plot_factor = 1, gc_step_plot_factor = 1,
                heater = heater,
                file_name_powerpoint = '',
                    folder_address = SN_selected_output_folder_address + measurement_types,
                       data_title=data_title, output_date_stamp = output_date_stamp)
    
    attach_image_names = choose_images_to_attach(len(temp_range), len(gc_range), output_date_stamp, heater = heater)
    
    if is_email_results:
        email_results(email_subject=data_title, attach_file_address = output_address, attach_image_names =  attach_image_names)
    return output  

In [ ]:
# Authored by Sergei 11-11-20
def bias_T_scan_gc_V2(data_title='',
                   gc_min = 49 , gc_max = 50, gc_step = -1, # Set min, max gain current, the step size in mA
                   temp_min = 22, temp_max = 30, temp_step = -1, # Set min, max temperature, the step size in Celcius
                   heater = 'bench2', # the heater to scan T
                   bench_enabled = 'on', # enable bench
                   temp_fscs = 25, fscs_enabled = 'on', # fscs temperature 
                   temp_comb = 25, comb_enabled = 'on', # comb temperature
                   bench_wait_time = 3, # wait time between each step while scanning the temperature so that the loop stabilizes
                   gc_wait_time = 0.5, # wait time at each gc step during the scan
                   mode_lock = 0 , 
                   is_maximize_mzis = 0, # to maximize the optical power with MZIs 
                   min_bench_temp = 16, max_bench_temp = 60, #for  safety check during mode_lock
                   is_print_scan = 1, # whether to print the results at each step during the scan
                   measurement_types = '', is_email_results = 1
):
    
    if data_title == '':
        raise ValueError('Always provide data_title')
    print('Data title: %s' %data_title)
    
    print_thermal_optical_state()
        
    start_time = time.time()

    # Prepare the Temperature and Gain Current Ranges
    temp_range = np.arange(temp_max, temp_min + temp_step, temp_step) if temp_step < 0 else np.arange(temp_min, temp_max, temp_step)    
    gc_range = np.arange(gc_max, gc_min + gc_step, gc_step) if gc_step < 0 else np.arange(gc_min, gc_max + gc_step, gc_step) 

    temp_bench2 = temp_range[0] # target is the first temperature value
    
    heater_temp('bench2', temp_bench2); board_io( 'bench2 enable '+ bench_enabled)     # Enable bench
    heater_temp('fscs', temp_fscs); board_io( 'fscs enable ' + fscs_enabled) # Enable the heater
    heater_temp('comb', temp_comb); board_io( 'comb enable ' + comb_enabled) # Enable the heater

    
    time_measured = [] ; vgain_temp = []; vl_temp = []
    power_temp = []; temp_diff = []; frequency_temp = []; gc = []; mod_i_oop_temp = []; mod_q_oop_temp= [];mod_i_apd_temp = []; mod_q_apd_temp= []; mod_i_vgain_temp = []; mod_q_vgain_temp = [];
    frequency_read = 0 ; temp = []; 
    fscs_heat_temp = []; comb_heat_temp = []
    T_fscs_actual_temp = []; T_bench_actual_temp = []; T_comb_actual_temp = [] 
    mpd1_temp = []; mpd2_temp = []; mpd3_temp = []; mpd4_temp = []; mpd_oop_temp = []; mpd_orp_temp = []; mpd_ap_temp = []; mpd_bp_temp = [];
    mzi1_max_temp = [] ; mzi2_max_temp = [] ; mzi3_max_temp = []; mzi4_max_temp = []
    
    temp_ext1_temp = []; temp_ext2_temp =[] # external temperatures measured by Thorlabs TPS01
    
    try :
        power_meter.sense.power.dc.range.upper = 60/1000 # 60 mW, set to maximum range of optical power by PM100D
    except:
        print('Power meter range cannot be set')
    
    print('Temperature Range: %s' %temp_range)

    for temp_write in temp_range:

        print_thermal_optical_state()
        gc_read = gain_current(gc_range[0])
        
        heater_temp('bench2', temp_write) 
        time.sleep(bench_wait_time) # so that temperature stabilizes 

        temp_actual = heater_temp('bench2')

        temp.append(temp_actual)
        temp_diff.append( abs(temp_write - temp_actual) ) # difference between the input and the read temp value

        print('T = %.3f °C (Target: %.3f °C)' %(temp_actual, temp_write) )

        gc = []; vgain = []; vl = []
        power = []; mpd = []; frequency = []; mod_i_oop = []; mod_q_oop = []; mod_i_apd = []; mod_q_apd = []; mod_i_vgain = []; mod_q_vgain = [];
        fscs_heat=[]; comb_heat=[]
        T_fscs_actual = []; T_bench_actual = []; T_comb_actual = [] 
        mpd1 = []; mpd2 = []; mpd3 = []; mpd4 = []; mpd_oop = []; mpd_orp = []; mpd_ap = []; mpd_bp = [];
        mzi1_max = [] ; mzi2_max = [] ; mzi3_max = []; mzi4_max = []
        
        temp_ext1 = [] ; temp_ext2 = []

        expected_measurement_duration = (2.4+gc_wait_time)*(gc_min-gc_max)/gc_step
        print('Starting Locked PI %s, ~%s s (%s)' %(time.strftime("on %Y_%m_%d at %H:%M"), expected_measurement_duration, str(datetime.timedelta(seconds=expected_measurement_duration)) ) ) 

        for gc_write in gc_range:


            gc_read = gain_current(gc_write); gc.append(gc_read); 
            if is_print_scan:
                print('Gain Current is %.3f mA' %gc_read)
            
            time.sleep(gc_wait_time) # so that the spectrum stabilizes 
            time_measured.append( time.strftime("%Y-%m-%d_%H:%M:%S") )
            
            # MZI heat values
            T_mzi1_actual_read = heater_temp('mzi 1'); T_mzi2_actual_read = heater_temp('mzi 2'); T_mzi3_actual_read = heater_temp('mzi 3'); T_mzi4_actual_read = heater_temp('mzi 4');
            
            if is_maximize_mzis:
                # optimize MZIs
                mzi_maximize_wait_time = 1
                mzi1_heat_read = maximize_power_via_mzi(1, 0.5, mzi_maximize_wait_time); mzi2_heat_read = maximize_power_via_mzi(2, 0.5, mzi_maximize_wait_time)
                mzi3_heat_read = maximize_power_via_mzi(3, 0.5, mzi_maximize_wait_time); mzi4_heat_read = maximize_power_via_mzi(4, 0.5, mzi_maximize_wait_time)
            else :
                # no optimization, append current values
                mzi1_heat_read = T_mzi1_actual_read; mzi2_heat_read = T_mzi2_actual_read; mzi3_heat_read = T_mzi3_actual_read; mzi4_heat_read = T_mzi4_actual_read;
                
            mzi1_max.append(mzi1_heat_read); mzi2_max.append(mzi2_heat_read); mzi3_max.append(mzi3_heat_read); mzi4_max.append(mzi4_heat_read)

            # FSCS Actual Temperature
            T_fscs_actual_read = heater_temp('fscs')
            T_fscs_actual.append(T_fscs_actual_read)
            
            # TPS01 Thorlabs Temperature Sensor
            try:
                temp_ext1_read = tsp01.measure_temperature('th0')
                temp_ext2_read = tsp01.measure_temperature('th1')
                
            except:
                temp_ext1_read = 0
                temp_ext2_read = 0
            
            temp_ext1.append(temp_ext1_read)
            temp_ext2.append(temp_ext2_read)

            # bench2 Actual Temperature
            T_bench_actual_read = heater_temp('bench2')
            T_bench_actual.append(T_bench_actual_read)
            
            temp_diff.append( abs(temp_write - T_bench_actual_read) ) # difference between the input and the read temp value
            

            # Comb Actual Temperature
            T_comb_actual_read = heater_temp('comb')
            T_comb_actual.append(T_comb_actual_read)

            temp_diff.append( abs(temp_write - T_bench_actual_read) ) # difference between the input and the read temp value

            ### Measurements ###
            
            vgain.append(float( re.findall('[0-9-.]+', board_io('adc vgain')[-1])[0] ))
            vl.append( float( re.findall('[0-9-.]+', board_io('gc vl')[-1])[0] ) )
            
            
            power_read = read_power()

            power.append(power_read)

            # Optical Power and Peak frequency from Bristol 438A Wavelength Meter
            #power_bristol.append(float(bristol.getSimpleMsg(b':READ:SCAL:ALL?').split(', ')[3]))
            frequency_read = read_frequency()
            frequency.append(frequency_read)

            # Read Photodiodes
            mpd_oop_read, mpd_orp_read, mpd_ap_read, mpd_bp_read, mpd1_read, mpd2_read, mpd3_read, mpd4_read = read_mpds()
            mpd_oop.append(mpd_oop_read); mpd_orp.append(mpd_orp_read); mpd_ap.append(mpd_ap_read); mpd_bp.append(mpd_bp_read) 
            mpd1.append(mpd1_read); mpd2.append(mpd2_read); mpd3.append(mpd3_read); mpd4.append(mpd4_read) 
            
            # Read modulation values
            mod_i_oop_read, mod_q_oop_read, mod_i_apd_read, mod_q_apd_read, mod_i_vgain_read, mod_q_vgain_read = read_modulation()
            mod_i_oop.append(mod_i_oop_read); mod_q_oop.append(mod_q_oop_read); mod_i_apd.append(mod_i_apd_read); mod_q_apd.append(mod_q_apd_read); mod_i_vgain.append(mod_i_vgain_read); mod_q_vgain.append(mod_q_vgain_read);
            
            fscs_heat_read = heater_heat('fscs'); fscs_heat.append(fscs_heat_read)
            comb_heat_read = heater_heat('comb'); comb_heat.append(comb_heat_read)
            
            if is_print_scan:
                print( '%.2f °C (actual %.2f °C), Comb:%.2f °C, FSCS:%.2f °C, %.1f mA P.current, %.1f mW Opt.Power, %.5f THz, %.1f mA GC' %(temp_write,T_bench_actual_read, T_comb_actual_read, T_fscs_actual_read, mpd_oop_read, power_read, frequency_read, gc_read))
            
            if T_bench_actual_read < min_bench_temp or T_bench_actual_read > max_bench_temp:
                print('Mode-locking did not work')
                board_io('bench2 enable off') # turn off the bench for safety
                print_thermal_optical_state()
                simple_email(email_subject='Experiment Interrupted', email_body='Mode locking did not work')
                break
            
            
        mzi1_max_temp.append(mzi1_max); mzi2_max_temp.append(mzi2_max); mzi3_max_temp.append(mzi3_max); mzi4_max_temp.append(mzi4_max)
            
        vgain_temp.append(vgain); vl_temp.append(vl)
        mpd_oop_temp.append(mpd_oop); mpd_orp_temp.append(mpd_orp); mpd_ap_temp.append(mpd_ap); mpd_bp_temp.append(mpd_bp);
        mpd1_temp.append(mpd1); mpd2_temp.append(mpd2); mpd3_temp.append(mpd3); mpd4_temp.append(mpd4);
        mod_i_oop_temp.append(mod_i_oop); mod_q_oop_temp.append(mod_q_oop);
        mod_i_apd_temp.append(mod_i_apd); mod_q_apd_temp.append(mod_q_apd); 
        mod_i_vgain_temp.append(mod_i_vgain); mod_q_vgain_temp.append(mod_q_vgain); 
        power_temp.append(power); frequency_temp.append(frequency)
        fscs_heat_temp.append(fscs_heat); comb_heat_temp.append(comb_heat)
        T_fscs_actual_temp.append(T_fscs_actual); T_bench_actual_temp.append(T_bench_actual); T_comb_actual_temp.append(T_comb_actual)
        
        temp_ext1_temp.append(temp_ext1); temp_ext2_temp.append(temp_ext2)


        elapsed_time = time.time() - start_time
        print('Elapsed time %.1f seconds (%s)' %(elapsed_time, str(datetime.timedelta(seconds=elapsed_time)) ) )

        
    print('Max difference between target & actual T = %.3f °C' %max(temp_diff))
    
    if not mode_lock:
        # Switch back to thermal control on the bench
        set_bench_mode('temp')

        # Set the bench2 back to temp_min
        heater_temp('bench2', min(temp_range) )
        print('bench2 is set to %s' %min(temp_range))

    # Turn off the GC
    gc_read = gain_current(0)
    
    print('Gain Current is %.3f mA now' %gc_read)
    elapsed_time = time.time() - start_time
    print('All took %.1f seconds (%s)' %(elapsed_time,  str(datetime.timedelta(seconds=elapsed_time)) ) )


    # No need to Transpose the measured arrays because the scan direction is Bias T, Scan GC
    
    output = [time_measured, temp_range, gc_range, vgain_temp, vl_temp, mpd_oop_temp, mpd_orp_temp, mpd_ap_temp, mpd_bp_temp, mpd1_temp, mpd2_temp, mpd3_temp, mpd4_temp, power_temp, frequency_temp, mod_i_oop_temp, mod_q_oop_temp, mod_i_apd_temp, mod_q_apd_temp, mod_i_vgain_temp, mod_q_vgain_temp, T_fscs_actual_temp, T_bench_actual_temp, T_comb_actual_temp, fscs_heat_temp, comb_heat_temp, mzi1_max_temp, mzi2_max_temp, mzi3_max_temp, mzi4_max_temp, temp_ext1_temp, temp_ext2_temp]  
    
    try :
        output_address, output_date_stamp = export_pit_data(output, data_title = data_title, folder_address = SN_selected_output_folder_address + measurement_types)
    except:
        print('Could not export the data, returning it here to save it.')
        simple_email(email_subject='Experiment Interrupted', email_body='Could not export the data, data is returned to save it')
        return output
        
    plot_pit_sweeps(temp_range, gc_range, mpd_oop_temp, power_temp, frequency_temp, 
                    mod_i_oop_temp, mod_q_oop_temp, mod_i_apd_temp, mod_q_apd_temp,
                    T_bench_actual_temp=T_bench_actual_temp, T_comb_actual_temp = T_comb_actual_temp,
                    mpd1_temp = mpd1_temp, mpd2_temp = mpd2_temp, mpd3_temp = mpd3_temp, mpd4_temp = mpd4_temp,
                temp_step_plot_factor = 1, gc_step_plot_factor = 1,
                heater = heater,
                file_name_powerpoint = '',
                    folder_address = SN_selected_output_folder_address + measurement_types,
                       data_title=data_title, output_date_stamp = output_date_stamp)
    
    attach_image_names = choose_images_to_attach(len(temp_range), len(gc_range), output_date_stamp, heater = heater)
    
    if is_email_results:
        email_results(email_subject=data_title, attach_file_address = output_address, attach_image_names =  attach_image_names)
    return output  

# Bias Gain Chip Current, Scan Temperature

In [16]:
def bias_gc_scan_T(data_title='',
                   gc_min = 49 , gc_max = 50, gc_step = -1, # Set min, max gain current, the step size in mA
                   temp_min = 22, temp_max = 30, temp_step = -1, # Set min, max temperature, the step size in Celcius
                   heater = 'bench2', # the heater to scan T
                   temp_bench2 = 25, bench_enabled = 'on', # bench2 temperature
                   temp_fscs  = 25, fscs_enabled = 'on', # fscs temperature
                   temp_comb  = 25, comb_enabled = 'on', # comb temperature
                   heater_wait_time = 3, # wait time between each step while scanning the temperature so that the loop stabilizes
                   gc_wait_time = 3, # wait time after changing the gain current
                   is_maximize_mzis = 0, # to maximize the optical power with MZIs
                   measurement_types = '', is_email_results = 1
                   
):
    
    if data_title == '':
        raise ValueError('Always provide data_title')
    print('Data title: %s' %data_title)

    bench_mode = set_bench_mode()
    if bench_mode != 'temp' and heater == 'bench2':
        raise ValueError('bench2 is not in the Thermal Control Mode')
    
    print_thermal_optical_state()
    
    start_time = time.time()

    # Prepare the Temperature and Gain Current Ranges
    temp_range = np.arange(temp_max, temp_min + temp_step, temp_step) if temp_step < 0 else np.arange(temp_min, temp_max + temp_step, temp_step)    
    gc_range = np.arange(gc_max, gc_min + gc_step, gc_step) if gc_step < 0 else np.arange(gc_min, gc_max + gc_step, gc_step) 

    # Enable the heaters
    heater_temp('bench2', temp_bench2); board_io( 'bench2 enable '+ bench_enabled)     # Enable bench
    heater_temp('fscs', temp_fscs); board_io( 'fscs enable ' + fscs_enabled) # Enable the heater
    heater_temp('comb', temp_comb); board_io( 'comb enable ' + comb_enabled) # Enable the heater

    time_measured = []; vgain_temp = []; vl_temp = []
    power_temp = []; temp_diff = []; frequency_temp = []; gc = []; mod_i_oop_temp = []; mod_q_oop_temp= []; mod_i_apd_temp = []; mod_q_apd_temp= []; mod_i_vgain_temp = []; mod_q_vgain_temp= [];  
    frequency_read = 0 ; temp = []; 
    fscs_heat_temp = []; comb_heat_temp = []
    T_fscs_actual_temp = []; T_bench_actual_temp = []; T_comb_actual_temp = [] 
    mpd1_temp = []; mpd2_temp = []; mpd3_temp = []; mpd4_temp = []; mpd_oop_temp = []; mpd_orp_temp = []; mpd_ap_temp = []; mpd_bp_temp = [];
    mzi1_max_temp = [] ; mzi2_max_temp = [] ; mzi3_max_temp = []; mzi4_max_temp = []

    temp_ext1_temp = []; temp_ext2_temp =[] # external temperatures measured by Thorlabs TPS01

    try :
        power_meter.sense.power.dc.range.upper = 60/1000 # 60 mW, set to maximum range of optical power by PM100D
    except:
        print('Power meter range cannot be set')

    for gc_write in gc_range:

        # change to the first value of temp_range and wait
        # so that this abrupt change will not cause any abnomalities
        board_io( '%s target %s' %(heater,temp_range[0]) )#

        gc_read = gain_current(gc_write); gc.append(gc_read); print('Gain Current is %.3f mA' %gc_read)
        

        time.sleep(gc_wait_time) # so that the spectrum stabilizes 

        mpd1 = []; mpd2 = []; mpd3 = []; mpd4 = []; mpd_oop = []; mpd_orp = []; mpd_ap = []; mpd_bp = [];
        power = []; frequency = []; vgain = []; vl = []; mod_i_oop = []; mod_q_oop = []; mod_i_apd = []; mod_q_apd = []; mod_i_vgain = []; mod_q_vgain = [];
        fscs_heat=[]; comb_heat=[]
        T_fscs_actual = []; T_bench_actual = []; T_comb_actual = [] 
        mzi1_max = [] ; mzi2_max = [] ; mzi3_max = []; mzi4_max = []
        
        temp_ext1 = [] ; temp_ext2 = []

        time_measured_temp = []
        
        expected_measurement_duration = (2.2+heater_wait_time)*abs((temp_max-temp_min)/temp_step)
        print('Starting Temp Scan %s, expected to take %s seconds (%s)' %(time.strftime("on %Y_%m_%d at %H:%M"), expected_measurement_duration, str(datetime.timedelta(seconds=expected_measurement_duration)) ) )

        for temp_write in temp_range:

            board_io( '%s target %s' %(heater,temp_write) )#

            time.sleep(heater_wait_time) # so that temperature stabilizes 
            
            time_measured_temp.append( time.strftime("%Y-%m-%d_%H:%M:%S") )
            
            # MZI heat values
            T_mzi1_actual_read = heater_temp('mzi 1'); T_mzi2_actual_read = heater_temp('mzi 2'); T_mzi3_actual_read = heater_temp('mzi 3'); T_mzi4_actual_read = heater_temp('mzi 4');
           
            
            if is_maximize_mzis:
                # optimize MZIs
                mzi_maximize_wait_time = 1
                mzi1_heat_read = maximize_power_via_mzi(1, 0.5, mzi_maximize_wait_time); mzi2_heat_read = maximize_power_via_mzi(2, 0.5, mzi_maximize_wait_time)
                mzi3_heat_read = maximize_power_via_mzi(3, 0.5, mzi_maximize_wait_time); mzi4_heat_read = maximize_power_via_mzi(4, 0.5, mzi_maximize_wait_time)
            else :
                # no optimization, append current values
                mzi1_heat_read = T_mzi1_actual_read; mzi2_heat_read = T_mzi2_actual_read; mzi3_heat_read = T_mzi3_actual_read; mzi4_heat_read = T_mzi4_actual_read;
                
            mzi1_max.append(mzi1_heat_read); mzi2_max.append(mzi2_heat_read); mzi3_max.append(mzi3_heat_read); mzi4_max.append(mzi4_heat_read)

            # FSCS Actual Temperature
            T_fscs_actual_read = heater_temp('fscs'); T_fscs_actual.append(T_fscs_actual_read)
            
            # bench2 Actual Temperature
            T_bench_actual_read = heater_temp('bench2'); T_bench_actual.append(T_bench_actual_read)
            
            # TPS01 Thorlabs Temperature Sensor
            try:
                temp_ext1_read = tsp01.measure_temperature('th0')
                temp_ext2_read = tsp01.measure_temperature('th1')
                
            except:
                temp_ext1_read = 0
                temp_ext2_read = 0
            
            temp_ext1.append(temp_ext1_read)
            temp_ext2.append(temp_ext2_read)
            
            # Comb Actual Temperature
            T_comb_actual_read = heater_temp('comb'); T_comb_actual.append(T_comb_actual_read)
            
            T_actual_read ={'bench2' : T_bench_actual_read, 'fscs' : T_fscs_actual_read, 'comb' : T_comb_actual_read ,
                            'mzi 1' : T_mzi1_actual_read, 'mzi 2' : T_mzi2_actual_read, 'mzi 3' : T_mzi3_actual_read,
                            'mzi 4' : T_mzi4_actual_read
                           }[heater]

            temp_diff.append( abs(temp_write - T_actual_read) ) # difference between the input and the read temp value

            ### Measurements ###
            
            vgain.append(float( re.findall('[0-9-.]+', board_io('adc vgain')[-1])[0] ))
            vl.append(float( re.findall('[0-9-.]+', board_io('gc vl')[-1])[0] ))

            
            power_read = read_power()

            power.append(power_read)

            # Optical Power and Peak frequency from Bristol 438A Wavelength Meter
            #power_bristol.append(float(bristol.getSimpleMsg(b':READ:SCAL:ALL?').split(', ')[3]))
            frequency_read = read_frequency()
            frequency.append(frequency_read)

            
            # Read Photodiodes
            mpd_oop_read, mpd_orp_read, mpd_ap_read, mpd_bp_read, mpd1_read, mpd2_read, mpd3_read, mpd4_read = read_mpds()
            mpd_oop.append(mpd_oop_read); mpd_orp.append(mpd_orp_read); mpd_ap.append(mpd_ap_read); mpd_bp.append(mpd_bp_read) 
            mpd1.append(mpd1_read); mpd2.append(mpd2_read); mpd3.append(mpd3_read); mpd4.append(mpd4_read) 
            
            # Read modulation values
            mod_i_oop_read, mod_q_oop_read, mod_i_apd_read, mod_q_apd_read, mod_i_vgain_read, mod_q_vgain_read = read_modulation()
            mod_i_oop.append(mod_i_oop_read); mod_q_oop.append(mod_q_oop_read); mod_i_apd.append(mod_i_apd_read); mod_q_apd.append(mod_q_apd_read); mod_i_vgain.append(mod_i_vgain_read); mod_q_vgain.append(mod_q_vgain_read);

            
            fscs_heat_read = heater_heat('fscs'); fscs_heat.append(fscs_heat_read)
            comb_heat_read = heater_heat('comb'); comb_heat.append(comb_heat_read)

            print( '%.3f °C, Bench:%.3f °C, Comb:%.3f °C, FSCS:%.2f °C' %(temp_write,T_bench_actual_read, T_comb_actual_read, T_fscs_actual_read))
            #print( '%.2f °C, Bench:%.2f °C, Comb:%.2f °C, FSCS:%.2f °C, %.1f mA P.current, %.1f mW Opt.Power, %.5f THz, %.1f mA GC' %(temp_write,T_bench_actual_read, T_comb_actual_read, T_fscs_actual_read, mpd_read, power_read, frequency_read, gc_read))

        mzi1_max_temp.append(mzi1_max); mzi2_max_temp.append(mzi2_max); mzi3_max_temp.append(mzi3_max); mzi4_max_temp.append(mzi4_max)
        vgain_temp.append(vgain); vl_temp.append(vl)
        mpd_oop_temp.append(mpd_oop); mpd_orp_temp.append(mpd_orp); mpd_ap_temp.append(mpd_ap); mpd_bp_temp.append(mpd_bp);
        mpd1_temp.append(mpd1); mpd2_temp.append(mpd2); mpd3_temp.append(mpd3); mpd4_temp.append(mpd4);
        mod_i_oop_temp.append(mod_i_oop); mod_q_oop_temp.append(mod_q_oop); 
        mod_i_apd_temp.append(mod_i_apd); mod_q_apd_temp.append(mod_q_apd); 
        mod_i_vgain_temp.append(mod_i_vgain); mod_q_vgain_temp.append(mod_q_vgain); 
        power_temp.append(power); frequency_temp.append(frequency)
        fscs_heat_temp.append(fscs_heat); comb_heat_temp.append(comb_heat)
        T_fscs_actual_temp.append(T_fscs_actual); T_bench_actual_temp.append(T_bench_actual); T_comb_actual_temp.append(T_comb_actual)
        
        temp_ext1_temp.append(temp_ext1); temp_ext2_temp.append(temp_ext2)
        
        time_measured.append( time_measured_temp )

        elapsed_time = time.time() - start_time
        print('Elapsed time %.1f seconds (%s)' %(elapsed_time, str(datetime.timedelta(seconds=elapsed_time)) ))

    print('Max difference between target & actual T = %.3f °C' %max(temp_diff))

    # Set the bench2 back to temp_min
    
    board_io( '%s target %s' %(heater, min(temp_range)))
    print('%s is set to %s' %(heater, min(temp_range) ) ) 

    # Turn off the GC
    gc_read = gain_current(0)
    
    print('Gain Current is %.3f mA now' %gc_read)
    elapsed_time = time.time() - start_time
    print('All took %.1f seconds (%s)' %(elapsed_time,  str(datetime.timedelta(seconds=elapsed_time)) ) )

    # Transpose the measured arrays because the scan direction is Bias GC, Scan T
    [time_measured, mpd_oop_temp, mpd_orp_temp, mpd_ap_temp, mpd_bp_temp, mpd1_temp, mpd2_temp, mpd3_temp, mpd4_temp, power_temp, frequency_temp, mod_i_oop_temp, mod_q_oop_temp, mod_i_apd_temp, mod_q_apd_temp, mod_i_vgain_temp, mod_q_vgain_temp] = transpose_lists([time_measured, mpd_oop_temp, mpd_orp_temp, mpd_ap_temp, mpd_bp_temp, mpd1_temp, mpd2_temp, mpd3_temp, mpd4_temp, power_temp, frequency_temp, mod_i_oop_temp, mod_q_oop_temp, mod_i_apd_temp, mod_q_apd_temp, mod_i_vgain_temp, mod_q_vgain_temp])
    [T_fscs_actual_temp, T_bench_actual_temp, T_comb_actual_temp, fscs_heat_temp, comb_heat_temp] =  transpose_lists([T_fscs_actual_temp, T_bench_actual_temp, T_comb_actual_temp, fscs_heat_temp, comb_heat_temp])
    
    time_measured =  list(itertools.chain.from_iterable(time_measured))
    
    output = [time_measured, temp_range, gc_range, vgain_temp, vl_temp, mpd_oop_temp, mpd_orp_temp, mpd_ap_temp, mpd_bp_temp, mpd1_temp, mpd2_temp, mpd3_temp, mpd4_temp, power_temp, frequency_temp, mod_i_oop_temp, mod_q_oop_temp, mod_i_apd_temp, mod_q_apd_temp, mod_i_vgain_temp, mod_q_vgain_temp, T_fscs_actual_temp, T_bench_actual_temp, T_comb_actual_temp, fscs_heat_temp, comb_heat_temp, mzi1_max_temp, mzi2_max_temp, mzi3_max_temp, mzi4_max_temp, temp_ext1_temp, temp_ext2_temp] 
    
    try :
        output_address, output_date_stamp = export_pit_data(output, data_title = data_title, folder_address = SN_selected_output_folder_address + measurement_types )
    except:
        print('Could not export the data, returning it here to save it.')
        simple_email(email_subject='Experiment Interrupted', email_body='Could not export the data, data is returned to save it')
        return output
    
    
    plot_pit_sweeps(temp_range, gc_range, mpd_oop_temp, power_temp, frequency_temp, 
                    mod_i_oop_temp, mod_q_oop_temp, mod_i_apd_temp, mod_q_apd_temp,
                    T_bench_actual_temp=T_bench_actual_temp, T_comb_actual_temp = T_comb_actual_temp,
                    mpd1_temp = mpd1_temp, mpd2_temp = mpd2_temp, mpd3_temp = mpd3_temp, mpd4_temp = mpd4_temp,
                temp_step_plot_factor = 1, gc_step_plot_factor = 1,
                heater = heater,
                file_name_powerpoint = '',
                    folder_address = SN_selected_output_folder_address + measurement_types,
                       data_title=data_title, output_date_stamp = output_date_stamp)
    
    attach_image_names = choose_images_to_attach(len(temp_range), len(gc_range), output_date_stamp, heater = heater)        
    
    if is_email_results:
        email_results(email_subject=data_title, attach_file_address = output_address, attach_image_names =  attach_image_names)
    
    return output 

# MZI Scans

In [17]:
# Lock at a certain frequency and do MZI sweeps
def mzi_scanner(frequency_range, gc_range, bench_temp = 50, bench_mode ='temp',
                mzi_heat_first=-5, mzi_heat_last=+5, mzi_step=+1, mzi_step_wait_time = 0,
                mzi_indices = [1,2,3,4], is_email_results = 1
               ):
    
    mzi_at_max_optical_power = []
    
    for mzi_index in mzi_indices:
        
        heater = 'mzi %s' %mzi_index
         
        set_laser_frequency(frequency_range = frequency_range, gc_range = gc_range, bench_temp_range = bench_temp, 
                    bench_mode = bench_mode,
                    is_email_results = is_email_results,
                        is_measure_spectrum = 0, # to measure spectrum with yokogawa 
                    yokogawa_sweep_wait_time = 15,
                        is_maximize_mzis = 0, # to maximize the optical power with MZIs 
                        n_repeat_for_stability = 1, # # of times to measure some of the parameters for stability
                    stability_wait_time = 1,
                    frequency_set_wait_time = 15, bench_wait_time = 15, lock_wait_time = 30, mzi_maximize_wait_time = 1,
                        data_title = '', 
                    calibration_file_name = "%s_SIP calibration_bench %sC" %(SN_selected, bench_temp),
                   comb_temp_file_name = '%s_Tune Table' %(SN_selected))
        
        #set_bench_mode('temp')
        
        bench_lock_temp = heater_temp('bench2', -200)
        comb_temp = heater_temp('comb', -200)
        
        # gc_range is converted to a list or np.ndarray, change it back
        if isinstance (gc_range, list) or isinstance (gc_range, np.ndarray):
            gc_range = gc_range[0]
        
        data_title = 'MZI Scan_%s_GC %smA_%s_%s to %s by %smW_Bench %sC_Comb %sC_%s_locked' %(SN_selected, gc_range, heater.upper(), mzi_heat_first, mzi_heat_last, mzi_step, bench_temp, comb_temp, bench_mode)
        print("Data title: %s" %data_title)

        output = bias_gc_scan_T( 
        data_title = data_title,
        gc_min = gc_range , gc_max = gc_range, gc_step = -4, 
        heater = heater, # the heater to scan T
        temp_min   = mzi_heat_first, temp_max = mzi_heat_last, temp_step = mzi_step,
        temp_bench2 = bench_lock_temp, bench_enabled = 'on', # bench2 temperature
        temp_fscs  = 85,  fscs_enabled  = 'off', # fscs temperature
        temp_comb  = comb_temp, comb_enabled  = 'on', # comb temperature
        heater_wait_time = mzi_step_wait_time,  gc_wait_time = 3,
        measurement_types = 'MZI Scans',  is_email_results = is_email_results)
        

        [time_measured, temp_range, gc_range, vgain_temp, vl_temp, 
     mpd_oop_temp, mpd_orp_temp, mpd_ap_temp, mpd_bp_temp, mpd1_temp, mpd2_temp, mpd3_temp, mpd4_temp, 
     power_temp, frequency_temp, 
     mod_i_oop_temp, mod_q_oop_temp, mod_i_apd_temp, mod_q_apd_temp, mod_i_vgain_temp, mod_q_vgain_temp,
     T_fscs_actual_temp, T_bench_actual_temp, T_comb_actual_temp, 
     fscs_heat_temp, comb_heat_temp, 
     mzi1_max_temp, mzi2_max_temp, mzi3_max_temp, mzi4_max_temp, temp_ext1_temp, temp_ext2_temp] = output
        
        
        mzi_scanning = [mzi1_max_temp, mzi2_max_temp, mzi3_max_temp, mzi4_max_temp][mzi_index-1][0]
        index_at_max_optical_power = power_temp.index(max(power_temp))
        mzi_at_max_optical_power.append( mzi_scanning[index_at_max_optical_power] )
        
    # Set the MZIs to the values maximizing the optical power
    for mzi_index in mzi_indices:
        heater = 'mzi %s' %mzi_index
        heater_temp(heater, mzi_at_max_optical_power[ mzi_indices.index(mzi_index) ] )
    
    return mzi_at_max_optical_power

# Set Laser Frequency

In [18]:
def check_F_T_G(F,T,G):

    if T not in ['25', '37', '50'] :
        print('%s C is invalid' %T)
        return 'not OK'

    if G < 0 or G > 600:
        print('%s mA is invalid' %G)
        return 'not OK'
    return 'OK'
    
def read_command_via_port(
    
    HOST = 'testlaptop1',  # IP address (server)
    PORT = 9999,        # Port to listen on (non-privileged ports are > 1023)
    N = 1 # number of connections allowed before closing the port
    ):
    
    dither = 'off' # whether to keep the dither on or off after setting the laser
    
    error_message = ' Command not recognized, try again!'
    ok_message = ' OK'
    
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT))
        s.listen()
        conn, addr = s.accept()
        with conn:
            print('Connected by', addr)
            while True:
                data = conn.recv(1024)
                if data:
                    command = data.decode()
                    
                    print('Customer sent: %s' %command)
                    conn.sendall(data)
                    
                    # command format : '193.5 THz 50 C 200 mA dither off'
                    if ' dither off' in command:
                        laser_parameters = command.replace(' dither off', '')
                        dither = 'off'
                        print('Dither off')
                        break
                    
                    # command format : '193.5 THz 50 C 200 mA dither on'
                    elif ' dither on' in command:
                        laser_parameters = command.replace(' dither on', '')
                        dither = 'on'
                        print('Dither on')
                        break
                        
                    elif 'shutdown' in command :
                        return 'shutdown'
                    
                    else :
                        print('Error: command sent by client not recognized: \'%s\' ' %command )
                        conn.sendall( bytes(error_message, encoding='utf8') )
                        return 'error' 
                        
                # The following makes sure only one instance of data is received per establhished connection
                elif not data:
                    break
    
            try:
                F, T, G = re.findall('[0-9-.]+' , laser_parameters) # extract the laser frequency, bench temperature and gain current
                F = float(F); G = float(G) # bench is either 25 or 50, do not let it be a float like 25.0 or 50.0 as it causes problems in other functions

                if check_F_T_G(F,T,G) == 'not OK':
                    raise ValueError()
                    
                conn.sendall( bytes(ok_message, encoding='utf8') )   
                return dither, F, T, G 
            
            except:
                print(error_message)
                conn.sendall( bytes(error_message, encoding='utf8') )
                return 'error'
        
        

In [19]:
# set the laser to a certain frequency 
# and perform scans: Cavity Phase Scan, FTF (Fine Tune Frequency) Scan, Locked PI Curve, ASE (Amplified Spontaneous Emission) PI Curve
def set_laser_frequency(frequency_range = [194], gc_range = [300], bench_temp_range = [50], 
                        dither = 'on', dither_off_wait_time = 10,
                        bench_mode = 'apd',
                        is_measure_spectrum = 0, # to measure spectrum with yokogawa 
                        is_maximize_mzis = 0, # to maximize the optical power with MZIs 
                        n_repeat_for_stability = 1, # # of times to measure some of the parameters for stability
                        stability_wait_time = 1,
                        # wait times
            frequency_set_wait_time = 15, bench_wait_time = 15, lock_wait_time = 10, mzi_maximize_wait_time = 1, yokogawa_sweep_wait_time = 10,
                        data_title = '', 
                        is_email_results = 1,
                        is_locked_PI = 0, is_ase_PI = 0, gc_step = -1, gc_wait_time = 3,
                        cavity_phase_scan_range = 0, ftf_scan_range = 0, temp_step = -0.02,  # Change temp_step sign here for reverse scan
                        calibration_file_name = '',
                        comb_temp_file_name = '', comb_enabled = 'on',
                        measurement_types = 'Channel Scans'
                       ):
                        
    
    
    if not isinstance (frequency_range, list) and not isinstance (frequency_range, np.ndarray):
        frequency_range = [frequency_range]
    if not isinstance (gc_range, list) and not isinstance (gc_range, np.ndarray):
        gc_range = list([gc_range])
    if not isinstance (bench_temp_range, list) and not isinstance (bench_temp_range, np.ndarray):
        bench_temp_range = [bench_temp_range]
    
    # if user provided only one gc_range value but there may be more than 1 frequency values
    if len(gc_range) == 1 and  len(frequency_range)>1 : #    
        gc_range = gc_range * len(frequency_range)
        bench_temp_range = bench_temp_range * len(frequency_range)

    if data_title == '':
        label_frequency_range =  str('%.3f' %frequency_range[0])+'-'+str('%.3f' %frequency_range[-1] ) if len(frequency_range) > 1 else frequency_range[0]
        data_title = 'Channel Scan_%s_%sThz_%smA_Bench %sC_%s_locked' %(SN_selected, label_frequency_range, str(gc_range[0]), str(bench_temp_range[0]), bench_mode) 
        if n_repeat_for_stability >1:
            data_title = data_title.replace('Channel Scan', 'Stability Test')
            data_title += '_repeat x%s' %n_repeat_for_stability
            measurement_types = 'Stability Tests'
    
    print('Data title: %s' %data_title)
    
    if calibration_file_name == '':
        calibration_file_name = "%s_SIP calibration_bench %sC" %(SN_selected, bench_temp)

    print('Calibration file needed: %s.xlsx' %calibration_file_name)
    
    if comb_temp_file_name == '':
        comb_temp_file_name = '%s_Tune Table_bench %sC' %(SN_selected, bench_temp)
    print ('Comb temperature file needed: %s.xlsx' %comb_temp_file_name)

    try :
        power_meter.sense.power.dc.range.upper = 60/1000 # 60 mW, set to maximum range of optical power by PM100D
    except:
        print('Power meter range cannot be set')

    T_bench_actual = []; T_comb_actual = []; H_bench_actual = []; H_comb_actual = []
    mod_i_oop = []; mod_q_oop = []; mod_i_apd = []; mod_q_apd = []; mod_i_vgain = []; mod_q_vgain = [];
    power = []; frequency = []; time_measured = []; comb_temp_range = []; smsr = []
    mzi1_set = [] ; mzi2_set = [] ; mzi3_set = []; mzi4_set = []
    mzi1_max = [] ; mzi2_max = [] ; mzi3_max = []; mzi4_max = []
    
    temp_ext1 = []; temp_ext2 =[] # external temperatures measured by Thorlabs TPS01

    mpd_oop_before = [];  mpd_orp_before = []; mpd_ap_before = []; mpd_bp_before = []; mpd1_before = []; mpd2_before = []; mpd3_before = []; mpd4_before = []; 
    mpd_oop = [];  mpd_orp = []; mpd_ap = []; mpd_bp = []; mpd1 = []; mpd2 = []; mpd3 = []; mpd4 = []; 
    spectrum_pd = pd.DataFrame();

    start_time = time.time()
    
    board_io('bench2 enable on') # make sure bench2 is enabled
    #board_io( 'dac phi 2.0'); board_io( 'mod enable 1000 127 255') # use module-specific settings set in test notebook, section 3.1

    for i in np.arange(len(frequency_range)):
        
        ith_time = time.time()
        
        # Turn off the gain current first: tuning in the dark is a customer requirement
        gc_read = gain_current(0); print('Gain Current is %.3f mA' %gc_read)

        print('#%d of %d' %(i+1, len(frequency_range)*n_repeat_for_stability))
        set_bench_mode('temp'); heater_temp('bench2', bench_temp_range[i] )

        time.sleep(bench_wait_time)

        mzi1_heat, mzi2_heat , mzi3_heat, mzi4_heat, set_comb_temp = set_frequency_via_mzi_comb(frequency_range[i] , bench_temp_range[i], calibration_file_name, comb_temp_file_name)
        mzi1_set.append(mzi1_heat); mzi2_set.append(mzi2_heat); mzi3_set.append(mzi3_heat); mzi4_set.append(mzi4_heat); 
        comb_temp_range.append(set_comb_temp)


        time.sleep(frequency_set_wait_time)
        gc_read = gain_current( gc_range[i] )

        set_bench_mode(bench_mode)

        time.sleep(lock_wait_time)

        T_bench_actual_read = heater_temp('bench2')# bench2 Actual Temperature
        T_comb_actual_read = heater_temp('comb') # Comb Actual Temperature
        print( 'Actual Comb Temperature :%.3f °C, Bench:%.3f °C (right before scanning MZIs)' %(T_comb_actual_read,T_bench_actual_read))
        
        if is_maximize_mzis:
            
            # Read Photodiodes before optimizing the MZIs
            mpd_oop_read, mpd_orp_read, mpd_ap_read, mpd_bp_read, mpd1_read, mpd2_read, mpd3_read, mpd4_read = read_mpds()
            mpd_oop_before.append(mpd_oop_read); mpd_orp_before.append(mpd_orp_read); mpd_ap_before.append(mpd_ap_read); mpd_bp_before.append(mpd_bp_read) 
            mpd1_before.append(mpd1_read); mpd2_before.append(mpd2_read); mpd3_before.append(mpd3_read); mpd4_before.append(mpd4_read) 
            
            mzi1_heat_read = maximize_power_via_mzi(1, 0.5, mzi_maximize_wait_time); mzi2_heat_read = maximize_power_via_mzi(2, 0.5, mzi_maximize_wait_time)
            mzi3_heat_read = maximize_power_via_mzi(3, 0.5, mzi_maximize_wait_time); mzi4_heat_read = maximize_power_via_mzi(4, 0.5, mzi_maximize_wait_time)
            
            print('MZI Maximized [1, 2, 3, 4]: [%.2f, %.2f, %.2f, %.2f]' %(mzi1_heat_read,mzi2_heat_read,mzi3_heat_read,mzi4_heat_read))
            
            time.sleep(mzi_maximize_wait_time)
            
        else:
            [mzi1_heat_read, mzi2_heat_read, mzi3_heat_read, mzi4_heat_read] = [mzi1_heat, mzi2_heat , mzi3_heat, mzi4_heat]

        mzi1_max.append(mzi1_heat_read); mzi2_max.append(mzi2_heat_read); mzi3_max.append(mzi3_heat_read); mzi4_max.append(mzi4_heat_read)
        
        if dither == 'off':
            time.sleep(dither_off_wait_time)
            
            T_bench_average_read = heater_temp('bench2', -500)
            # switch to temp mode
            board_io('bench2 mode temp'); heater_temp('bench2', T_bench_average_read )
            
            # disable dither
            board_io( 'mod disable'); print('Dither is off now')

        if is_measure_spectrum:
            
            # Perform a Single Sweep
            Write_OSA(yokogawa, ':INITiate:SMODe SING;:INIT')
            time.sleep(yokogawa_sweep_wait_time) # so that the sweep will be done
            x_data_yokogawa, y_data_yokogawa = get_yokogawa_trace(yokogawa)
            
            # Read SMSR
            Write_OSA(yokogawa, ':CALCulate:CATegory SMSR;:CALCulate;:CALCulate:DATA?')
            smsr_read = float(Read_OSA(yokogawa).split(',')[-1])
            print('SMSR: %.3f dB' %smsr_read)
            smsr.append(smsr_read)
            x_data_yokogawa.insert(0, 'SMSR (dB)'); y_data_yokogawa.insert(0, smsr_read)
            
            if spectrum_pd.empty: # Add the x-axis only once to the data frame as they are identical
                spectrum_pd['%.3f_' %frequency_range[i] + 'Frequency (THz)' ] = x_data_yokogawa
                
            spectrum_pd['%.3f_' %frequency_range[i] + 'Intensity (dBm)' ] = y_data_yokogawa
        
        for k in range(n_repeat_for_stability):
            
            try:
            
                if n_repeat_for_stability>1:
                    print("Stability Test %s of %s" %(k+1,n_repeat_for_stability))
                    time.sleep(stability_wait_time)
                time_measured.append( time.strftime("%Y-%m-%d_%H:%M:%S") )

                T_bench_actual_read = heater_temp('bench2'); T_bench_actual.append(T_bench_actual_read) # bench2 Actual Temperature
                T_comb_actual_read = heater_temp('comb'); T_comb_actual.append(T_comb_actual_read) # Comb Actual Temperature

                H_bench_actual_read = heater_heat('bench2'); H_bench_actual.append(H_bench_actual_read) # bench2 current
                H_comb_actual_read = heater_heat('comb'); H_comb_actual.append(H_comb_actual_read) # Comb heat

                power_read = read_power()
                power.append(power_read)

                # TPS01 Thorlabs Temperature Sensor
                try:
                    temp_ext1_read = tsp01.measure_temperature('th0')
                    temp_ext2_read = tsp01.measure_temperature('th1')

                except:
                    temp_ext1_read = 0
                    temp_ext2_read = 0

                temp_ext1.append(temp_ext1_read)
                temp_ext2.append(temp_ext2_read)


                # Read Photodiodes
                mpd_oop_read, mpd_orp_read, mpd_ap_read, mpd_bp_read, mpd1_read, mpd2_read, mpd3_read, mpd4_read = read_mpds()
                mpd_oop.append(mpd_oop_read); mpd_orp.append(mpd_orp_read); mpd_ap.append(mpd_ap_read); mpd_bp.append(mpd_bp_read) 
                mpd1.append(mpd1_read); mpd2.append(mpd2_read); mpd3.append(mpd3_read); mpd4.append(mpd4_read) 

                # Read modulation values
                mod_i_oop_read, mod_q_oop_read, mod_i_apd_read, mod_q_apd_read, mod_i_vgain_read, mod_q_vgain_read = read_modulation()
                mod_i_oop.append(mod_i_oop_read); mod_q_oop.append(mod_q_oop_read); mod_i_apd.append(mod_i_apd_read); mod_q_apd.append(mod_q_apd_read); mod_i_vgain.append(mod_i_vgain_read); mod_q_vgain.append(mod_q_vgain_read);

                frequency_read = read_frequency()
                frequency.append(frequency_read)
            except:
                print('Experiment %s is interrupted' %data_title)
                simple_email(email_subject='Experiment %s is interrupted' %data_title, email_body='%s' %data_title)


        
        print( 'Comb:%.3f °C, Bench:%.3f °C, %.3f mA P.current, %.3f mW Opt.Power, %.5f THz, %.1f mA GC, Mod i oop: %.3f\n' %(T_comb_actual_read,T_bench_actual_read, mpd_oop_read, power_read, frequency_read, gc_read, mod_i_oop_read))

        jth_time = time.time()
        print('Scan #%d took %.1f seconds' %(i+1, jth_time-ith_time) )
    elapsed_time = time.time() - start_time
    print('Scanning took %.1f seconds (= %.2f hours)' %(elapsed_time, elapsed_time/3600) )

    # convert the data collected to a data frame
    output_labels = ['Time',  'Set Frequency (THz)', 'Set Bench Temperature (°C)', 'Set Comb Temperature (°C)' ,  'Gain Current (mA)', 
                     #'MPD_OOP before MZI opt.(mA)','MPD_ORP before MZI opt.(mA)','MPD_AP before MZI opt.(mA)','MPD_BP before MZI opt.(mA)','MPD1 before MZI opt.(mA)','MPD2 before MZI opt.(mA)','MPD3 before MZI opt.(mA)','MPD4 before MZI opt.(mA)',
                     'MPD_OOP (mA)','MPD_ORP (mA)','MPD_AP (mA)','MPD_BP (mA)','MPD1 (mA)','MPD2 (mA)','MPD3 (mA)','MPD4 (mA)', 
                     'Optical Power (mW)', 'Frequency (THz)', 'SMSR', 'Mod i oop', 'Mod q oop',
                     'Bench Temperature (°C)', 'Comb Temperature (°C)', 'Calculated MZI 1 Heat (mW)', 'Calculated MZI 2 Heat (mW)', 'Calculated MZI 3 Heat (mW)', 'Calculated MZI 4 Heat (mW)', 
                     'MZI 1 Heat Max. (mW)', 'MZI 2 Heat Max. (mW)', 'MZI 3 Heat Max. (mW)', 'MZI 4 Heat Max. (mW)',
                     'Bench Current (mA)', 'Comb Heat (mW)', 'External Temp 1 (°C)', 'External Temp 2 (°C)']
                         
    output = [time_measured, frequency_range, bench_temp_range, comb_temp_range, gc_range, mpd_oop, mpd_orp, mpd_ap, mpd_bp, mpd1, mpd2, mpd3, mpd4, power, frequency, smsr, mod_i_oop, mod_q_oop, T_bench_actual , T_comb_actual, mzi1_set, mzi2_set, mzi3_set, mzi4_set, mzi1_max, mzi2_max, mzi3_max, mzi4_max, H_bench_actual, H_comb_actual, temp_ext1, temp_ext2 ]
    df_dict = dict(zip( output_labels, output))

    data_pd = pd.DataFrame.from_dict( df_dict, orient='index').transpose()
    
    output_date_stamp = time.strftime("_%Y_%m_%d at %H-%M")

    if len(frequency_range)>1 or n_repeat_for_stability >1:
        gc_read = gain_current(0) # turn off the laser after a channel scan or a stability test
        
        try :
            # Export the data taken
            output_address = export_pd(data_pd, folder_address = SN_selected_output_folder_address + measurement_types, file_name = data_title, header=True, date_label = output_date_stamp)
        except:
            print('Could not export the data, returning it here to save it.')
            simple_email(email_subject='Experiment Interrupted', email_body='Could not export the data, data is returned to save it')
            return data_pd

    if is_email_results and (len(frequency_range)>1 or n_repeat_for_stability >1):
        email_results(email_subject=data_title, attach_file_address = output_address)

    if is_measure_spectrum and (len(frequency_range)>1 or n_repeat_for_stability >1):
        # Export the spectra taken
        output_address = export_pd(spectrum_pd, folder_address = SN_selected_output_folder_address + measurement_types, file_name = data_title + '_spectra', header=True, date_label = output_date_stamp)
        
        if is_email_results and (len(frequency_range)>1 or n_repeat_for_stability >1):
            email_results(email_subject=data_title + '_spectra', attach_file_address = output_address)
    
    if len(frequency_range) > 1 or n_repeat_for_stability > 1: # if a long experiment is being performed, send email notification
        simple_email(email_subject='Experiment %s is done' %data_title, email_body='%s' %data_title)
        
    if is_locked_PI:# perform a locked PI 
        
        # Remove the lines below. Burak Aslan 2020-03-02
        #Optimized MZIs for SN26 at 191.7 THz and Comb=60 C, Bench=50 C
        #heater_heat('mzi 1', 15); heater_heat('mzi 2', 3); heater_heat('mzi 3', -9.5); heater_heat('mzi 4', 27)

        #Optimized MZIs for SN26 at 193.5 THz and Comb=104 C, Bench=50 C
        #heater_heat('mzi 1', -19); heater_heat('mzi 2', -21); heater_heat('mzi 3', -12.5); heater_heat('mzi 4', -23.5)
        
        #Optimized MZIs for SN26 at 196.2 THz and Comb=104 C, Bench=50 C
        #heater_heat('mzi 1', 1); heater_heat('mzi 2', 24); heater_heat('mzi 3', 25.5); heater_heat('mzi 4', 2)
        
        #Optimized MZIs for SN26 at 191.7 THz and Comb=94 C, Bench=50 C
        #heater_heat('mzi 1', 13.5); heater_heat('mzi 2', -2.5); heater_heat('mzi 3', -21.5); heater_heat('mzi 4', 6.5)
        
        
        #Optimized MZIs for SN31 at 193.5 THz and Comb=60 C, Bench=50 C
        #heater_heat('mzi 1', -20); heater_heat('mzi 2', 13); heater_heat('mzi 3', -11); heater_heat('mzi 4', -14.5)
        
        
        data_title = data_title.replace('Channel Scan_', '') # this is not a 'Channel Scan' anymore
        data_title = 'Locked PI_' + data_title +'_Comb %.3fC_by %smA per %ss' %(comb_temp_range[0], gc_step, gc_wait_time)
        
        locked_PI_output = bias_T_scan_gc(data_title = data_title,
                   gc_min = 0 , gc_max = gc_range[0], gc_step = gc_step, gc_wait_time= gc_wait_time,# Set min, max gain current, the step size in mA
                   temp_min = bench_temp_range[0], temp_max = bench_temp_range[0] + 0.1, temp_step = .2, # Set min, max temperature, the step size in Celcius
                   temp_fscs = 90, fscs_enabled = 'off', # fscs temperature 
                   temp_comb = comb_temp_range[0], comb_enabled = comb_enabled, # comb temperature
                   # Remove the line below and use the line above. Burak Aslan 2020-03-02
                   #temp_comb = 60, comb_enabled = comb_enabled, # comb temperature
                   bench_wait_time = bench_wait_time*2, # wait time between each step while scanning the temperature so that the loop stabilizes
                   mode_lock = 1, is_maximize_mzis = 0,
                   min_bench_temp = bench_temp_range[0]-10, max_bench_temp = bench_temp_range[0]+10, #for  safety check during mode_lock
                   measurement_types = 'Locked PIs'
            )
        return data_pd, locked_PI_output
    
    if is_ase_PI:# perform an amplified spontaneous emission (ASE) PI curve
        
        data_title = data_title.replace('Channel Scan_', '') # this is not a 'Channel Scan' anymore
        data_title = 'ASE PI_' + data_title +'_Comb %.3fC_by %smA per %ss' %(comb_temp_range[0], gc_step, gc_wait_time)
        
        ase_PI_output = bias_T_scan_gc(data_title = data_title,
                   gc_min = 0 , gc_max = gc_range[0], gc_step = gc_step, gc_wait_time= gc_wait_time,# Set min, max gain current, the step size in mA
                   temp_min = bench_temp_range[0], temp_max = bench_temp_range[0] + 0.1, temp_step = .2, # Set min, max temperature, the step size in Celcius
                   temp_fscs = 90, fscs_enabled = 'off', # fscs temperature 
                   temp_comb = comb_temp_range[0], comb_enabled = comb_enabled, # comb temperature
                   bench_wait_time = 3, # wait time between each step while scanning the temperature so that the loop stabilizes
                   mode_lock = 1, is_maximize_mzis = 0,
                   min_bench_temp = bench_temp_range[0]-10, max_bench_temp = bench_temp_range[0]+10, #for  safety check during mode_lock
                   measurement_types = 'ASE PIs'
            )
        return data_pd, ase_PI_output
    
    
    if cavity_phase_scan_range:
        data_title = data_title.replace('Channel Scan_', '') # this is not a 'Channel Scan' anymore
        data_title = 'Cavity Phase Scan_' + data_title +'+-%sC per %sC_Comb %.3fC' %(cavity_phase_scan_range, temp_step, comb_temp_range[0])
        cavity_phase_scan_output = bias_gc_scan_T(data_title=data_title,
                   gc_min = gc_range[0] , gc_max = gc_range[0], gc_step = -1, # Set min, max gain current, the step size in mA
                   temp_min = bench_temp_range[0] - cavity_phase_scan_range, temp_max = bench_temp_range[0] + cavity_phase_scan_range, temp_step = temp_step, # Set min, max temperature, the step size in Celcius
                   heater = 'bench2', # the heater to scan T
                   temp_bench2 = bench_temp_range[0] + cavity_phase_scan_range * np.sign(-temp_step), bench_enabled = 'on', # bench2 temperature
                   temp_fscs  = 25, fscs_enabled = 'off', # fscs temperature
                   temp_comb  = comb_temp_range[0], comb_enabled = comb_enabled, # comb temperature
                   heater_wait_time = 3, gc_wait_time = 3, measurement_types = 'Cavity Phase Scans' )
        return data_pd, cavity_phase_scan_output
    
    if ftf_scan_range:
        data_title = data_title.replace('Channel Scan_', '') # this is not a 'Channel Scan' anymore
        data_title = 'FTF Scan_' + data_title +'_Comb %.3fC +-%sC per %sC' %(comb_temp_range[0], ftf_scan_range, temp_step)
        ftf_scan_output = bias_gc_scan_T(data_title=data_title,
                   gc_min = gc_range[0] , gc_max = gc_range[0], gc_step = -1, # Set min, max gain current, the step size in mA
                   temp_min = comb_temp_range[0] - ftf_scan_range, temp_max = comb_temp_range[0] + ftf_scan_range, temp_step = temp_step, # Set min, max temperature, the step size in Celcius
                   heater = 'comb', # the heater to scan T
                   temp_bench2 = bench_temp_range[0], bench_enabled = 'on', # bench2 temperature
                   temp_fscs  = 25, fscs_enabled = 'off', # fscs temperature
                   temp_comb  = comb_temp_range[0] + ftf_scan_range*np.sign(-temp_step), comb_enabled = comb_enabled, # comb temperature
                   heater_wait_time = 3, gc_wait_time = 3, measurement_types = 'FTF Scans' )
        return data_pd, ftf_scan_output
                   
    return data_pd

In [ ]:
# Authored by Sergei 11-11-20
# set the laser to a certain frequency 
# and perform scans: Cavity Phase Scan, FTF (Fine Tune Frequency) Scan, Locked PI Curve, ASE (Amplified Spontaneous Emission) PI Curve
def set_laser_frequency_V2(frequency_range = [194], gc_range = [300], bench_temp_range = [50], 
                        dither = 'on', dither_off_wait_time = 10,
                        bench_mode = 'apd',
                        is_measure_spectrum = 0, # to measure spectrum with yokogawa 
                        is_maximize_mzis = 0, # to maximize the optical power with MZIs 
                        n_repeat_for_stability = 1, # # of times to measure some of the parameters for stability
                        stability_wait_time = 1,
                        # wait times
            frequency_set_wait_time = 15, bench_wait_time = 15, lock_wait_time = 10, mzi_maximize_wait_time = 1, yokogawa_sweep_wait_time = 10,
                        data_title = '', 
                        is_email_results = 1,
                        is_locked_PI = 0, is_ase_PI = 0, gc_step = -1, gc_wait_time = 3,
                        cavity_phase_scan_range = 0, ftf_scan_range = 0, temp_step = 0.01,
                        calibration_file_name = '',
                        comb_temp_file_name = '', comb_enabled = 'on',
                        measurement_types = 'Channel Scans'
                       ):
                        
    
    
    if not isinstance (frequency_range, list) and not isinstance (frequency_range, np.ndarray):
        frequency_range = [frequency_range]
    if not isinstance (gc_range, list) and not isinstance (gc_range, np.ndarray):
        gc_range = list([gc_range])
    if not isinstance (bench_temp_range, list) and not isinstance (bench_temp_range, np.ndarray):
        bench_temp_range = [bench_temp_range]
    
    # if user provided only one gc_range value but there may be more than 1 frequency values
    if len(gc_range) == 1 and  len(frequency_range)>1 : #    
        gc_range = gc_range * len(frequency_range)
        bench_temp_range = bench_temp_range * len(frequency_range)

    if data_title == '':
        label_frequency_range =  str('%.3f' %frequency_range[0])+'-'+str('%.3f' %frequency_range[-1] ) if len(frequency_range) > 1 else frequency_range[0]
        data_title = 'Channel Scan_%s_%sThz_%smA_Bench %sC_%s_locked' %(SN_selected, label_frequency_range, str(gc_range[0]), str(bench_temp_range[0]), bench_mode) 
        if n_repeat_for_stability >1:
            data_title = data_title.replace('Channel Scan', 'Stability Test')
            data_title += '_repeat x%s' %n_repeat_for_stability
            measurement_types = 'Stability Tests'
    
    print('Data title: %s' %data_title)
    
    if calibration_file_name == '':
        calibration_file_name = "%s_SIP calibration_bench %sC" %(SN_selected, bench_temp)

    print('Calibration file needed: %s.xlsx' %calibration_file_name)
    
    if comb_temp_file_name == '':
        comb_temp_file_name = '%s_Tune Table_bench %sC' %(SN_selected, bench_temp)
    print ('Comb temperature file needed: %s.xlsx' %comb_temp_file_name)

    try :
        power_meter.sense.power.dc.range.upper = 60/1000 # 60 mW, set to maximum range of optical power by PM100D
    except:
        print('Power meter range cannot be set')

    T_bench_actual = []; T_comb_actual = []; H_bench_actual = []; H_comb_actual = []
    mod_i_oop = []; mod_q_oop = []; mod_i_apd = []; mod_q_apd = []; mod_i_vgain = []; mod_q_vgain = [];
    power = []; frequency = []; time_measured = []; comb_temp_range = []; smsr = []
    mzi1_set = [] ; mzi2_set = [] ; mzi3_set = []; mzi4_set = []
    mzi1_max = [] ; mzi2_max = [] ; mzi3_max = []; mzi4_max = []
    
    temp_ext1 = []; temp_ext2 =[] # external temperatures measured by Thorlabs TPS01

    mpd_oop_before = [];  mpd_orp_before = []; mpd_ap_before = []; mpd_bp_before = []; mpd1_before = []; mpd2_before = []; mpd3_before = []; mpd4_before = []; 
    mpd_oop = [];  mpd_orp = []; mpd_ap = []; mpd_bp = []; mpd1 = []; mpd2 = []; mpd3 = []; mpd4 = []; 
    spectrum_pd = pd.DataFrame();

    start_time = time.time()
    
    board_io('bench2 enable on') # make sure bench2 is enabled
    #board_io( 'dac phi 2.0'); board_io( 'mod enable 1000 127 255') # use module-specific settings set in test notebook, section 3.1

    for i in np.arange(len(frequency_range)):
        
        ith_time = time.time()
        
        # Turn off the gain current first: tuning in the dark is a customer requirement
        gc_read = gain_current(0); print('Gain Current is %.3f mA' %gc_read)

        print('#%d of %d' %(i+1, len(frequency_range)*n_repeat_for_stability))
        set_bench_mode('temp'); heater_temp('bench2', bench_temp_range[i] )

        time.sleep(bench_wait_time)

        mzi1_heat, mzi2_heat , mzi3_heat, mzi4_heat, set_comb_temp = set_frequency_via_mzi_comb(frequency_range[i] , bench_temp_range[i], calibration_file_name, comb_temp_file_name)
        mzi1_set.append(mzi1_heat); mzi2_set.append(mzi2_heat); mzi3_set.append(mzi3_heat); mzi4_set.append(mzi4_heat); 
        comb_temp_range.append(set_comb_temp)


        time.sleep(frequency_set_wait_time)
        gc_read = gain_current( gc_range[i] )

        set_bench_mode(bench_mode)

        time.sleep(lock_wait_time)

        T_bench_actual_read = heater_temp('bench2')# bench2 Actual Temperature
        T_comb_actual_read = heater_temp('comb') # Comb Actual Temperature
        print( 'Actual Comb Temperature :%.3f °C, Bench:%.3f °C (right before scanning MZIs)' %(T_comb_actual_read,T_bench_actual_read))
        
        if is_maximize_mzis:
            
            # Read Photodiodes before optimizing the MZIs
            mpd_oop_read, mpd_orp_read, mpd_ap_read, mpd_bp_read, mpd1_read, mpd2_read, mpd3_read, mpd4_read = read_mpds()
            mpd_oop_before.append(mpd_oop_read); mpd_orp_before.append(mpd_orp_read); mpd_ap_before.append(mpd_ap_read); mpd_bp_before.append(mpd_bp_read) 
            mpd1_before.append(mpd1_read); mpd2_before.append(mpd2_read); mpd3_before.append(mpd3_read); mpd4_before.append(mpd4_read) 
            
            mzi1_heat_read = maximize_power_via_mzi(1, 0.5, mzi_maximize_wait_time); mzi2_heat_read = maximize_power_via_mzi(2, 0.5, mzi_maximize_wait_time)
            mzi3_heat_read = maximize_power_via_mzi(3, 0.5, mzi_maximize_wait_time); mzi4_heat_read = maximize_power_via_mzi(4, 0.5, mzi_maximize_wait_time)
            
            print('MZI Maximized [1, 2, 3, 4]: [%.2f, %.2f, %.2f, %.2f]' %(mzi1_heat_read,mzi2_heat_read,mzi3_heat_read,mzi4_heat_read))
            
            time.sleep(mzi_maximize_wait_time)
            
        else:
            [mzi1_heat_read, mzi2_heat_read, mzi3_heat_read, mzi4_heat_read] = [mzi1_heat, mzi2_heat , mzi3_heat, mzi4_heat]

        mzi1_max.append(mzi1_heat_read); mzi2_max.append(mzi2_heat_read); mzi3_max.append(mzi3_heat_read); mzi4_max.append(mzi4_heat_read)
        
        if dither == 'off':
            time.sleep(dither_off_wait_time)
            
            T_bench_average_read = heater_temp('bench2', -500)
            # switch to temp mode
            board_io('bench2 mode temp'); heater_temp('bench2', T_bench_average_read )
            
            # disable dither
            board_io( 'mod disable'); print('Dither is off now')

        if is_measure_spectrum:
            
            # Perform a Single Sweep
            Write_OSA(yokogawa, ':INITiate:SMODe SING;:INIT')
            time.sleep(yokogawa_sweep_wait_time) # so that the sweep will be done
            x_data_yokogawa, y_data_yokogawa = get_yokogawa_trace(yokogawa)
            
            # Read SMSR
            Write_OSA(yokogawa, ':CALCulate:CATegory SMSR;:CALCulate;:CALCulate:DATA?')
            smsr_read = float(Read_OSA(yokogawa).split(',')[-1])
            print('SMSR: %.3f dB' %smsr_read)
            smsr.append(smsr_read)
            x_data_yokogawa.insert(0, 'SMSR (dB)'); y_data_yokogawa.insert(0, smsr_read)
            
            if spectrum_pd.empty: # Add the x-axis only once to the data frame as they are identical
                spectrum_pd['%.3f_' %frequency_range[i] + 'Frequency (THz)' ] = x_data_yokogawa
                
            spectrum_pd['%.3f_' %frequency_range[i] + 'Intensity (dBm)' ] = y_data_yokogawa
        
        for k in range(n_repeat_for_stability):
            
            try:
            
                if n_repeat_for_stability>1:
                    print("Stability Test %s of %s" %(k+1,n_repeat_for_stability))
                    time.sleep(stability_wait_time)
                time_measured.append( time.strftime("%Y-%m-%d_%H:%M:%S") )

                T_bench_actual_read = heater_temp('bench2'); T_bench_actual.append(T_bench_actual_read) # bench2 Actual Temperature
                T_comb_actual_read = heater_temp('comb'); T_comb_actual.append(T_comb_actual_read) # Comb Actual Temperature

                H_bench_actual_read = heater_heat('bench2'); H_bench_actual.append(H_bench_actual_read) # bench2 current
                H_comb_actual_read = heater_heat('comb'); H_comb_actual.append(H_comb_actual_read) # Comb heat

                power_read = read_power()
                power.append(power_read)

                # TPS01 Thorlabs Temperature Sensor
                try:
                    temp_ext1_read = tsp01.measure_temperature('th0')
                    temp_ext2_read = tsp01.measure_temperature('th1')

                except:
                    temp_ext1_read = 0
                    temp_ext2_read = 0

                temp_ext1.append(temp_ext1_read)
                temp_ext2.append(temp_ext2_read)


                # Read Photodiodes
                mpd_oop_read, mpd_orp_read, mpd_ap_read, mpd_bp_read, mpd1_read, mpd2_read, mpd3_read, mpd4_read = read_mpds()
                mpd_oop.append(mpd_oop_read); mpd_orp.append(mpd_orp_read); mpd_ap.append(mpd_ap_read); mpd_bp.append(mpd_bp_read) 
                mpd1.append(mpd1_read); mpd2.append(mpd2_read); mpd3.append(mpd3_read); mpd4.append(mpd4_read) 

                # Read modulation values
                mod_i_oop_read, mod_q_oop_read, mod_i_apd_read, mod_q_apd_read, mod_i_vgain_read, mod_q_vgain_read = read_modulation()
                mod_i_oop.append(mod_i_oop_read); mod_q_oop.append(mod_q_oop_read); mod_i_apd.append(mod_i_apd_read); mod_q_apd.append(mod_q_apd_read); mod_i_vgain.append(mod_i_vgain_read); mod_q_vgain.append(mod_q_vgain_read);

                frequency_read = read_frequency()
                frequency.append(frequency_read)
            except:
                print('Experiment %s is interrupted' %data_title)
                simple_email(email_subject='Experiment %s is interrupted' %data_title, email_body='%s' %data_title)


        
        print( 'Comb:%.3f °C, Bench:%.3f °C, %.3f mA P.current, %.3f mW Opt.Power, %.5f THz, %.1f mA GC, Mod i oop: %.3f\n' %(T_comb_actual_read,T_bench_actual_read, mpd_oop_read, power_read, frequency_read, gc_read, mod_i_oop_read))

        jth_time = time.time()
        print('Scan #%d took %.1f seconds' %(i+1, jth_time-ith_time) )
    elapsed_time = time.time() - start_time
    print('Scanning took %.1f seconds (= %.2f hours)' %(elapsed_time, elapsed_time/3600) )

    # convert the data collected to a data frame
    output_labels = ['Time',  'Set Frequency (THz)', 'Set Bench Temperature (°C)', 'Set Comb Temperature (°C)' ,  'Gain Current (mA)', 
                     #'MPD_OOP before MZI opt.(mA)','MPD_ORP before MZI opt.(mA)','MPD_AP before MZI opt.(mA)','MPD_BP before MZI opt.(mA)','MPD1 before MZI opt.(mA)','MPD2 before MZI opt.(mA)','MPD3 before MZI opt.(mA)','MPD4 before MZI opt.(mA)',
                     'MPD_OOP (mA)','MPD_ORP (mA)','MPD_AP (mA)','MPD_BP (mA)','MPD1 (mA)','MPD2 (mA)','MPD3 (mA)','MPD4 (mA)', 
                     'Optical Power (mW)', 'Frequency (THz)', 'SMSR', 'Mod i oop', 'Mod q oop',
                     'Bench Temperature (°C)', 'Comb Temperature (°C)', 'Calculated MZI 1 Heat (mW)', 'Calculated MZI 2 Heat (mW)', 'Calculated MZI 3 Heat (mW)', 'Calculated MZI 4 Heat (mW)', 
                     'MZI 1 Heat Max. (mW)', 'MZI 2 Heat Max. (mW)', 'MZI 3 Heat Max. (mW)', 'MZI 4 Heat Max. (mW)',
                     'Bench Current (mA)', 'Comb Heat (mW)', 'External Temp 1 (°C)', 'External Temp 2 (°C)']
                         
    output = [time_measured, frequency_range, bench_temp_range, comb_temp_range, gc_range, mpd_oop, mpd_orp, mpd_ap, mpd_bp, mpd1, mpd2, mpd3, mpd4, power, frequency, smsr, mod_i_oop, mod_q_oop, T_bench_actual , T_comb_actual, mzi1_set, mzi2_set, mzi3_set, mzi4_set, mzi1_max, mzi2_max, mzi3_max, mzi4_max, H_bench_actual, H_comb_actual, temp_ext1, temp_ext2 ]
    df_dict = dict(zip( output_labels, output))

    data_pd = pd.DataFrame.from_dict( df_dict, orient='index').transpose()
    
    output_date_stamp = time.strftime("_%Y_%m_%d at %H-%M")

    if len(frequency_range)>1 or n_repeat_for_stability >1:
        gc_read = gain_current(0) # turn off the laser after a channel scan or a stability test
        
        try :
            # Export the data taken
            output_address = export_pd(data_pd, folder_address = SN_selected_output_folder_address + measurement_types, file_name = data_title, header=True, date_label = output_date_stamp)
        except:
            print('Could not export the data, returning it here to save it.')
            simple_email(email_subject='Experiment Interrupted', email_body='Could not export the data, data is returned to save it')
            return data_pd

    if is_email_results and (len(frequency_range)>1 or n_repeat_for_stability >1):
        email_results(email_subject=data_title, attach_file_address = output_address)

    if is_measure_spectrum and (len(frequency_range)>1 or n_repeat_for_stability >1):
        # Export the spectra taken
        output_address = export_pd(spectrum_pd, folder_address = SN_selected_output_folder_address + measurement_types, file_name = data_title + '_spectra', header=True, date_label = output_date_stamp)
        
        if is_email_results and (len(frequency_range)>1 or n_repeat_for_stability >1):
            email_results(email_subject=data_title + '_spectra', attach_file_address = output_address)
    
    if len(frequency_range) > 1 or n_repeat_for_stability > 1: # if a long experiment is being performed, send email notification
        simple_email(email_subject='Experiment %s is done' %data_title, email_body='%s' %data_title)
        
    if is_locked_PI:# perform a locked PI 
        
        # Remove the lines below. Burak Aslan 2020-03-02
        #Optimized MZIs for SN26 at 191.7 THz and Comb=60 C, Bench=50 C
        #heater_heat('mzi 1', 15); heater_heat('mzi 2', 3); heater_heat('mzi 3', -9.5); heater_heat('mzi 4', 27)

        #Optimized MZIs for SN26 at 193.5 THz and Comb=104 C, Bench=50 C
        #heater_heat('mzi 1', -19); heater_heat('mzi 2', -21); heater_heat('mzi 3', -12.5); heater_heat('mzi 4', -23.5)
        
        #Optimized MZIs for SN26 at 196.2 THz and Comb=104 C, Bench=50 C
        #heater_heat('mzi 1', 1); heater_heat('mzi 2', 24); heater_heat('mzi 3', 25.5); heater_heat('mzi 4', 2)
        
        #Optimized MZIs for SN26 at 191.7 THz and Comb=94 C, Bench=50 C
        #heater_heat('mzi 1', 13.5); heater_heat('mzi 2', -2.5); heater_heat('mzi 3', -21.5); heater_heat('mzi 4', 6.5)
        
        
        #Optimized MZIs for SN31 at 193.5 THz and Comb=60 C, Bench=50 C
        #heater_heat('mzi 1', -20); heater_heat('mzi 2', 13); heater_heat('mzi 3', -11); heater_heat('mzi 4', -14.5)
        
        
        data_title = data_title.replace('Channel Scan_', '') # this is not a 'Channel Scan' anymore
        data_title = 'Locked PI_' + data_title +'_Comb %.3fC_by %smA per %ss' %(comb_temp_range[0], gc_step, gc_wait_time)
        
        locked_PI_output = bias_T_scan_gc_V2(data_title = data_title,
                   gc_min = 300 , gc_max = gc_range[0], gc_step = gc_step, gc_wait_time= gc_wait_time,# Set min, max gain current, the step size in mA
                   temp_min = bench_temp_range[0], temp_max = bench_temp_range[0] + 0.1, temp_step = .2, # Set min, max temperature, the step size in Celcius
                   temp_fscs = 90, fscs_enabled = 'off', # fscs temperature 
                   temp_comb = comb_temp_range[0], comb_enabled = comb_enabled, # comb temperature
                   # Remove the line below and use the line above. Burak Aslan 2020-03-02
                   #temp_comb = 60, comb_enabled = comb_enabled, # comb temperature
                   bench_wait_time = bench_wait_time*2, # wait time between each step while scanning the temperature so that the loop stabilizes
                   mode_lock = 1, is_maximize_mzis = 0,
                   min_bench_temp = bench_temp_range[0]-10, max_bench_temp = bench_temp_range[0]+20, #for  safety check during mode_lock
                   measurement_types = 'Locked PIs'
            )
        return data_pd, locked_PI_output
    
    if is_ase_PI:# perform an amplified spontaneous emission (ASE) PI curve
        
        data_title = data_title.replace('Channel Scan_', '') # this is not a 'Channel Scan' anymore
        data_title = 'ASE PI_' + data_title +'_Comb %.3fC_by %smA per %ss' %(comb_temp_range[0], gc_step, gc_wait_time)
        
        ase_PI_output = bias_T_scan_gc(data_title = data_title,
                   gc_min = 0 , gc_max = gc_range[0], gc_step = gc_step, gc_wait_time= gc_wait_time,# Set min, max gain current, the step size in mA
                   temp_min = bench_temp_range[0], temp_max = bench_temp_range[0] + 0.1, temp_step = .2, # Set min, max temperature, the step size in Celcius
                   temp_fscs = 90, fscs_enabled = 'off', # fscs temperature 
                   temp_comb = comb_temp_range[0], comb_enabled = comb_enabled, # comb temperature
                   bench_wait_time = 3, # wait time between each step while scanning the temperature so that the loop stabilizes
                   mode_lock = 1, is_maximize_mzis = 0,
                   min_bench_temp = bench_temp_range[0]-10, max_bench_temp = bench_temp_range[0]+10, #for  safety check during mode_lock
                   measurement_types = 'ASE PIs'
            )
        return data_pd, ase_PI_output
    
    
    if cavity_phase_scan_range:
        data_title = data_title.replace('Channel Scan_', '') # this is not a 'Channel Scan' anymore
        data_title = 'Cavity Phase Scan_' + data_title +'+-%sC per %sC_Comb %.3fC' %(cavity_phase_scan_range, temp_step, comb_temp_range[0])
        cavity_phase_scan_output = bias_gc_scan_T(data_title=data_title,
                   gc_min = gc_range[0] , gc_max = gc_range[0], gc_step = -1, # Set min, max gain current, the step size in mA
                   temp_min = bench_temp_range[0] - cavity_phase_scan_range, temp_max = bench_temp_range[0] + cavity_phase_scan_range, temp_step = temp_step, # Set min, max temperature, the step size in Celcius
                   heater = 'bench2', # the heater to scan T
                   temp_bench2 = bench_temp_range[0] + cavity_phase_scan_range * np.sign(-temp_step), bench_enabled = 'on', # bench2 temperature
                   temp_fscs  = 25, fscs_enabled = 'off', # fscs temperature
                   temp_comb  = comb_temp_range[0], comb_enabled = comb_enabled, # comb temperature
                   heater_wait_time = 3, gc_wait_time = 3, measurement_types = 'Cavity Phase Scans' )
        return data_pd, cavity_phase_scan_output
    
    if ftf_scan_range:
        data_title = data_title.replace('Channel Scan_', '') # this is not a 'Channel Scan' anymore
        data_title = 'FTF Scan_' + data_title +'_Comb %.3fC +-%sC per %sC' %(comb_temp_range[0], ftf_scan_range, temp_step)
        ftf_scan_output = bias_gc_scan_T(data_title=data_title,
                   gc_min = gc_range[0] , gc_max = gc_range[0], gc_step = -1, # Set min, max gain current, the step size in mA
                   temp_min = comb_temp_range[0] - ftf_scan_range, temp_max = comb_temp_range[0] + ftf_scan_range, temp_step = temp_step, # Set min, max temperature, the step size in Celcius
                   heater = 'comb', # the heater to scan T
                   temp_bench2 = bench_temp_range[0], bench_enabled = 'on', # bench2 temperature
                   temp_fscs  = 25, fscs_enabled = 'off', # fscs temperature
                   temp_comb  = comb_temp_range[0] + ftf_scan_range*np.sign(-temp_step), comb_enabled = comb_enabled, # comb temperature
                   heater_wait_time = 3, gc_wait_time = 3, measurement_types = 'FTF Scans' )
        return data_pd, ftf_scan_output
                   
    return data_pd

# Transient Measurement

In [21]:
def measure_transient(N= 10 # how many data points
                     ):

    time_measured = []; frequency = []; power =[]

    for i in np.arange(N):

        time_measured.append( time.strftime("%Y-%m-%d_%H:%M:%S") )

        frequency_read = read_frequency()
        frequency.append(frequency_read)

        power_read = read_power()
        power.append(power_read)

    output = np.transpose([time_measured, frequency, power])

    return output

# FSCS Temperature Scans

In [22]:
def scan_fscs_temp(data_title='',
                   temp_min = 25, temp_max = 26, temp_step = -1, # fscs temperature range
                   temp_bench2 = 25, # bench2 temperature 
                   temp_comb = 25, # comb temperature
                   gc_min = 25 , gc_max = 50, gc_step = -25, # Set min, max gain current, the step size in mA
                   wait_time = 3, # wait time between each step while scanning the temperature so that the loop stabilizes
):
    
    if data_title == '':
        raise ValueError('Always provide data_title')
    print('Data title: %s' %data_title)        
    
    start_time = time.time()

    # Prepare the Temperature and Gain Current Ranges
    temp_range = np.arange(temp_max, temp_min + temp_step, temp_step) if temp_step < 0 else np.arange(temp_min, temp_max + temp_step, temp_step)    
    gc_range = np.arange(gc_max, gc_min + gc_step, gc_step) if gc_step < 0 else np.arange(gc_min, gc_max + gc_step, gc_step) 

    if len(temp_range) < 2 or len(gc_range) < 2:
        raise ValueError('Length of Temperature and GC Ranges need to be at least 2')
        
    temp_fscs = temp_range[0] # target is the first temperature value

    heater_temp('fscs', temp_fscs)
    board_io( 'fscs enable ' + fscs_enabled) # Enable the heater
    
    heater_temp('bench2', temp_bench2 )
    board_io( 'bench2 enable '+ bench_enabled)  # Enable bench

    heater_temp('fscs', temp_comb)
    board_io( 'comb enable ' + comb_enabled) # Enable the heater

    mpd_temp = []; power_temp = []; temp_diff = []; frequency_temp = []; gc = []; mod_i_oop_temp = []; mod_q_oop_temp= []
    frequency_read = 0 ; temp = []; 
    fscs_heat_temp = []; comb_heat_temp = []
    T_fscs_actual_temp = []; T_bench_actual_temp = []; T_comb_actual_temp = [] 

    try :
        power_meter.sense.power.dc.range.upper = 60/1000 # 60 mW, set to maximum range of optical power by PM100D
    except:
        print('Power meter range cannot be set')

    for gc_write in gc_range:

        # change to the first value of temp_range and wait
        # so that this abrupt change will not cause any abnomalities
        heater_temp('fscs', temp_range[0])

        gc_read = gain_current(gc_write); gc.append(gc_read); print('Gain Current is %.3f mA' %gc_read)
        
        time.sleep(10) # so that the spectrum stabilizes 

        power = []; mpd = []; frequency = []; mod_i_oop = []; mod_q_oop = []; 
        fscs_heat=[]; comb_heat=[]
        T_fscs_actual = []; T_bench_actual = []; T_comb_actual = [] 


        for temp_write in temp_range:

            heater_temp('fscs', temp_write)
            time.sleep(wait_time) # so that temperature stabilizes 

            # FSCS Actual Temperature
            T_fscs_actual_read = heater_temp('fscs')
            T_fscs_actual.append(T_fscs_actual_read)

            temp_diff.append( abs(temp_write - T_fscs_actual_read) ) # difference between the input and the read temp value
            
            # bench2 Actual Temperature
            T_bench_actual_read = heater_temp('bench2')
            T_bench_actual.append(T_bench_actual_read)

            # Comb Actual Temperature
            T_comb_actual_read = heater_temp('comb')
            T_comb_actual.append(T_comb_actual_read)

            ### Measurements ###
            power_read = read_power()

            power.append(power_read)

            # Optical Power and Peak frequency from Bristol 438A Wavelength Meter
            #power_bristol.append(float(bristol.getSimpleMsg(b':READ:SCAL:ALL?').split(', ')[3]))
            frequency_read = read_frequency()
            frequency.append(frequency_read)

            # Read oop, connected to mpd photodiode on TV0, adc to read may be 'oop' or 'ap'
            mpd_read = board_io( 'adc oop')[-1]
            mpd_read = float (re.findall('[0-9-.]+', mpd_read)[0]) # extract the mpd value from the returned string
            mpd.append(mpd_read) 

            # Read modulation values
            mod_i_oop_read, mod_q_oop_read, mod_i_apd_read, mod_q_apd_read, mod_i_vgain_read, mod_q_vgain_read = read_modulation()
            mod_i_oop.append(mod_i_oop_read); mod_q_oop.append(mod_q_oop_read); mod_i_apd.append(mod_i_apd_read); mod_q_apd.append(mod_q_apd_read); mod_i_vgain.append(mod_i_vgain_read); mod_q_vgain.append(mod_q_vgain_read);

            
            fscs_heat_read = heater_heat('fscs')
            fscs_heat.append(fscs_heat_read)

            comb_heat_read = heater_heat('comb')
            comb_heat.append(comb_heat_read)

            print( '%.2f mA P.current, %.2f mW Opt. Power, %.5f THz, %.1f mA GC, target %.2f °C (actual %.2f °C)' %(mpd_read, power_read, frequency_read, gc_read, temp_write,T_fscs_actual_read))

        mod_i_oop_temp.append(mod_i_oop); mod_q_oop_temp.append(mod_q_oop); 
        mpd_temp.append(mpd); power_temp.append(power); frequency_temp.append(frequency)
        
        T_fscs_actual_temp.append(T_fscs_actual); T_bench_actual_temp.append(T_bench_actual); T_comb_actual_temp.append(T_comb_actual)
        fscs_heat_temp.append(fscs_heat); comb_heat_temp.append(comb_heat)

        elapsed_time = time.time() - start_time
        print('Elapsed time %.1f seconds (%s)' %(elapsed_time, str(datetime.timedelta(seconds=elapsed_time)) ))

    print('Max difference between target & actual T = %.3f °C' %max(temp_diff))

    # Set the FSCS back to temp_min
    heater_temp('fscs' %min(temp_range))
    print('FSCS is set to %s' %min(temp_range))

    # Turn off the GC
    gc_read = gain_current(0)
    print('Gain Current is %.3f mA now' %gc_read)
    elapsed_time = time.time() - start_time
    print('All took %.1f seconds (%s)' %(elapsed_time,  str(datetime.timedelta(seconds=elapsed_time)) ) )

    # Transpose the measured arrays because the scan direction is Bias GC, Scan T
    [mpd_temp, power_temp, frequency_temp, mod_i_oop_temp, mod_q_oop_temp] = transpose_lists([mpd_temp, power_temp, frequency_temp, mod_i_oop_temp, mod_q_oop_temp])
    [T_fscs_actual_temp, T_bench_actual_temp, T_comb_actual_temp, fscs_heat_temp, comb_heat_temp] =  transpose_lists([T_fscs_actual_temp, T_bench_actual_temp, T_comb_actual_temp, fscs_heat_temp, comb_heat_temp])
    
    return [temp_range, gc_range, mpd_temp, power_temp, frequency_temp, mod_i_oop_temp, mod_q_oop_temp, T_fscs_actual_temp, T_bench_actual_temp, T_comb_actual_temp, fscs_heat_temp, comb_heat_temp]

# Export PIT Data

In [23]:
def export_pit_data(outputs, data_title = '', folder_address = SN_selected_output_folder_address, no_of_data = 1 ):

    if data_title == '' :
        raise ValueError('Always provide data_title')
    print('Data title: %s' %data_title)
    
    output_labels  = ['Time', 'Temperature (°C or mW)', 'Gain Current (mA)', 'Gain Voltage (V)', 'Laser Voltage (V)', 'MPD_OOP (mA)','MPD_ORP (mA)','MPD_AP (mA)','MPD_BP (mA)','MPD1 (mA)','MPD2 (mA)','MPD3 (mA)','MPD4 (mA)', 'Optical Power (mW)', 'Frequency (THz)', 
                      'Mod i oop', 'Mod q oop', 'Mod i apd', 'Mod q apd', 'Mod i vgain', 'Mod q vgain',  
                      'FSCS Temperature (°C)', 'bench2 Temperature (°C)', 'Comb Temperature (°C)',
                      'FSCS Heat (mW)', 'Comb Heat (mW)',
                      'MZI 1 Heat Max. (mW)', 'MZI 2 Heat Max. (mW)', 'MZI 3 Heat Max. (mW)', 'MZI 4 Heat Max. (mW)',
                      'External Temp 1 (°C)', 'External Temp 2 (°C)']
    
    flat_outputs = []; data_pd = pd.DataFrame(columns = output_labels)

    for i in range(no_of_data):
        
        if no_of_data >1:
            output = outputs[i]
        else:
            output = outputs
            
        [time_measured, temp_range, gc_range, vgain_temp, vl_temp, 
         mpd_oop_temp, mpd_orp_temp, mpd_ap_temp, mpd_bp_temp, mpd1_temp, mpd2_temp, mpd3_temp, mpd4_temp, 
         power_temp, frequency_temp, 
         mod_i_oop_temp, mod_q_oop_temp, mod_i_apd_temp, mod_q_apd_temp, mod_i_vgain_temp, mod_q_vgain_temp,
         T_fscs_actual_temp, T_bench_actual_temp, T_comb_actual_temp, 
         fscs_heat_temp, comb_heat_temp, 
         mzi1_max_temp, mzi2_max_temp, mzi3_max_temp, mzi4_max_temp,
         temp_ext1_temp, temp_ext2_temp] = output


        temp_range_expanded = np.repeat(temp_range, len(gc_range))
        gc_range_expanded = list(gc_range) * len(temp_range) #np.repeat(gc_range,len(temp_range))


        # flatten the lists to convert them properly to a dataframe column
        [ vgain_temp_expanded, vl_temp_expanded, mpd_oop_temp_expanded, mpd_orp_temp_expanded, mpd_ap_temp_expanded, mpd_bp_temp_expanded, 
        mpd1_temp_expanded, mpd2_temp_expanded, mpd3_temp_expanded, mpd4_temp_expanded,
        power_temp_expanded, frequency_temp_expanded,
        mod_i_oop_temp_expanded, mod_q_oop_temp_expanded,
        mod_i_apd_temp_expanded, mod_q_apd_temp_expanded,
        mod_i_vgain_temp_expanded, mod_q_vgain_temp_expanded,
        T_fscs_actual_temp_expanded, T_bench_actual_temp_expanded, T_comb_actual_temp_expanded,
        fscs_heat_temp_expanded, comb_heat_temp_expanded, 
        mzi1_max_temp_expanded, mzi2_max_temp_expanded, 
        mzi3_max_temp_expanded, mzi4_max_temp_expanded,
        temp_ext1_temp_expanded, temp_ext2_temp_expanded ] = [ list(itertools.chain.from_iterable(list_temp)) for list_temp in 
                                                                [vgain_temp, vl_temp, mpd_oop_temp, mpd_orp_temp, mpd_ap_temp, mpd_bp_temp, 
                                                                mpd1_temp, mpd2_temp, mpd3_temp, mpd4_temp, 
                                                                power_temp, frequency_temp, 
                                                                mod_i_oop_temp, mod_q_oop_temp, 
                                                                mod_i_apd_temp, mod_q_apd_temp, 
                                                                mod_i_vgain_temp, mod_q_vgain_temp,
                                                                T_fscs_actual_temp, T_bench_actual_temp, T_comb_actual_temp, 
                                                                fscs_heat_temp, comb_heat_temp,
                                                                mzi1_max_temp, mzi2_max_temp, mzi3_max_temp, mzi4_max_temp,
                                                                temp_ext1_temp, temp_ext2_temp]
                                                                      ]

        flat_output = [time_measured, temp_range_expanded, gc_range_expanded, vgain_temp_expanded, vl_temp_expanded, mpd_oop_temp_expanded, mpd_orp_temp_expanded, mpd_ap_temp_expanded, mpd_bp_temp_expanded, mpd1_temp_expanded, mpd2_temp_expanded, mpd3_temp_expanded, mpd4_temp_expanded,
        power_temp_expanded, frequency_temp_expanded,
                mod_i_oop_temp_expanded, mod_q_oop_temp_expanded,
                  mod_i_apd_temp_expanded, mod_q_apd_temp_expanded,
                   mod_i_vgain_temp_expanded, mod_q_vgain_temp_expanded,
                  T_fscs_actual_temp_expanded, T_bench_actual_temp_expanded, T_comb_actual_temp_expanded,
                fscs_heat_temp_expanded, comb_heat_temp_expanded,
                mzi1_max_temp_expanded, mzi2_max_temp_expanded, 
                 mzi3_max_temp_expanded, mzi4_max_temp_expanded,
                  temp_ext1_temp_expanded, temp_ext2_temp_expanded]


        flat_output_pd = pd.DataFrame( np.transpose(flat_output), columns = output_labels) 
        data_pd = pd.concat([data_pd, flat_output_pd])
        
    
    output_date_stamp = time.strftime("_%Y_%m_%d at %H-%M")
    output_address = export_pd(data_pd, folder_address = folder_address, file_name = data_title, header=True, date_label = output_date_stamp)

    return output_address, output_date_stamp

# Print Thermal, Optical Properties

In [24]:
def print_thermal_optical_state():
    
    print_thermal_state()
    print_optical_state()
    
def print_thermal_state():
    
    bench_mode = set_bench_mode()
    
    try:
        mod_iq = board_io( 'mod iq %s' %bench_mode)[-1]
        mod_i = re.findall('[0-9-.]+', mod_iq)[0];
    except:
        mod_i = 'None'
    
    bench_control = heater_heat('bench2'); bench_temp = heater_temp('bench2')
    comb_heat = heater_heat('comb'); comb_temp = heater_temp('comb')
    fscs_heat = heater_heat('fscs'); fscs_temp = heater_temp('fscs')
    
    # Heater Temperatures
    print(tabulate ( [['i', mod_i, bench_mode + ' mode']] ) )
    
    print(tabulate([['Bench2 (%s)' %bench_mode, bench_temp, '°C',bench_control, 'A'], 
                    ['Comb', comb_temp, '°C', comb_heat, 'mW'], 
                    ['FSCS', fscs_temp, '°C', fscs_heat, 'mW']
                   ], 
                   headers=['Heater/TEC', 'Temperature', 'Unit', 'Current/Heat', 'Unit'],  floatfmt=['', '.4f' ,  ".3f"] ) )   
    
    
def print_heats():
    # Heater/TEC Current/Powers   
    print(tabulate([['Bench2', heater_heat('bench2'), 'A'], 
                    ['Comb', heater_heat('comb'), 'mW'], 
                    ['FSCS', heater_heat('fscs'), 'mW']
                   ], 
                   headers=['Heater/TEC', 'Current/Heat', 'Unit'],  floatfmt=".3f") )  
def read_frequency():
    
    try :
        frequency_read = float(hp86120c.ask(":MEAS:SCAL:POW:FREQ? MAX").split('\n')[0])*10**-12
    except:
        try:
            frequency_read = float(bristol.getSimpleMsg(b':READ:SCAL:ALL?').split(', ')[2])
        except:
            frequency_read = 0
    if frequency_read >197 or frequency_read <191:
        frequency_read = 0
    return frequency_read

def read_power():
    
    try:
        # Optical Power from Thorlabs PM100D
        power_read = power_meter.read * 1000 # converted to mW
        if power_read > 1000 : # if the PM100D is saturated
            power_read = 0
       
    except:
        try:
            # Optical Power from Agilent HP86120C Power Meter
            power_read = 10**(float(hp86120c.ask(":MEAS:SCAL:POW? MAX").split('\n')[0])/10)
            
        except:
            power_read = 0

    return power_read

def print_optical_state():
    
    frequency_read = read_frequency()
    power_read = read_power()
    
    print(tabulate([['Gain Current', '%.3f' %gain_current(), 'mA'], 
                    ['Optical Power', power_read, 'mW'], 
                    ['Frequency', frequency_read, 'THz']
                   ], 
                   floatfmt=".6f") )   

# Plotting Temperature and Gain Chip Current Sweep Results
Photocurrent in a photodiode, Optical Power Meter and Peak Frequency.  
Also exports to slides in a powerpoint file.

In [25]:
def plot_pit_sweeps(temp, gc, mpd_temp, power_temp, frequency_temp, mod_i_oop_temp, mod_q_oop_temp, mod_i_apd_temp, mod_q_apd_temp,
                    mpd1_temp = [], mpd2_temp = [], mpd3_temp = [], mpd4_temp = [],
                    T_bench_actual_temp = [], T_comb_actual_temp = [],
                    temp_step_plot_factor=1, gc_step_plot_factor=1, # to skip some of the temperature and gc values to be plotted
                    heater = 'bench2',
                    file_name_powerpoint = '', # export the figures to slides if file_name_powerpoint is specified
                    data_title = '', output_date_stamp='',
                    folder_address = SN_selected_output_folder_address
                   ):
    
    folder_address = folder_address + '\\'
    

    gc_axis_label = 'Gain Current (mA)';  temp_axis_label = 'Temperature (°C)' ;  comb_temp_axis_label = 'Comb Temperature (°C)' ; 
    heat_axis_label = 'Heat (mW)'
    mod_iq_oop_axis_label = 'Mod iq oop'; mod_i_oop_axis_label = 'Mod i oop';
    mod_iq_apd_axis_label = 'Mod iq apd'; mod_i_apd_axis_label = 'Mod i apd';
    mpd_axis_label = 'Photocurrent (mA)'; power_axis_label = 'Optical Power (mW)'; frequency_axis_label = 'Frequency (THz)'
    delta_mpd_axis_label = 'ΔPhotocurrent (mA)'; delta_power_axis_label = 'ΔOptical Power (mW)'; delta_frequency_axis_label = 'ΔFrequency (THz)'    
    
    if 'mzi' in heater: # Correct the label for Temperature/Heat if MZIs are used.
        temp_axis_label = heat_axis_label
    
    figsize = (20,14); fontsize = 20
    plt.rcParams['axes.linewidth'] = 3 #set the value globally
    
    line_kwargs = {'ls' :'-', 'lw' : 1, 'ms' : 1, 'marker' : '', 'markeredgewidth' : 2, 'markerfacecolor' : 'None'}
    grid_kwargs = {'color':'gray', 'ls': '--', 'lw': 1, 'axis' : 'both'}
    tick_params = {'direction' : 'in', 'labelsize' : fontsize/1.5}
    temp_color_map = []
    
    if temp_step_plot_factor > len(temp):
        ValueError('temp_step_plot_factor can not be larger than the number of temperature points')
    if gc_step_plot_factor > len(gc):
        ValueError('gc_step_plot_factor can not be larger than the number of GC points')

    if len(gc) > 1: # this is not a 1D T scan, we can plot as a function of GC
        
        fig, ax = plt.subplots(figsize=figsize, constrained_layout=True)
        fig.suptitle(data_title, y=1.03, fontsize = fontsize)

        for i in range(0, len(temp), temp_step_plot_factor):

            temp_plot_range =  temp[0:len(temp):temp_step_plot_factor] # the temperature values being plotted


            try :

                if temp[0] < temp[1]: # if temperature decreases in the scan
                    RGB = ( i/len(temp), 0, 1 - i/len(temp) ) # Lowest T = blue
                else :                                                         
                    RGB = ( 1 - i/len(temp), 0, i/len(temp) ) # Lowest T = blue
                # Setting up a colormap that's same as the line colors
                temp_color_map_i = (temp[i] - min(temp_plot_range))/(max(temp_plot_range) - min(temp_plot_range))
                temp_color_map.append(  (temp_color_map_i, RGB ) ) 

                line_kwargs['color'] = RGB
            except :
                pass

            line_kwargs['label'] = '%.1f' %temp[i]

            ax=plt.subplot(241)
            ax.plot(gc, mpd_temp[i], **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
            #ax.legend(title = temp_axis_label, loc='best', fontsize = fontsize/1.5)
            ax.set_xlabel(gc_axis_label, fontsize = fontsize); ax.set_ylabel(mpd_axis_label, fontsize = fontsize)

            ax=plt.subplot(242)
            ax.plot(gc, power_temp[i],  **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
            ax.set_xlabel(gc_axis_label, fontsize = fontsize); ax.set_ylabel(power_axis_label, fontsize = fontsize)

            ax=plt.subplot(243)
            ax.plot(gc, frequency_temp[i],  **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
            ax.set_xlabel(gc_axis_label, fontsize = fontsize); ax.set_ylabel(frequency_axis_label, fontsize = fontsize)

            ax=plt.subplot(244)
            ax.plot(gc, T_bench_actual_temp[i],  **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
            ax.set_xlabel(gc_axis_label, fontsize = fontsize); ax.set_ylabel(temp_axis_label, fontsize = fontsize)
            
            ax=plt.subplot(245)
            ax.plot(gc, T_comb_actual_temp[i],  **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
            ax.set_xlabel(gc_axis_label, fontsize = fontsize); ax.set_ylabel(comb_temp_axis_label, fontsize = fontsize)

            RGB_mod_q =  ( 1- i/len(temp), 0.5, 0 ); RGB_mod_i = ( 0, 0.5, 1 - i/len(temp) ) # use different colors for mod_i and mod_q

            ax=plt.subplot(247)
            line_kwargs['color'] = RGB_mod_i
            ax.plot(gc, mod_i_oop_temp[i],  **line_kwargs); ax.grid(**grid_kwargs); #ax.tick_params(**tick_params)

            line_kwargs['color'] = RGB_mod_q 
            ax.plot(gc, mod_q_oop_temp[i],  **line_kwargs); ax.grid(**grid_kwargs); #ax.tick_params(**tick_params)
            ax.set_xlabel(gc_axis_label, fontsize = fontsize); ax.set_ylabel(mod_iq_oop_axis_label, fontsize = fontsize)

            ax=plt.subplot(248)
            line_kwargs['color'] = RGB_mod_i 
            ax.plot(gc, mod_i_apd_temp[i] ,  **line_kwargs); ax.grid(**grid_kwargs); #ax.tick_params(**tick_params)

            line_kwargs['color'] = RGB_mod_q # use different colors for mod_i and mod_q
            ax.plot(gc, mod_q_apd_temp[i] ,  **line_kwargs); ax.grid(**grid_kwargs); #ax.tick_params(**tick_params)
            ax.set_xlabel(gc_axis_label, fontsize = fontsize); ax.set_ylabel(mod_iq_apd_axis_label, fontsize = fontsize)


        try :
            if temp[1] < temp[0]: # if temperature is decreasing during the scan
                temp_color_map = temp_color_map[::-1] # invert the order of temperature color map

            my_cmap = mpl.colors.LinearSegmentedColormap.from_list( 'mycolors', temp_color_map )
            sm = plt.cm.ScalarMappable(cmap = my_cmap, norm = plt.Normalize( vmin = temp[0], vmax = temp[i] ) )
            # fake up the array of the scalar mappable.
            sm._A = []

            plt.colorbar(sm, label=temp_axis_label)
        except:
            pass
        

        plt.tight_layout(pad=0.2)
        plt.savefig(folder_address + '\\' + gc_axis_label + '_sweep' + output_date_stamp + '.png', bbox_inches='tight')
        plt.show()


    if 'mzi' in heater: # to plot the corresponding mzi complementary photodiode
        fig, ax = plt.subplots(figsize=(20,6))
        fig.suptitle(data_title, y=1.03, fontsize = fontsize)
        gc_color_map = []
        for i in range(0, len(gc), gc_step_plot_factor):

            RGB = ( 0, 1 - i/len(gc), 0 )
            line_kwargs['color'] = RGB
            line_kwargs['label'] = '%.1f' %gc[i]

            ax=plt.subplot(141)
            ax.plot(temp, [mpd1[i] for mpd1 in mpd1_temp], **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
            ax.legend(title = 'PD1',loc='best', fontsize = fontsize)
            ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(mpd_axis_label, fontsize = fontsize)
                
            ax=plt.subplot(142)
            ax.plot(temp, [mpd2[i] for mpd2 in mpd2_temp],  **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
            ax.legend(title = 'PD2',loc='best', fontsize = fontsize)
            ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(mpd_axis_label, fontsize = fontsize)
            
            ax=plt.subplot(143)
            ax.plot(temp, [mpd3[i] for mpd3 in mpd3_temp],  **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
            ax.legend(title = 'PD3',loc='best', fontsize = fontsize)
            ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(mpd_axis_label, fontsize = fontsize)
            
            ax=plt.subplot(144)
            ax.plot(temp, [mpd4[i] for mpd4 in mpd4_temp],  **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
            ax.legend(title = 'PD4',loc='best', fontsize = fontsize)
            ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(mpd_axis_label, fontsize = fontsize)
            

            # Setting up a colormap that's same as the line colors
            if len(gc) == 1:
                gc_color_map= [( 0, RGB ), ( 1, RGB )]
            else:
                gc_color_map.append( ( (gc[i] - min(gc))/(max(gc) - min(gc)), RGB )) 


        try :
            if gc[1] < gc[0]: # if gain chip current decreases during the scan
                gc_color_map = gc_color_map[::-1] # invert the order of gc color map
            my_cmap = mpl.colors.LinearSegmentedColormap.from_list( 'mycolors', gc_color_map )
            sm = plt.cm.ScalarMappable(cmap = my_cmap, norm = plt.Normalize( vmin = gc[0] + 0.01, vmax = gc[i] -0.01 ) )
            # fake up the array of the scalar mappable.
            sm._A = []
            plt.colorbar(sm, label=gc_axis_label)
        except:
            pass
        
        plt.tight_layout(pad=0.2)
        
        plt.savefig(folder_address + '\\' + temp_axis_label + '_compl_PD_sweep' + output_date_stamp + '.png', bbox_inches='tight')
        plt.show()


    if len(temp) > 1: # If this is not a 1D GC scan, we can plot as a function of T

        figsize = (20,12)
        fig, ax = plt.subplots(figsize=figsize)
        fig.suptitle(data_title, y=1.03, fontsize = fontsize)
        gc_color_map = []
        for i in range(0, len(gc), gc_step_plot_factor):


            RGB = ( 0, 1 - i/len(gc), 0 )
            line_kwargs['color'] = RGB
            line_kwargs['label'] = '%.1f' %gc[i]

            ax=plt.subplot(231)
            ax.plot(temp, [mpd_gc[i] for mpd_gc in mpd_temp],  **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
            #ax.legend(title = gc_axis_label,loc='best', fontsize = fontsize/1.5)
            ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(mpd_axis_label, fontsize = fontsize)

            ax=plt.subplot(232)
            ax.plot(temp, [power_gc[i] for power_gc in power_temp], **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
            #ax.legend(title = gc_axis_label,loc='best', fontsize = fontsize/1.5)
            ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(power_axis_label, fontsize = fontsize)
            
            ax=plt.subplot(234)
            ax.plot(temp, [T_comb_actual_gc[i] for T_comb_actual_gc in T_comb_actual_temp], **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
            ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(comb_temp_axis_label, fontsize = fontsize)


            ax=plt.subplot(235)
            ax.plot(temp, [mod_i_oop_gc[i] for mod_i_oop_gc in mod_i_oop_temp] ,  **line_kwargs); ax.grid(**grid_kwargs); #ax.tick_params(**tick_params)

            RGB_mod_q = ( 0, 0.5, 1 - i/len(gc) )
            line_kwargs['color'] = RGB_mod_q # use different colors for mod_i and mod_q
            ax.plot(temp, [mod_q_oop_gc[i] for mod_q_oop_gc in mod_q_oop_temp] ,  **line_kwargs); ax.grid(**grid_kwargs); #ax.tick_params(**tick_params)
            #ax.set_xlim([46,54]) # set the x axis Temperature range on demand

            #ax.legend(title = gc_axis_label,loc='best', fontsize = fontsize/1.5)
            ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(mod_iq_oop_axis_label, fontsize = fontsize)

            ax=plt.subplot(236)
            line_kwargs['color'] = RGB
            ax.plot(temp, [mod_i_apd_gc[i] for mod_i_apd_gc in mod_i_apd_temp] ,  **line_kwargs); ax.grid(**grid_kwargs); #ax.tick_params(**tick_params)

            RGB_mod_q = ( 0, 0.5, 1 - i/len(gc) )
            line_kwargs['color'] = RGB_mod_q # use different colors for mod_i and mod_q
            ax.plot(temp, [mod_q_apd_gc[i] for mod_q_apd_gc in mod_q_apd_temp] ,  **line_kwargs); ax.grid(**grid_kwargs); #ax.tick_params(**tick_params)
            ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(mod_iq_apd_axis_label, fontsize = fontsize)

            ax=plt.subplot(233)
            line_kwargs['color'] = RGB
            ax.plot(temp, [frequency_gc[i] for frequency_gc in frequency_temp] ,  **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
            #ax.legend(title = gc_axis_label,loc='best', fontsize = fontsize/1.5)
            ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(frequency_axis_label, fontsize = fontsize)

            # Setting up a colormap that's same as the line colors
            if len(gc) == 1:
                gc_color_map= [( 0, RGB ), ( 1, RGB )]
            else:
                gc_color_map.append( ( (gc[i] - min(gc))/(max(gc) - min(gc)), RGB )) 

        try :
            if gc[1] < gc[0]: # if gain chip current decreases during the scan
                gc_color_map = gc_color_map[::-1] # invert the order of gc color map
            my_cmap = mpl.colors.LinearSegmentedColormap.from_list( 'mycolors', gc_color_map )
            sm = plt.cm.ScalarMappable(cmap = my_cmap, norm = plt.Normalize( vmin = gc[0] + 0.01, vmax = gc[i] -0.01 ) )
            # fake up the array of the scalar mappable.
            sm._A = []
            plt.colorbar(sm, label=gc_axis_label)
        except:
            pass

        plt.tight_layout(pad=0.2)
        plt.savefig(folder_address + '\\'+ temp_axis_label + '_sweep' + output_date_stamp + '.png', bbox_inches='tight')
        plt.show()

    
        # Plot the differentials of the data as a function of temperature
        figsize = (20,6)
        fig, ax = plt.subplots(figsize=figsize)
        fig.suptitle(data_title, y=1.03, fontsize = fontsize)
        gc_color_map = []
        for i in range(0, len(gc), gc_step_plot_factor):

            RGB = ( 0, 1 - i/len(gc), 0 )
            line_kwargs['color'] = RGB
            line_kwargs['label'] = '%.1f' %gc[i]

            ax=plt.subplot(131)
            ax.plot(temp[1:], np.diff([mpd_gc[i] for mpd_gc in mpd_temp]), **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
            #ax.legend(title = gc_axis_label,loc='best', fontsize = fontsize/1.5)
            ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(delta_mpd_axis_label, fontsize = fontsize)

            ax=plt.subplot(132)
            ax.plot(temp[1:], np.diff([power_gc[i] for power_gc in power_temp]), **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
            #ax.legend(title = gc_axis_label,loc='best', fontsize = fontsize/1.5)
            ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(delta_power_axis_label, fontsize = fontsize)
            ax.set_ylim([-0.5,0.5])

            ax=plt.subplot(133)
            ax.plot(temp[1:], np.diff([frequency_gc[i] for frequency_gc in frequency_temp]), **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
            #ax.legend(title = gc_axis_label,loc='best', fontsize = fontsize/1.5)
            ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(delta_frequency_axis_label, fontsize = fontsize)

            # Setting up a colormap that's same as the line colors
            if len(gc) == 1:
                gc_color_map= [( 0, RGB ), ( 1, RGB )]
            else:
                gc_color_map.append( ( (gc[i] - min(gc))/(max(gc) - min(gc)), RGB )) 

        try :
            if gc[1] < gc[0]: # if gain chip current decreases during the scan
                gc_color_map = gc_color_map[::-1] # invert the order of gc color map
        except:
            pass


        my_cmap = mpl.colors.LinearSegmentedColormap.from_list( 'mycolors', gc_color_map )

        sm = plt.cm.ScalarMappable(cmap = my_cmap, norm = plt.Normalize( vmin = gc[0], vmax = gc[i] ) )
        # fake up the array of the scalar mappable.
        sm._A = []

        plt.colorbar(sm, label=gc_axis_label)#, cax=cbaxes,  orientation = 'horizontal', ticklocation = 'top')
        plt.tight_layout(pad=0.2)
        plt.savefig(folder_address + '\\' + temp_axis_label + '_sweep_differential' + output_date_stamp + '.png', bbox_inches='tight')
        plt.show()
    
    if len(gc)>1 and len(temp) > 1 : # if a 2D scan
        # 2D colormaps as a function of Temperature and GC
        mod_i_oop_temp_vflipped = [sublist[::-1] for sublist in mod_i_oop_temp ] # flip around the vertical axis to plot gc in the increasing direction
        mod_i_apd_temp_vflipped = [sublist[::-1] for sublist in mod_i_apd_temp ] # flip around the vertical axis to plot gc in the increasing direction
        mpd_temp_vflipped = [sublist[::-1] for sublist in mpd_temp ] # flip around the vertical axis to plot gc in the increasing direction
        power_temp_vflipped = [sublist[::-1] for sublist in power_temp ] # flip around the vertical axis to plot gc in the increasing direction
        frequency_temp_vflipped = [sublist[::-1] for sublist in frequency_temp ] # flip around the vertical axis to plot gc in the increasing direction

        if temp[1] < temp[0]: # if temperature decreases during the scan
            # flip around the horizontal axis to plot temperature in the increasing direction
            mod_i_oop_temp_vhflipped = mod_i_oop_temp_vflipped[::-1] 
            mod_i_apd_temp_vhflipped = mod_i_apd_temp_vflipped[::-1] 
            mpd_temp_vhflipped = mpd_temp_vflipped[::-1] 
            power_temp_vhflipped = power_temp_vflipped[::-1]
            frequency_temp_vhflipped = frequency_temp_vflipped[::-1]
        else: # no need to flip
            mod_i_oop_temp_vhflipped = mod_i_oop_temp_vflipped
            mod_i_apd_temp_vhflipped = mod_i_apd_temp_vflipped
            mpd_temp_vhflipped = mpd_temp_vflipped
            power_temp_vhflipped = power_temp_vflipped
            frequency_temp_vhflipped = frequency_temp_vflipped

        colormap_kwargs = {}
        colormap_figures = []

        cmap_colors=["white", "k"]

        plt.figure(figsize = (10,8))
        plt.imshow(mod_i_oop_temp_vhflipped, extent=[min(gc), max(gc), min(temp), max(temp)], cmap=colors.ListedColormap(cmap_colors),
                  origin='lowest', aspect='auto'
                ,vmin=-2, vmax=2)
                #, origin='lowest', aspect='auto')
        plt.xlabel(gc_axis_label, fontsize=fontsize); plt.ylabel(temp_axis_label, fontsize=fontsize)
        plt.xticks(fontsize=fontsize); plt.yticks(fontsize=fontsize)
        plt.suptitle(mod_i_oop_axis_label, fontsize=fontsize)
        cbar = plt.colorbar()
        cbar.set_label(mod_i_oop_axis_label, size=fontsize/1.5); cbar.ax.tick_params(labelsize=fontsize/1.5)
        plt.savefig(folder_address + '\\' + mod_i_oop_axis_label + '' + output_date_stamp + '.png', bbox_inches='tight')
        colormap_figures.append(mod_i_oop_axis_label + '' + output_date_stamp + '.png')
        plt.show()
        
        plt.figure(figsize = (10,8))
        plt.imshow(mod_i_apd_temp_vhflipped, extent=[min(gc), max(gc), min(temp), max(temp)], cmap=colors.ListedColormap(cmap_colors),
                  origin='lowest', aspect='auto'
                ,vmin=-2, vmax=2)
                #, origin='lowest', aspect='auto')
        plt.xlabel(gc_axis_label, fontsize=fontsize); plt.ylabel(temp_axis_label, fontsize=fontsize)
        plt.xticks(fontsize=fontsize); plt.yticks(fontsize=fontsize)
        plt.suptitle(mod_i_apd_axis_label, fontsize=fontsize)
        cbar = plt.colorbar()
        cbar.set_label(mod_i_apd_axis_label, size=fontsize/1.5); cbar.ax.tick_params(labelsize=fontsize/1.5)
        plt.savefig(folder_address + '\\' + mod_i_apd_axis_label + '' + output_date_stamp + '.png', bbox_inches='tight')
        colormap_figures.append(mod_i_apd_axis_label + '' + output_date_stamp + '.png')
        plt.show()

        plt.figure(figsize = (10,8))
        plt.imshow(mpd_temp_vhflipped, extent=[min(gc), max(gc), min(temp), max(temp)], cmap='gray',
                  origin='lowest', aspect='auto')
                #vmin=0, vmax=0.35, origin='lowest', aspect='auto')
        plt.xlabel(gc_axis_label, fontsize=fontsize); plt.ylabel(temp_axis_label, fontsize=fontsize)
        plt.xticks(fontsize=fontsize); plt.yticks(fontsize=fontsize)
        plt.suptitle(mpd_axis_label, fontsize=fontsize)
        cbar = plt.colorbar()
        cbar.set_label(mpd_axis_label, size=fontsize/1.5); cbar.ax.tick_params(labelsize=fontsize/1.5)
        plt.savefig(folder_address + '\\' + mpd_axis_label + '' + output_date_stamp + '.png', bbox_inches='tight')
        colormap_figures.append(mpd_axis_label + '' + output_date_stamp + '.png')
        plt.show()


        plt.figure(figsize = (10,8))
        plt.imshow(power_temp_vhflipped, extent=[min(gc), max(gc), min(temp), max(temp)], cmap='gray',
                    origin='lowest', aspect='auto')
                #vmin=0, vmax=15, origin='lowest', aspect='auto')
        plt.xlabel(gc_axis_label, fontsize=fontsize); plt.ylabel(temp_axis_label , fontsize=fontsize)
        plt.xticks(fontsize=fontsize); plt.yticks(fontsize=fontsize)
        plt.suptitle(power_axis_label,fontsize=fontsize)
        cbar = plt.colorbar()
        cbar.set_label(power_axis_label, size=fontsize/1.5); cbar.ax.tick_params(labelsize=fontsize/1.5)

        plt.savefig(folder_address + '\\' + power_axis_label + '' + output_date_stamp + '.png', bbox_inches='tight')
        colormap_figures.append(power_axis_label + '' + output_date_stamp + '.png')
        plt.show()

        plt.figure(figsize = (10,8))
        plt.imshow(frequency_temp_vhflipped, extent=[min(gc), max(gc), min(temp), max(temp)], cmap='RdBu',
                    origin='lowest', aspect='auto')
                   #vmin=194.18, vmax=194.25, origin='lowest', aspect='auto')
        plt.xlabel(gc_axis_label, fontsize=fontsize); plt.ylabel(temp_axis_label , fontsize=fontsize)
        plt.xticks(fontsize=fontsize); plt.yticks(fontsize=fontsize)
        plt.suptitle(frequency_axis_label,fontsize=fontsize)
        cbar = plt.colorbar()
        cbar.set_label(frequency_axis_label, size=fontsize/1.5); cbar.ax.tick_params(labelsize=fontsize/1.5)

        plt.savefig(folder_address + '\\' + frequency_axis_label + '' + output_date_stamp + '.png', bbox_inches='tight')
        colormap_figures.append(frequency_axis_label + '' + output_date_stamp + '.png')
        plt.show()

    if file_name_powerpoint != '' :
        images_to_powerpoint(file_name_powerpoint, [temp_axis_label + '_sweep' + output_date_stamp + '.png'], slide_title = 'GC sweep', slide_label = data_title)
        images_to_powerpoint(file_name_powerpoint, [gc_axis_label + '_sweep' + output_date_stamp + '.png'], slide_title = 'Temperature sweep', slide_label = data_title)
        images_to_powerpoint(file_name_powerpoint, [gc_axis_label + '_sweep_differential' + output_date_stamp + '.png'], slide_title = 'Temperature sweep (Differential)', slide_label = data_title)

        images_to_powerpoint(file_name_powerpoint, colormap_figures, slide_title = '2D Plotting', slide_label = data_title)
    
    return ax

In [26]:
def choose_images_to_attach(len_temp_range, len_gc_range,  output_date_stamp='', heater= ''):
    
    attach_image_names = []
    
    # if temperature sweep has been performed
    if len_temp_range > 1 and 'mzi' in heater:
        attach_image_names.append('Heat (mW)_sweep.png')
    elif len_temp_range > 1:
        attach_image_names.extend(['Temperature (°C)_sweep.png', 'Temperature (°C)_sweep_differential.png', 'Temperature (°C)_compl_PD_sweep.png'])
    
    # if gain current sweep has been performed        
    if len_gc_range > 1:
        attach_image_names.extend(['Gain Current (mA)_sweep.png'])
        
    if len_temp_range > 1 and len_gc_range > 1 : # if a 2D scan
        attach_image_names.extend(['Mod i oop.png', 'Mod i apd.png', 'Optical Power (mW).png', 'Photocurrent (mA).png', 'Frequency (THz).png'])
        
    # Correct image name = Standard image name + date stamp
    attach_image_names = [attach_image_name.replace('.png', output_date_stamp + '.png')  for attach_image_name in attach_image_names]
    
    return attach_image_names
    

In [27]:
# this function is out of date, it may be better to re-create it from the plot_pit_sweeps function
# to plot 2 identical datasets with opposite temperature scan directions
def plot_multiple_pit_sweeps(temp, gc, mpd_temp, power_temp, frequency_temp, 
                    temp_step_plot_factor=1, gc_step_plot_factor=1, # to skip some of the temperature and gc values to be plotted
                    file_name_powerpoint = '', # export the figures to slides if file_name_powerpoint is specified
                    data_title = '', 
                    folder_address = 'Outputs'
                   ):

    gc_axis_label = 'Gain Current (mA)'; mpd_axis_label = 'Photocurrent (mA)'; temp_axis_label = 'Temperature (°C)'
    power_axis_label = 'Optical Power (mW)'; frequency_axis_label = 'Frequency (THz)'
    figsize = (20,8); fontsize = 20
    fig, ax = plt.subplots(figsize=figsize)
    plt.rcParams['axes.linewidth'] = 3 #set the value globally
    
    line_kwargs = {'ls' :'-', 'lw' : 1, 'ms' : 1, 'marker' : '', 'markeredgewidth' : 2, 'markerfacecolor' : 'None'}
    grid_kwargs = {'color':'gray', 'ls': '--', 'lw': 1, 'axis' : 'both'}
    tick_params = {'direction' : 'in', 'labelsize' : fontsize/1.5}
    temp_color_map = []
    
    temp2 = temp[1]
    temp = temp[0]
    
    mpd_temp2 = mpd_temp[1]
    mpd_temp = mpd_temp[0]
    
    power_temp2 = power_temp[1]
    power_temp = power_temp[0]

    frequency_temp2 = frequency_temp[1]
    frequency_temp = frequency_temp[0]
    

    for i in range(0, len(temp), temp_step_plot_factor):

        temp_plot_range =  temp[0:len(temp):temp_step_plot_factor] # the temperature values being plotted
        
        if temp[0] < temp[1]: # if temperature decreases in the scan
            RGB = ( i/len(temp), 0, 1 - i/len(temp) ) # Lowest T = blue
        else :                                                         
            RGB = ( 1 - i/len(temp), 0, i/len(temp) ) # Lowest T = blue
        
        # Setting up a colormap that's same as the line colors
        temp_color_map_i = (temp[i] - min(temp_plot_range))/(max(temp_plot_range) - min(temp_plot_range))
        temp_color_map.append(  (temp_color_map_i, RGB ) ) 
        
        line_kwargs['color'] = RGB
        line_kwargs['label'] = '%.1f' %temp[i]
        
        ax=plt.subplot(131)
        ax.plot(gc, mpd_temp[i], **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
        #ax.legend(title = temp_axis_label, loc='best', fontsize = fontsize/1.5)
        ax.set_xlabel(gc_axis_label, fontsize = fontsize); ax.set_ylabel(mpd_axis_label, fontsize = fontsize)
    
        ax=plt.subplot(132)
        ax.plot(gc, power_temp[i],  **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
        #ax.legend(title = temp_axis_label, loc='best', fontsize = fontsize/1.5)
        ax.set_xlabel(gc_axis_label, fontsize = fontsize); ax.set_ylabel(power_axis_label, fontsize = fontsize)
        
        ax=plt.subplot(133)
        ax.plot(gc, frequency_temp[i],  **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
        #ax.legend(title = temp_axis_label, loc='best', fontsize = fontsize/1.5)
        ax.set_xlabel(gc_axis_label, fontsize = fontsize); ax.set_ylabel(frequency_axis_label, fontsize = fontsize)
    
    if temp[1] < temp[0]: # if temperature is decreasing during the scan
        temp_color_map = temp_color_map[::-1] # invert the order of temperature color map
        
    my_cmap = mpl.colors.LinearSegmentedColormap.from_list( 'mycolors', temp_color_map )
    sm = plt.cm.ScalarMappable(cmap = my_cmap, norm = plt.Normalize( vmin = temp[0], vmax = temp[i] ) )
    # fake up the array of the scalar mappable.
    sm._A = []
    
    plt.colorbar(sm, label=temp_axis_label)
    plt.savefig(folder_address + '\\' + temp_axis_label + '_sweep.png', bbox_inches='tight')
    plt.subplots_adjust(wspace=0.30)
    plt.show()
    
    fig, ax = plt.subplots(figsize=figsize)
    gc_color_map = []
    for i in range(0, len(gc), gc_step_plot_factor):
        
        RGB = ( 0, 1 - i/len(gc), 0 )
        RGB2 = ( 0, 0, 1 - i/len(gc) )
        line_kwargs['color'] = RGB
        line_kwargs['label'] = '%.1f' %gc[i]
        
        line_kwargs2 = line_kwargs.copy()
        line_kwargs2['color'] = RGB2

        ax=plt.subplot(131)
        ax.plot(temp, [mpd_gc[i] for mpd_gc in mpd_temp], **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
        ax.plot(temp2, [mpd_gc2[i] for mpd_gc2 in mpd_temp2], **line_kwargs2);
        #ax.legend(title = gc_axis_label,loc='best', fontsize = fontsize/1.5)
        ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(mpd_axis_label, fontsize = fontsize)
        
        ax=plt.subplot(132)
        ax.plot(temp, [power_gc[i] for power_gc in power_temp], **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
        ax.plot(temp2, [power_gc2[i] for power_gc2 in power_temp2], **line_kwargs2)
        #ax.legend(title = gc_axis_label,loc='best', fontsize = fontsize/1.5)
        ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(power_axis_label, fontsize = fontsize)
        #ax.set_xlim([22,24])
        
        ax=plt.subplot(133)
        ax.plot(temp, [frequency_gc[i] for frequency_gc in frequency_temp], **line_kwargs); ax.grid(**grid_kwargs); ax.tick_params(**tick_params)
        ax.plot(temp2, [frequency_gc2[i] for frequency_gc2 in frequency_temp2], **line_kwargs2);
        #ax.legend(title = gc_axis_label,loc='best', fontsize = fontsize/1.5)
        ax.set_xlabel(temp_axis_label, fontsize = fontsize); ax.set_ylabel(frequency_axis_label, fontsize = fontsize)
        
        # Setting up a colormap that's same as the line colors
        gc_color_map.append( ( (gc[i] - min(gc))/(max(gc) - min(gc)), RGB )) 
      
    try :
        if gc[1] < gc[0]: # if gain chip current decreases during the scan
            gc_color_map = gc_color_map[::-1] # invert the order of gc color map
    except:
        pass
    
    my_cmap = mpl.colors.LinearSegmentedColormap.from_list( 'mycolors', gc_color_map )

    sm = plt.cm.ScalarMappable(cmap = my_cmap, norm = plt.Normalize( vmin = gc[0], vmax = gc[i] ) )
    # fake up the array of the scalar mappable.
    sm._A = []
    
    plt.colorbar(sm, label=gc_axis_label)#, cax=cbaxes,  orientation = 'horizontal', ticklocation = 'top')
    plt.savefig(folder_address + '\\' + gc_axis_label + '_sweep.png', bbox_inches='tight')
    plt.subplots_adjust(wspace=0.30)
    plt.show()
    
    # 2D colormaps as a function of Temperature and GC
    mpd_temp_vflipped = [sublist[::-1] for sublist in mpd_temp ] # flip around the vertical axis to plot gc in the increasing direction
    power_temp_vflipped = [sublist[::-1] for sublist in power_temp ] # flip around the vertical axis to plot gc in the increasing direction
    frequency_temp_vflipped = [sublist[::-1] for sublist in frequency_temp ] # flip around the vertical axis to plot gc in the increasing direction
     
    if temp[1] < temp[0]: # if temperature decreases during the scan
        # flip around the horizontal axis to plot temperature in the increasing direction
        mpd_temp_vhflipped = mpd_temp_vflipped[::-1] 
        power_temp_vhflipped = power_temp_vflipped[::-1]
        frequency_temp_vhflipped = frequency_temp_vflipped[::-1]
    else:
        mpd_temp_vhflipped = mpd_temp_vflipped
        power_temp_vhflipped = power_temp_vflipped
        frequency_temp_vhflipped = frequency_temp_vflipped

    colormap_kwargs = {}
    colormap_figures = []
    
    plt.figure(figsize = (10,8))
    plt.imshow(mpd_temp_vhflipped, extent=[min(gc), max(gc), min(temp), max(temp)], cmap='gray',
              origin='lowest', aspect='auto')
            #vmin=0, vmax=0.35, origin='lowest', aspect='auto')
    plt.xlabel(gc_axis_label, fontsize=fontsize); plt.ylabel(temp_axis_label, fontsize=fontsize)
    plt.xticks(fontsize=fontsize); plt.yticks(fontsize=fontsize)
    plt.suptitle(mpd_axis_label, fontsize=fontsize)
    plt.colorbar(label= mpd_axis_label)
    plt.savefig(folder_address + '\\' + mpd_axis_label + '.png', bbox_inches='tight')
    colormap_figures.append(mpd_axis_label + '.png')
    plt.show()
    
    
    plt.figure(figsize = (10,8))
    plt.imshow(power_temp_vhflipped, extent=[min(gc), max(gc), min(temp), max(temp)], cmap='gray',
                origin='lowest', aspect='auto')
            #vmin=0, vmax=15, origin='lowest', aspect='auto')
    plt.xlabel(gc_axis_label, fontsize=fontsize); plt.ylabel(temp_axis_label , fontsize=fontsize)
    plt.xticks(fontsize=fontsize); plt.yticks(fontsize=fontsize)
    plt.suptitle(power_axis_label,fontsize=fontsize)
    plt.colorbar(label= power_axis_label)
    plt.savefig(folder_address + '\\' + power_axis_label + '.png', bbox_inches='tight')
    colormap_figures.append(power_axis_label + '.png')
    plt.show()
    
    plt.figure(figsize = (10,8))
    plt.imshow(frequency_temp_vhflipped, extent=[min(gc), max(gc), min(temp), max(temp)], cmap='RdBu',
                origin='lowest', aspect='auto')
               #vmin=194.18, vmax=194.25, origin='lowest', aspect='auto')
    plt.xlabel(gc_axis_label, fontsize=fontsize); plt.ylabel(temp_axis_label , fontsize=fontsize)
    plt.xticks(fontsize=fontsize); plt.yticks(fontsize=fontsize)
    plt.suptitle(frequency_axis_label,fontsize=fontsize)
    plt.colorbar(label= frequency_axis_label)
    plt.savefig(folder_address + '\\' + frequency_axis_label + '.png', bbox_inches='tight')
    colormap_figures.append(frequency_axis_label + '.png')
    plt.show()
    
    if file_name_powerpoint != '' :
        images_to_powerpoint(file_name_powerpoint, [temp_axis_label + '_sweep.png'], slide_title = 'Temperature sweep', slide_label = data_title)
        images_to_powerpoint(file_name_powerpoint, [gc_axis_label + '_sweep.png'], slide_title = 'GC sweep', slide_label = data_title)
        #images_to_powerpoint(file_name_powerpoint, colormap_figures, slide_title = 'GC sweep')

        images_to_powerpoint(file_name_powerpoint, colormap_figures, slide_title = '2D Plotting', slide_label = data_title)
    
    return ax

# FSCS angle tuning analysis

In [28]:
def analyze_crystal_tuning(
     file_name='', file_labels = [] , tuning='angle' # or 'temperature'
):
    
    temperatures = {}
    angles = {}; angle_values = []
    peak_rightmost = []; peak_leftmost = []
    dip_rightmost = [] ;  dip_leftmost = []


    data_title = file_name
    data_pd, data, data_labels = import_data(folder_address='Inputs', file_name= file_name, file_labels = file_labels, header_rows=1, concatenate=0)
    x_axis_label = {}; y_axis_label = {}; x={} ; y = {}; peaks={}; dips={} 

    figsize = (16,12); fontsize = 24
    fig, ax = plt.subplots(figsize=figsize)
    plt.rcParams['axes.linewidth'] = 3 #set the value globally
    line_kwargs = {'ls' :'--', 'lw' : 2, 'ms' : 12}

    #for file_label in ['stage_4-minus20_degrees_2019_02_14']:
    for file_label in file_labels:

        # auto-detect the angles and temperatures from the file_label
        
        if  re.findall('(\d+)C', file_label) :# works on integer temperatures only
            temperature = re.findall('(?:(\w+.\d+)C|(\w+)C)' , file_label )[0][0] + re.findall('(?:-(\d+.\d+)C|-(\d+)C)' , file_label )[0][1]
            
            if 'minus' in temperature:
                temperature = temperature.replace('minus','-')
            temperature = float(temperature)
            temperatures[file_label] = temperature

        
        #angle = re.findall('-(.*?)_degrees', file_label)[0]
        angle = re.findall('-(\w+)_degrees', file_label)[0]
      
        if 'minus' in angle:
            angle = angle.replace('minus','-')
            
        angle = float(angle)
        angles[file_label] = angle
        angle_values.append(angle)

        x_axis_label[file_label]=data_labels[file_label][0]
        y_axis_label[file_label]=data_labels[file_label][1]
        x[file_label] = data_pd[file_label] [ x_axis_label[file_label] ].values
        y[file_label] = data_pd[file_label] [ y_axis_label[file_label] ].values    

        peaks[file_label], _ = find_peaks( y[file_label], distance=5000, prominence= 0.15)
        dips[file_label] , _ = find_peaks(-y[file_label], distance=5000, prominence= 0.15)

        ax.plot(x[file_label][peaks[file_label]], y[file_label][peaks[file_label]], 'x',label='', markeredgewidth=5, markersize=12)
        ax.plot(x[file_label][dips [file_label]], y[file_label][dips [file_label]], 'x',label='', markeredgewidth=5, markersize=12)


        for i, txt in enumerate( x[file_label][peaks[file_label]]):
            ax.annotate('%.2f' %txt, (x[file_label][peaks[file_label]][i], y[file_label][peaks[file_label]][i]), xytext = (-20, -45), textcoords='offset pixels', arrowprops={'arrowstyle': '->'} )
        for i, txt in enumerate( x[file_label][dips [file_label]]):
            ax.annotate('%.2f' %txt, (x[file_label][dips [file_label]][i], y[file_label][dips [file_label]][i]), xytext = (-20, -45), textcoords='offset pixels', arrowprops={'arrowstyle': '->'} )
    
        ax.plot(x[file_label],y[file_label], label=angles[file_label], **line_kwargs)
        if x[file_label][peaks[file_label]][-1] < 1564.5:
            peak_rightmost.append(x[file_label][peaks[file_label]][-1])
        else : 
            peak_rightmost.append(x[file_label][peaks[file_label]][-2])

        peak_leftmost.append(x[file_label][peaks[file_label]][0])
        dip_leftmost.append( x[file_label][ dips[file_label]][0])
        dip_rightmost.append( x[file_label][ dips[file_label]][0])


    ax.set_xlabel(x_axis_label[file_label], fontsize = fontsize)
    ax.set_ylabel(y_axis_label[file_label], fontsize = fontsize)    
    grid_kwargs = {'color':'gray', 'ls': '--', 'lw': 1}
    ax.grid(b=None, which='both', axis='both', **grid_kwargs)
    ax.tick_params(direction='in', labelsize=fontsize)
    ax.legend(loc='best', fontsize = fontsize, title='Angle (°)')

    plt.show()
    
    figsize = (12,6); fontsize = 24
    fig = plt.figure(1, figsize = figsize)

    ax=plt.subplot(221)
    line_kwargs['marker'] = 'o'; line_kwargs['markerfacecolor'] = 'None'; line_kwargs['markeredgewidth'] = 2
    ax.plot(angle_values, peak_leftmost, label=file_label, **line_kwargs)
    grid_kwargs = {'color':'gray', 'ls': '--', 'lw': 1}
    ax.grid(b=None, which='both', axis='both', **grid_kwargs)
    ax.tick_params(direction='in', labelsize=fontsize/1.5)
    ax.set_title('Leftmost Peak' , fontsize = fontsize); ax.set_xlabel("Angle (°)", fontsize = fontsize) ; ax.set_ylabel("λ (nm)", fontsize = fontsize)    

    ax=plt.subplot(222)
    ax.plot(angle_values, peak_rightmost, label=file_label, **line_kwargs, color='r')
    grid_kwargs = {'color':'gray', 'ls': '--', 'lw': 1}
    ax.grid(b=None, which='both', axis='both', **grid_kwargs)
    ax.tick_params(direction='in', labelsize=fontsize/1.5)
    ax.set_title('Rightmost Peak' , fontsize = fontsize); ax.set_xlabel("Angle (°)", fontsize = fontsize) ; ax.set_ylabel("λ (nm)", fontsize = fontsize)    

    ax=plt.subplot(223)
    line_kwargs['marker'] = 'o'; line_kwargs['markerfacecolor'] = 'None'; line_kwargs['markeredgewidth'] = 2
    ax.plot(angle_values, dip_leftmost, label=file_label, **line_kwargs)
    grid_kwargs = {'color':'gray', 'ls': '--', 'lw': 1}
    ax.grid(b=None, which='both', axis='both', **grid_kwargs)
    ax.tick_params(direction='in', labelsize=fontsize/1.5)
    ax.set_title('Leftmost Dip' , fontsize = fontsize); ax.set_xlabel("Angle (°)", fontsize = fontsize) ; ax.set_ylabel("λ (nm)", fontsize = fontsize)    

    ax=plt.subplot(224)
    ax.plot(angle_values, dip_rightmost, label=file_label, **line_kwargs, color='r')
    grid_kwargs = {'color':'gray', 'ls': '--', 'lw': 1}
    ax.grid(b=None, which='both', axis='both', **grid_kwargs)
    ax.tick_params(direction='in', labelsize=fontsize/1.5)
    ax.set_title('Rightmost Dip' , fontsize = fontsize); ax.set_xlabel("Angle (°)", fontsize = fontsize) ; ax.set_ylabel("λ (nm)", fontsize = fontsize)    

    
    plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.55, wspace=0.45)
    plt.show()
    
    return peaks, angles, temperatures

In [29]:
# Set the Frequency of the Silicon Photonics laser via MZI filter heater values and Comb Filter Temperature
# Provide the Calibration Excel file with proper foramattting
def set_frequency_via_mzi_comb(frequency, bench_temp, calibration_file_name, comb_temp_file_name = '' ):

    # Give the location of the calibration file 
    calibration_file_address = calibration_folder_address + calibration_file_name + '.xlsx'

    # Open Workbook 
    calibration_wb = openpyxl.load_workbook(calibration_file_address)
    calibration_sheet  = calibration_wb['Calibration']
    
    calibration_sheet['F4'].value = frequency
    
    # Make use of Microsoft Excel in order to update the values with the new set frequency. Otherwise the values are not recalculated
    saved_calibration_file_address = calibration_folder_address + calibration_file_name + '_saved.xlsx'
    
    try: # If the excel file is left open, the following will not work
        calibration_wb.save(saved_calibration_file_address)
    except:
        simple_email(email_subject='Experiment Interrupted', email_body='%s can not be saved' %saved_calibration_file_address)
    try: # Office Aplication sometimes does not work
        office = win32com.client.Dispatch("Excel.Application")
    except: # Re-try running the Excel
        try:
            print('Excel did not run in 1st trial, retrying')
            office = win32com.client.Dispatch("Excel.Application")
        except: # Re-try running the Excel
            try:
                print('Excel did not run in 2nd trial, retrying')
                office = win32com.client.Dispatch("Excel.Application")
            except: # After 3 trials, send and email and reaise error
                simple_email(email_subject='Experiment Interrupted', email_body='Error in win32com when running Excel')
                raise ValueError('Error in win32com when running Excel, tried 3 times but did not work')
        
    #calibration_wb = office.Workbooks.Open('Inputs\\' + calibration_file_name + '_saved.xlsx')
    calibration_wb = office.Workbooks.Open(saved_calibration_file_address)
    calibration_wb.RefreshAll(); calibration_wb.Save(); calibration_wb.Close() 
    
    # Open Workbook 
    print('Calibration file to use: ' + saved_calibration_file_address)
    calibration_wb = openpyxl.load_workbook(saved_calibration_file_address, data_only=True)
    calibration_sheet  = calibration_wb['Calibration']
    
    # Read the bench temperature at calibration point and see if it is the same as the target bench temperature
    
    bench_temp_calibration = calibration_sheet['C5'].value
    if bench_temp_calibration != float(bench_temp):
        raise ValueError('Bench target and calibration temperatures are different: %f vs %s' %(bench_temp_calibration,bench_temp))
    
    # Read the range of Target values
    target_column = 'J'; target_row = 8
    target_cells = calibration_sheet[target_column + str(target_row) + ':' + target_column + str(target_row + 6) ]
    target = target_cells[0][0].value
    
    # Make sure that we are using the right set of values
    if target.lower() != 'target':
        print(target)
        raise ValueError('%s%d is not the right cell for target values of MZIs and Comb' %(target_column, target_row))
    
    # Assign the target MZIs and Comb temperature
    [mzi1_heat, mzi2_heat, mzi3_heat, mzi4_heat, comb_temperature] = [target_cells[i][0].value  for i in [2,3,4,5,6] ]
    
    set_frequency = calibration_sheet['F4'].value
    # Make sure that the frequency set is reasonable
    if set_frequency < min_frequency or set_frequency > max_frequency:
        raise ValueError('Frequency of %.3f THz is not valid' %set_frequency)
    print('Frequency Set: %.3f THz' %(set_frequency))        
            
    if comb_temp_file_name != '':
        try:
            # Give the location of the comb temperature file 
            comb_temp_file_address = 'Inputs/Calibration/' + comb_temp_file_name + '.xlsx' # make sure that it is not .csv
            
            # Open Workbook 
            comb_temp_wb = openpyxl.load_workbook(comb_temp_file_address, data_only=True)
            comb_temp_sheet  = comb_temp_wb.active #[comb_temp_file_name]
            comb_temp_pd = pd.DataFrame(comb_temp_sheet.values)

            comb_temp_column = 1 # note that the 2nd column of the table is used here (not the 3rd)
            comb_temp_index = comb_temp_pd.index[ round(comb_temp_pd[0],3) == round(set_frequency, 3)].values.astype(int)[0]
            print('%s on row #%s of %s.xlsx' %(set_frequency, str(comb_temp_index), comb_temp_file_name))
            comb_temperature = comb_temp_pd[comb_temp_column][comb_temp_index]
        except:
            print('%.3f is not in %s or in the right column' %(set_frequency,comb_temp_file_name))
            print('Calibration sheet will be used instead')

    # Make sure that the comb temperature is reasonable
    if comb_temperature < min_comb_temperature or comb_temperature > max_comb_temperature:
        raise ValueError('Comb Temperature of %.3f °C is not valid' %comb_temperature)
        
    
    print('MZI Set [1, 2, 3, 4]: [%.2f, %.2f, %.2f, %.2f]' %(mzi1_heat,mzi2_heat,mzi3_heat,mzi4_heat))
    print('Comb Temperature Needed: %.3f °C' %comb_temperature)
    
    # Set the MZI heat values and Comb temperature
    board_io('mzi 1 enable on'); board_io('mzi 2 enable on'); board_io('mzi 3 enable on'); board_io('mzi 4 enable on');     
    board_io('tps pp mzi1 %s' %mzi1_heat); board_io('tps pp mzi2 %s' %mzi2_heat); board_io('tps pp mzi3 %s' %mzi3_heat); board_io('tps pp mzi4 %s' %mzi4_heat)
    
    board_io('comb enable on'); heater_temp('comb', comb_temperature)
    
    return mzi1_heat, mzi2_heat , mzi3_heat, mzi4_heat, comb_temperature

def calibrate_frequency_comb_mzis(bench_temp = 25, comb_temp = 60, mzi_heats = [0,0,0,0],
                                    bench_mode = 'oop' , gc = 400,
                                    mzi_wait_time = 2, max_power_wait_time = 2):
    
    optimum_mzi_heats = []

    enable_heater('on', 'bench2'); enable_heater('on', 'comb') # make sure bench2, comb are enabled
    set_bench_mode('temp')
    heater_temp('bench2', bench_temp); heater_temp('comb', comb_temp)
    time.sleep(30)
    
    gain_current(gc); print('Gain Current is %.3f mA' %gc)
    time.sleep(1)
    set_bench_mode(bench_mode)
    time.sleep(30)

    for n in [1,2,3,4]: 
        heater_heat('mzi %s' %n, mzi_heats[n-1])
    
    for n in [4,3,2,1]:
        is_power_increase = 1
        mzi_scan_direction = +1
        while is_power_increase or mzi_scan_direction == -1:
            n_heat_current = heater_heat('mzi %s' %n)
            power_up = 0                              
            power_center = read_max_power(max_power_wait_time, 'mW')
            print('Power: %.3f at MZI %s=%.2f mW' %(power_center, n, n_heat_current))

            for n_heat in [n_heat_current + 0.5*mzi_scan_direction]: 
                heater_heat('mzi %s' %n, n_heat)
                time.sleep(mzi_wait_time)
                power_up = read_max_power(max_power_wait_time, 'mW')
                print('Power: %.3f at MZI %s=%.2f mW' %(power_up, n, n_heat))

            if power_up > power_center*1.001:
                is_power_increase = 1
                print('Changing mzi %s increases the power by more than .1%%' %n)
            else :
                is_power_increase = 0; mzi_scan_direction *= -1
                print('Changing mzi %s does not increase the power by more than .1%%, reverting scan direction' %n)
        optimum_mzi_heats.append(n_heat)
    
    return optimum_mzi_heats

# 1D search for maximum power via mzi filters
def maximize_power_via_mzi(n, mzi_heat_step= 0.5, mzi_wait_time = 1):

    print(n)

    # Enable MZIs
    mzi_heat = heater_heat('mzi %s' %n)
    mzi_range = np.arange(mzi_heat - 2*mzi_heat_step, mzi_heat + 3*mzi_heat_step, mzi_heat_step)
    power_range = []
    
    for mzi_heat in mzi_range:
        heater_heat('mzi %s' %n, mzi_heat)
        time.sleep(mzi_wait_time)
        try:
            power_read = power_meter.read * 1000 # converted to mW
            if power_read > 1000 : # if the PM100D is saturated
                power_read = 0
        except:
            print('Cannot read power from powermeter')
            return

        power.append(power_read)
        print('mzi %s: %.3f %.3f mW' %(n, mzi_heat, power_read))
    
    
    
    max_power = max(power_range)
    max_index = power_range.index(max_power)
    
    heater_heat('mzi %s' %n, mzi_range[max_index])
    
    return mzi_range[max_index]
    
    power_read = read_power()
    next_power_read = power_read
    
    print('mzi %s: %.3f %.3f mW' %(n, mzi_heat, next_power_read))
    
    while power_read <= next_power_read :
        
        mzi_heat += mzi_heat_step
        mzi(mzi_heat, n)
        time.sleep(mzi_wait_time)
        
        power_read = next_power_read
        next_power_read = power_meter.read * 1000 # converted to mW
        print('mzi %s: %.3f %.3f mW' %(n, mzi_heat, next_power_read))

    mzi_heat -= mzi_heat_step 
    mzi(mzi_heat, n)        
    time.sleep(mzi_wait_time)

    power_read = power_meter.read * 1000 # converted to mW
    next_power_read = power_read
    print('mzi %s: %.3f %.3f mW' %(n, mzi_heat, next_power_read))
    
    next_power_read = power_read
    while power_read <= next_power_read :
        
        mzi_heat -= mzi_heat_step
        mzi(mzi_heat, n)
        
        time.sleep(mzi_wait_time)
        
        power_read = next_power_read
        next_power_read = power_meter.read * 1000 # converted to mW
        print('mzi %s: %.3f %.3f mW' %(n, mzi_heat, next_power_read))
        
    mzi_heat += mzi_heat_step
    mzi(mzi_heat, n)

    return mzi_heat

def read_modulation():
    if board_io( 'mod info')[-1] != 'modulation disabled':

        mod_iq_oop_read = board_io( 'mod iq oop')[-1]
        mod_i_oop_read = float(re.findall('[0-9-.]+', mod_iq_oop_read)[0]); mod_q_oop_read = float(re.findall('[0-9-.]+', mod_iq_oop_read)[-1])

        mod_iq_apd_read = board_io( 'mod iq apd')[-1]
        mod_i_apd_read = float(re.findall('[0-9-.]+', mod_iq_apd_read)[0]); mod_q_apd_read = float(re.findall('[0-9-.]+', mod_iq_apd_read)[-1])

        mod_iq_vgain_read = board_io( 'mod iq vgain')[-1]
        mod_i_vgain_read = float(re.findall('[0-9-.]+', mod_iq_vgain_read)[0]); mod_q_vgain_read = float(re.findall('[0-9-.]+', mod_iq_vgain_read)[-1])

        return mod_i_oop_read, mod_q_oop_read, mod_i_apd_read, mod_q_apd_read, mod_i_vgain_read, mod_q_vgain_read
    
    else:
        return [0]*6
    

def read_mpds():
    
    mpd_oop_read = board_io( 'adc oop')[1]
    mpd_oop_read = float (re.findall('[0-9-.]+', mpd_oop_read)[-1]) # extract the mpd_oop value from the returned string
    
    mpd_orp_read = board_io( 'adc orp')[1]
    mpd_orp_read = float (re.findall('[0-9-.]+', mpd_orp_read)[-1]) # extract the mpd_oop value from the returned string
    
    mpd_ap_read = board_io( 'adc ap')[1]
    mpd_ap_read = float (re.findall('[0-9-.]+', mpd_ap_read)[-1]) # extract the mpd_ap value from the returned string
    
    mpd_bp_read = board_io( 'adc bp')[1]
    mpd_bp_read = float (re.findall('[0-9-.]+', mpd_bp_read)[-1]) # extract the mpd_bp value from the returned string
    
    mpd1_read = board_io( 'adc pd1')[1]
    mpd1_read = float (re.findall('[0-9-.]+', mpd1_read)[-1]) # extract the mpd1 value from the returned string
    
    mpd2_read = board_io( 'adc pd2')[1]
    mpd2_read = float (re.findall('[0-9-.]+', mpd2_read)[-1]) # extract the mpd2 value from the returned string
    
    mpd3_read = board_io( 'adc pd3')[1]
    mpd3_read = float (re.findall('[0-9-.]+', mpd3_read)[-1]) # extract the mpd3 value from the returned string
    
    mpd4_read = board_io( 'adc pd4')[1]
    mpd4_read = float (re.findall('[0-9-.]+', mpd4_read)[-1]) # extract the mpd4 value from the returned string
    
    return mpd_oop_read, mpd_orp_read, mpd_ap_read, mpd_bp_read, mpd1_read, mpd2_read, mpd3_read, mpd4_read

# Interactive Functions

In [30]:
%%javascript 
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "current_notebook_name = " + "'"+thename+"'"; 
kernel.execute(command);

<IPython.core.display.Javascript object>

In [2]:
def get_pid(heater ='bench2'):

    return [ float(pid) for pid in re.findall('[0-9-.]+' , board_io('%s pid' %heater)[-1]) ] 

def gain_current(gc_write=-1):
    
    if gc_write == -1: # user wants to read the gain_current
        gc_read = board_io( 'gc')[-1]
        return float(re.findall('[0-9-.]+' , gc_read)[0]) # extract the current from the returned string
    
    else: 
        gc_read = board_io( 'gc %s' %gc_write)[-1]
        return float(re.findall('[0-9-.]+' , gc_read)[0]) # extract the current from the returned string

def mzi(mzi_write, n=0):
    
    if n not in [1,2,3,4]:
        raise ValueError('mzi%s is not a valid mzi' %n)
        
    mzi_read = heater_heat( 'mzi %d' %n, mzi_write)
    
    return mzi_read
    
    
try:
    # Slider for Gain Current 
    # Initialize with the Current value to not change it
    gc_current = gain_current()
    gain_current_slider = interactive( gain_current, gc_write=widgets.FloatSlider(min=0, max=max_gc_current, step=1,value=gc_current, description='Gain Current (mA)' , style=style) )

except:
    pass

try:
    
    max_power = 52 
    # Sliders for MZI1
    
    #Enable MZIs
    board_io('mzi 1 enable on'); board_io('mzi 2 enable on'); board_io('mzi 3 enable on'); board_io('mzi 4 enable on');     
    
    # Initialize with the Current value to not change it
    
    mzi1 = heater_heat('mzi 1')
    mzi1_slider = interactive( mzi, n=fixed(1) , mzi_write=widgets.FloatSlider(min=-max_power,max=max_power,step=0.5,value = mzi1, description='MZI 1 (mW)' , style=style) )

    mzi2 = heater_heat('mzi 2')
    mzi2_slider = interactive( mzi, n=fixed(2), mzi_write=widgets.FloatSlider(min=-max_power,max=max_power,step=0.5,value = mzi2, description='MZI 2 (mW)' , style=style) )
    
    mzi3 = heater_heat('mzi 3')
    mzi3_slider = interactive( mzi, n=fixed(3), mzi_write=widgets.FloatSlider(min=-max_power,max=max_power,step=0.5,value = mzi3, description='MZI 3 (mW)' , style=style) )

    
    mzi4 = heater_heat('mzi 4')
    mzi4_slider = interactive( mzi, n=fixed(4), mzi_write=widgets.FloatSlider(min=-max_power,max=max_power,step=0.5,value = mzi4, description='MZI 4 (mW)' , style=style) )

except:
    pass


def heater_temp(heater, target=-1000):
    
    if target == -1000: # user has not set a target temperature value, thus read the actual temperature
        temp_actual = board_io( '%s actual' %heater)[-1]
        temp_actual = float ( re.findall('[0-9-.]+' , temp_actual)[0]) # extract the temperature from the returned string
        return temp_actual
    elif target == -500: # user has not set a target temperature value, thus read the average actual temperature
        temp_average_actual = board_io( '%s average_actual' %heater)[-1]
        temp_average_actual = float ( re.findall('[0-9-.]+' , temp_average_actual)[0]) # extract the temperature from the returned string
        return temp_average_actual
    elif target == -200:# user wants to read the target temperature
        temp_target = board_io( '%s target' %heater)[-1]
        temp_target = float ( re.findall('[0-9-.]+' , temp_target)[0]) # extract the temperature from the returned string
        return temp_target
    else:# set the  target temperature
        board_io( '%s target %s' %(heater,target))

def heater_heat(heater, target=-1000):
    
    if target == -1000: # user has not set a heat value, thus read the heat applied
        heat_actual = board_io( '%s heat' %heater)[-1]
        heat_actual = float ( re.findall('[0-9-.]+' , heat_actual)[0]) # extract the temperature from the returned string
        return heat_actual
    else: # user has set a heat value
        board_io( '%s heat %s' %(heater,target))
    
    
def set_bench_mode(mode=''):
    if mode == '': # return the current mode
        return re.findall('[a-z-.]+' ,board_io('bench mode')[-1] )[1]
    
    print('Bench mode: %s' %mode)
    
    mode = {'Off':'off', 'Temperature':'temperature', 'Wavelength':'wavelength', 'Split':'split'}[mode]
    board_io( 'bench mode %s' %mode)
    

def set_bench_scale(scale, phi, mod_frequency, mod_gain, mod_averaging):
    board_io( 'bench2 scale %s' %scale)
    board_io( 'dac phi %s' %phi)
    board_io('mod enable %s %s %s' %(mod_frequency, mod_gain, mod_averaging))
    

def heater_state(heater):
    
    return re.findall('\=(.*)', board_io( '%s enable' %heater)[-1])[0]
    
def enable_heater(state, heater):
    board_io( '%s enable %s' %(heater,state))

# Read the available SN# Build# Options from the .txt file they are stored
DUT_options_pd, DUT_options, DUT_options_labels = import_data(folder_address= initialization_folder_address, file_labels=['Available SN# Build# Options_v2'], file_name= '', header_rows=1, numeric=0, concatenate=0)
SN = [SN_option[0] for SN_option in DUT_options[list(DUT_options.keys())[0]]]
Build = [SN_option[1] for SN_option in DUT_options[list(DUT_options.keys())[0]]]
Phase = [SN_option[2] for SN_option in DUT_options[list(DUT_options.keys())[0]]]
SN_options = [SN_option + ' ' + Build_option for (SN_option, Build_option) in zip(SN, Build)]

try:
    SN_current = SN_selected
except:
    SN_current = SN_options[-1] # choose the last one

def SN_widget(x):
    print('%s is selected' %x)
    return x

SN_selector = interactive(SN_widget, x = widgets.Dropdown( options = SN_options, value = SN_current,  description='SN:'))


test_options_pd, test_options, test_options_labels = import_data(folder_address= initialization_folder_address, file_labels=['Available test Options'], file_name= '', header_rows=1, numeric=0, concatenate=0)
test_types = [test_option[0] for test_option in test_options[list(test_options.keys())[0]]]

try:
    test_type_current = test_type_selected
except:
    test_type_current = test_types[-1] # choose the last one

def test_type_widget(x):
    print('%s is selected' %x)
    return x

test_type_selector = interactive(test_type_widget, x = widgets.Dropdown( options = test_types, value = test_type_current,  description='Test Type:'))

def temperature_widget(x):
    print('%s °C is selected' %x)
    return x

temperature_selector = interactive(temperature_widget, x = widgets.Dropdown( options = ['21','25','37','50'], value = '21',  description='Temperature (C:'))


def axis_widget(x):
    print('%s is selected as x-axis' %x)
    return x

def port(x):
    print('%s is selected' %x)
    return x

# Port for the device
try :
    # find the COM ports in use on this PC
    comport_devices = [comport.device for comport in serial.tools.list_ports.comports()]
    port_manufacturers = [comport.manufacturer for comport in serial.tools.list_ports.comports()]
    
    # if any COM is in use, guess that the first one that has FTDI as its manufacturer is the default one
    for (comport_device, port_manufacturer) in zip(comport_devices, port_manufacturers):
        if 'FTDI' in port_manufacturer:
            print('%s is a FTDI device' %comport_device)
            default_port = comport_device
            break
except:
    comport_devices = ['COM1', 'COM2', 'COM3', 'COM4', 'COM5', 'COM6']
    default_port = 'COM3'
    print('Could not find a FTDI port, selected %s by default, may not be an available port' %default_port)
        

port_selector = interactive(port, x = widgets.Dropdown( options = comport_devices, value = default_port,  description='PORT:'))

NameError: name 'import_data' is not defined

# Initialize Smart Board

In [32]:
def report_laser_properties():
    
    SN_properties = []
    for SN_selected in SN_options:
        SN_init_file_name = '%s initialization' %SN_selected
        data_pd, data, data_labels = import_data(folder_address= initialization_folder_address, file_labels=[SN_init_file_name], file_name= '', header_rows=2, numeric=0, concatenate=0)

        SN_parameters = [re.findall( '(.*)\_\*', data_label)[0] for data_label in data_labels[SN_init_file_name]]
        SN_parameter_commands = [re.findall( '\*(.*)\*', data_label)[0] for data_label in data_labels[SN_init_file_name]]
        SN_properties.append([SN_selected] + list(data[SN_init_file_name][0]) )
    laser_properties_pd = pd.DataFrame ( SN_properties, columns = ['SN'] + (SN_parameters)) 
    export_pd( laser_properties_pd, file_name='All SN initilization parameters', folder_address = output_folder_address, index=False, header=True, date_label='')

In [143]:
#laser_properties_pd

,SN,RTD R_RT (ohms),RTD TCR (ohms/C),Comb Heater Resistance (ohms),FSCS Coarse Resistance (ohms),FSCS Thermistor,Bench Thermistor,k_bench_p,k_bench_i,k_bench_d,k_bench_dp,k_bench_di,k_bench_dd,k_comb_p,k_comb_i,k_comb_d,k_fscs_p,k_fscs_i,k_fscs_d,bench2_scale
0,SN14 Build 4.2,635.860800,0.002100,70.0,6.0,vh10,vh10,2.8,0.05,0.9,0.5,0.5,0.01,4.0,3.2,0.002,6.0,3.0,3.0,NaN
1,SN16 Build 5.1,639.907324,0.002038,70.0,6.0,vh10,vh10,2.2,1.50,0.7,0.5,0.5,0.01,4.0,3.2,0.002,6.0,3.0,3.0,NaN
2,SN17 Build 6.1,629.592411,0.002046,70.0,6.0,vh10,vh10,2.0,0.20,1.0,0.5,0.5,0.01,4.0,3.2,0.100,6.0,3.0,3.0,NaN
3,SN23 Build 8.1,637.760509,0.002149,70.0,6.0,vh10,vh10,1.2,0.10,0.6,0.5,0.5,0.01,4.0,3.2,0.002,6.0,3.0,3.0,-0.05
4,SN24 Build 8.2,647.161052,0.002130,70.0,6.0,vh10,vh10,1.2,0.10,0.6,0.5,0.5,0.01,4.0,3.2,0.002,6.0,3.0,3.0,-0.05
5,SN25 Build 9.1,631.868713,0.002610,70.0,6.0,vh10,vh10,1.2,0.10,0.6,0.5,0.5,0.01,4.0,3.2,0.002,6.0,3.0,3.0,-0.05
6,SN26 Build 9.2,638.126285,0.002569,70.0,6.0,vh10,vh10,1.2,0.10,0.6,0.5,0.5,0.01,4.0,3.2,0.002,6.0,3.0,3.0,-0.05
7,SN27 Build 10.1,634.553650,0.002149,70.0,6.0,vh10,vh10,1.2,0.10,0.6,0.5,0.5,0.01,4.0,3.2,0.002,6.0,3.0,3.0,-0.05
8,SN28 Build 10.2,707.614136,0.002133,70.0,6.0,vh10,vh10,1.2,0.10,0.6,0.5,0.5,0.01,4.0,3.2,0.002,6.0,3.0,3.0,-0.05
9,SN29 Build 11.1,696.449890,0.002268,70.0,6.0,vh10,vh10,1.2,0.10,0.6,0.5,0.5,0.01,4.0,3.2,0.002,6.0,3.0,3.0,-0.05


In [1]:
def initialize_laser(SN_selected):
    
    print( '%s is selected' %SN_selected)
    

    # Create an initialization file for this SN, if it does not already exist
    SN_init_file_name = '%s initialization' %SN_selected
    if not SN_init_file_name +'.txt' in os.listdir (initialization_folder_address):
        SN_init_file_address = shutil.copy(initialization_folder_address+ 'SN# Build# Initialization Template.txt' , initialization_folder_address + SN_init_file_name +'.txt')
        print('Initialization file of %s is created at %s' %(SN_selected, SN_init_file_address))
    else:
        print('Initialization file of %s already exists' %SN_selected)
        
    for operating_temp in [21,25,37,50]:
        # Create a calibration file for this SN, if it does not already exist
        SN_cal_file_name = '%s_SIP calibration_bench %sC' %(SN_selected, operating_temp)
        if not SN_cal_file_name +'.xlsx' in os.listdir (calibration_folder_address):
            SN_cal_file_address = shutil.copy(calibration_folder_address+ 'SN# Build#_SIP calibration_bench %sC Template.xlsx' %operating_temp , calibration_folder_address + SN_cal_file_name +'.xlsx')
            print('Calibration file of %s is created at %s' %(SN_selected, SN_cal_file_address))
            print('Remember to calibrate %s' %SN_selected)
        else:
            print('Calibration file of %s at %s °C already exists, ensure its accuracy' %(SN_selected, operating_temp))

    # Initialize SN, import parameters
    SN_init_file_name = '%s initialization' %SN_selected
    data_pd, data, data_labels = import_data(folder_address= initialization_folder_address, file_labels=[SN_init_file_name], file_name= '', header_rows=2, numeric=0, concatenate=0)
    
    SN_parameters = [re.findall( '(.*)\_\*', data_label)[0] for data_label in data_labels[SN_init_file_name]]
    SN_parameter_commands = [re.findall( '\*(.*)\*', data_label)[0] for data_label in data_labels[SN_init_file_name]]
    
    # Create an output folder for this SN
    if SN_selected not in os.listdir (output_folder_address):

        SN_selected_output_folder_address = shutil.copytree(output_folder_address+ 'SN# Build# Output Folder Template' , output_folder_address + SN_selected)
        print('Output folder of %s is created at %s' %(SN_selected, SN_selected_output_folder_address))
    else:
        print('Output folder of %s already exists' %SN_selected)
    
    for (command,value) in zip(SN_parameter_commands, data[SN_init_file_name][0] ):
        print ('%s: %s' %(command, value))
        board_io(command +' %s' %value)        
    
    print(data_labels[SN_init_file_name])
    
#     k_benchSN = [float(data_pd[file_label]['k_bench_p'].iloc[0,0]), float(data_pd[file_label]['k_bench_i'].iloc[0,0]), float(data_pd[file_label]['k_bench_d'].iloc[0,0])]
#     k_comb_SN = [float(data_pd[file_label]['k_comb_p'].iloc[0,0]), float(data_pd[file_label]['k_comb_i'].iloc[0,0]), float(data_pd[file_label]['k_comb_d'].iloc[0,0])]
#     k_fscs_SN = [float(data_pd[file_label]['k_fscs_p'].iloc[0,0]), float(data_pd[file_label]['k_fscs_i'].iloc[0,0]), float(data_pd[file_label]['k_fscs_d'].iloc[0,0])]

#     # set the default bench, COM5, FSCS loop pid values
#     board_io( 'bench2 pid %s %s %s' %(tuple(k_benchSN)))
#     board_io( 'comb pid %s %s %s' %(tuple(k_comb_SN)))
#     board_io( 'fscs pid %s %s %s' %(tuple(k_fscs_SN)))

    # Enable ac modulation
    board_io( 'dac phi 2'); board_io( 'mod enable 1000 127 255')
    #board_io( 'bench2 scale -0.005') READ THIS FROM THE INITIALIZATION FILE INSTEAD
    
    
    # Default mpd bias
    board_io('dac mpd_bias 0.5')
    
    # Perform GC scan to obtain the necessary voltage needed for each GC range
    board_io('gc scan %d %d' %(max_gc_current, max_gc_current))
    
    # average temperature smoothing parameter
    board_io('bench2 smoothing_actual 0.02')
    
    
    # To read any initial extra/undesired output sent by the board
    board_io('')
    board_io('')
    
    return data_pd, data_labels

In [33]:
def save_laser_parameters(SN):
    
    # Initialize SN, import parameters
    SN_init_file_name = '%s initialization' %SN
    data_pd, data, data_labels = import_data(folder_address = initialization_folder_address, file_labels=[SN_init_file_name], file_name= '', header_rows=2, numeric=0, concatenate=0 )
    
    SN_parameters = [re.findall( '(.*)\_\*', data_label)[0] for data_label in data_labels[SN_init_file_name] ]
    SN_parameter_commands = [re.findall( '\*(.*)\*', data_label)[0] for data_label in data_labels[SN_init_file_name] ]
    values = []
    
    for command in SN_parameter_commands:
        
        if command in ['bench2 thermistor', 'fscs thermistor']: # those that may return non-digit values
            # find what is returned after '=' for instance in 'OK thermistor=vh10' return 'vh10'
            value = re.findall('(?<==)[\w]+' , board_io(command)[-1] )[0]
        else : 
            # find digits in the output
            value = re.findall('[0-9-.]+' , board_io(command)[-1] )[0]
            
        print ('%s: %s' %(command, value))
        values.append(value)
    
    SN_parameter_commands_to_export = ['*' + SN_parameter_command + '*' for SN_parameter_command in SN_parameter_commands]
    index = pd.MultiIndex.from_arrays([SN_parameters, SN_parameter_commands_to_export])
    values_pd = pd.DataFrame([values], columns=index)
    
    #print (index)
    #print(values)
    if SN_init_file_name +'.txt' in os.listdir (initialization_folder_address):
        shutil.move(initialization_folder_address + SN_init_file_name +'.txt' , initialization_folder_address + SN_init_file_name + '_backup.txt' )

                        
    export_pd( values_pd, file_name=SN_init_file_name, folder_address = initialization_folder_address, index=False, header=True, date_label='')


# Terminate connection with the SmartBoard

In [34]:
def disconnect_smartboard():
    
    try:

        # Turn off the GC
        gc_read = gain_current(0)
        print('Gain Current is %.3f mA' %gc_read)

        # Disable the bench
        set_bench_mode('temp') # go to the thermal control mode first
        bench_disabled = board_io( 'bench2 enable off')
        print('bench2 TEC is disabled')

        # Disable the comb heater
        comb_heater_disabled = board_io( 'comb enable off')
        heater_heat('comb', 0)
        print('Comb heater is disabled')

        # Disable the FSCS heater
        fscs_heater_disabled = board_io( 'fscs enable off')
        heater_heat('fscs', 0)
        print('FSCS heater is disabled')

        # Disable the modulation
        board_io( 'mod disable')
        
        # turn off tps/mzi heaters
        board_io('tps off')
        print('MZIs are turned off')

        smartboard.close()
        print('SmartBoard is disconnected')
    
    except:
        print('Is Smartboard connected?')
    

interact_shutdown = interact_manual.options(manual_name="Disconnect Smart Board"); 
disconnect_board_button = interact_shutdown(disconnect_smartboard)

interactive(children=(Button(description='Disconnect Smart Board', style=ButtonStyle()), Output()), _dom_class…